<a href="https://colab.research.google.com/github/grayox/Auto1111SDK/blob/main/story_writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## References


- [worldbuilding_app_v2.ipynb](https://colab.research.google.com/drive/1MMo7GSXEnkxGvOVwqcvb-vwwGs6hF4LN)
- [worldbuilding_app_v1.ipynb](https://colab.research.google.com/drive/14tBkG4WH27FnBIWRzBP3jjyB_TCwJI7Q)

IMDB APIs

- APIs
  - https://rapidapi.com/rapihub-rapihub-default/api/imdb-top-100-movies ⭐
  - https://rapidapi.com/dataapiman/api/imdb231 ⭐🔥
  - https://rapidapi.com/DataCrawler/api/imdb188
  - https://rapidapi.com/amrelrafie/api/movies-tv-shows-database
  - https://rapidapi.com/pierregoutheraud/api/movies-ratings2

- Not subscribed
  - https://rapidapi.com/rapidapi/api/movie-database-alternative

## Code

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -Uqr "/content/drive/My Drive/Colab Notebooks/Python Packages/ai_bookwriter/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 28.0 MB/s eta 0:00:00


In [3]:
# handle module imports
!ln -s "/content/drive/My Drive/Colab Notebooks/Python Packages/ai_bookwriter" /content/ai_bookwriter # creates symlink
!rm -f /content/ai_bookwriter/utils.py # Remove existing utils.py file (if it exists)
!jupyter nbconvert --to script /content/ai_bookwriter/utils.ipynb # defaults to .txt
!mv /content/ai_bookwriter/utils.txt /content/ai_bookwriter/utils.py # rename to .py

[NbConvertApp] Converting notebook /content/ai_bookwriter/utils.ipynb to script
[NbConvertApp] Writing 43061 bytes to /content/ai_bookwriter/utils.txt


In [8]:
import os, io, yaml, json, csv, re, random, names

# from ebooklib import epub
from enum import Enum
from time import sleep
from dataclasses import dataclass
from typing import Tuple, List, Dict, Optional, Any, Union, TypedDict, Literal
from ai_bookwriter.utils import SaveDestination, GeminiLLM, GithubManager
from ai_bookwriter.utils import Logger, logger, logging, setup_config, get_caller_name
from ai_bookwriter.utils import save_to_file, get_and_set_environment_variables

# <entropic_creation>

class ArcShape(Enum):
    FLAT = "FLAT - No arc"
    RISE = "RISE arc, like 'Rags to riches'"
    FALL = "FALL arc, like 'Tragedy' or 'Riches to rags'"
    FALL_RISE = "FALL_RISE arc, like Kurt Vonnegut's 'Man in a hole'"
    RISE_FALL = "RISE_FALL arc, like 'Icarus'"
    RISE_FALL_RISE = "RISE_FALL_RISE arc, like 'Cinderella'"
    FALL_RISE_FALL = "FALL_RISE_FALL arc, like 'Oedipus'"

class Character:
    """
        A class representing a character

        Attributes:
            first_name (str): The first name of the character.
            last_name (str): The last name of the character.
            name (str): The full name of the character.
            full_name (str): The full name of the character.
            gender (str): The gender of the character ('male', 'female').
            personality (str): The personality type of the character.
            mtbi (str): The Myers-Briggs Type Indicator of the character.

        # Example usage:
        my_character = Character()
        print(f"Name: {my_character.name}")
        print(f"Gender: {my_character.gender}")
        print(f"Personality: {my_character.personality}")
        my_female_character = Character("female")
        print(f"Name: {my_female_character.name}")
        print(f"Gender: {my_female_character.gender}")
        print(f"Personality: {my_female_character.personality}")
        my_male_character = Character("male")
        print(f"Name: {my_male_character.name}")
        print(f"Gender: {my_male_character.gender}")
        print(f"Personality: {my_male_character.personality}")

        # Example usage:
          random:
            character = Character(gender="female", primary_archetype=random.choice(archetype_list))

          inference time assignment:
            character = Character(gender="male", primary_archetype="Hero", secondary_archetypes=["Lover", "Caregiver"])

          rules based:
            if world_essence == "magic is fading from existence":
                primary_archetype = "Mage"
            elif world_essence == "technology has become sentient":
                primary_archetype = "Hacker"
        """

    PERSONALITIES = {  # Gist IDs
        "A": "Assertive, LOW neurotic behavior.",
        "T": "Turbulent: HIGH neurotic behavior.",
        "Analysts": "Analysts | Intuitive (N) and Thinking (T) personality types, known for their rationality, impartiality, and intellectual excellence.",
        "Diplomats": "Diplomats | Intuitive (N) and Feeling (F) personality types, known for their empathy, diplomatic skills, and passionate idealism.",
        "Sentinels": "Sentinels | Observant (S) and Judging (J) personality types, known for their practicality and focus on order, security, and stability.",
        "Explorers": "Explorers | Observant (S) and Prospecting (P) personality types, known for their spontaneity, ingenuity, and flexibility.",
        # Analysts
        "INTJ": {
            # Architect
            "name": "Architect",
            "class": "Analysts",
            "description": "Architects are imaginative and strategic thinkers, with a plan for everything.",
            "insight": "Architects are the most likely personality type to consciously try limiting the influence others have on them.",
            "gist_id": "840f57770d59135bef334f8730f64fa1",
            },
        "INTP": {
            # Logician
            "name": "Logician",
            "class": "Analysts",
            "description": "Logicians are innovative inventors with an unquenchable thirst for knowledge.",
            "insight": "Logicians are the most likely personality type to leave mentally challenging tasks for the evening.",
            "gist_id": "3e3359901c43e7faf7b1582db289245c",
            },
        "ENTJ": {
            # Commander
            "name": "Commander",
            "class": "Analysts",
            "description": "Commanders are bold, imaginative, and strong-willed, always finding a way – or making one.",
            "insight": "Commanders are the least likely personality type to want their leaders to be humble.",
            "gist_id": "352addaf3ee5023ec9925ebda5b42076",
            },
        "ENTP": {
            # Debater
            "name": "Debater",
            "class": "Analysts",
            "description": "Debaters are curious and flexible thinkers who cannot resist an intellectual challenge.",
            "insight": "Debaters are the most likely personality type to assume a large percentage of online content surrounding popular topics is created by trolls.",
            "gist_id": "22bc24452e91daf686c915ad26a7e44a",
            },
        # Diplomats
        "INFJ": {
            # Advocate
            "name": "Advocate",
            "class": "Diplomats",
            "description": "Advocates are quiet visionaries, often serving as inspiring and tireless idealists.",
            "insight": "Advocates are the most likely personality type to often feel too busy.",
            "gist_id": "2e47bac285a88fbcd43a095e0bdcc516",
            },
        "INFP": {
            # Mediator
            "name": "Mediator",
            "class": "Diplomats",
            "description": "Mediators are poetic, kind, and altruistic people, always eager to help a good cause.",
            "insight": "Mediators are the most likely personality type to say negative emotions affect their ability to think clearly.",
            "gist_id": "c01dfcf22464216dbb184db33a6dd995",
            },
        "ENFJ": {
            # Protagonist
            "name": "Protagonist",
            "class": "Diplomats",
            "description": "Protagonists are inspiring optimists, readily taking action to do what they feel is right.",
            "insight": "Protagonists are the most likely personality type to say helping others gives them purpose.",
            "gist_id": "d6608681e43a6ffad98f03995019e7f7",
            },
        "ENFP": {
            # Campaigner
            "name": "Campaigner",
            "class": "Diplomats",
            "description": "Campaigners are enthusiastic, creative, and sociable free spirits, who can always find a reason to smile.",
            "insight": "Campaigners are the most likely personality type to enjoy playing a role on a television show as a job.",
            "gist_id": "d1245a5ce32e668e79697a9c692c7af3",
            },
        # Sentinels
        "ISTJ": {
            # Logistician
            "name": "Logistician",
            "class": "Sentinels",
            "description": "Logisticians are practical and fact-minded individuals, whose reliability cannot be doubted.",
            "insight": "Logisticians are the least likely personality type to like sampling new and different foods.",
            "gist_id": "7bb57e516d571b0e6d9542294e92e8fd",
            },
        "ISFJ": {
            # Defender
            "name": "Defender",
            "class": "Sentinels",
            "description": "Defenders are very dedicated and warm protectors, always ready to defend their loved ones.",
            "insight": "Defenders are the least likely personality type to be suspicious of people who always follow rules.",
            "gist_id": "b60e07e82bbb7607c161ff5420f73ccb",
            },
        "ESTJ": {
            # Executive
            "name": "Executive",
            "class": "Sentinels",
            "description": "Executives are excellent organizers, unsurpassed at managing things – or people.",
            "insight": "Executives are the most likely personality type to always run with the same level of intensity.",
            "gist_id": "3ba04cb2e9346cb3868b20c05a56026d",
            },
        "ESFJ": {
            # Consul
            "name": "Consul",
            "class": "Sentinels",
            "description": "Consuls are very caring, social, community-minded people who are always eager to help.",
            "insight": "Consuls are the least likely personality type to enjoy watching foreign films, even with subtitles.",
            "gist_id": "2d53f6d6cf73ccd12fe2817853a034b3",
            },
        # Explorers
        "ISTP": {
            # Virtuoso
            "name": "Virtuoso",
            "class": "Explorers",
            "description": "Virtuosos are innovative and practical experimenters, masters of all kinds of tools.",
            "insight": "Virtuosos are the most likely personality type to say there are certain people whom they respect that they wouldn’t want to have a conversation with.",
            "gist_id": "1b490bbd0cb84410a1dae304b7cc49bf",
            },
        "ISFP": {
            # Adventurer
            "name": "Adventurer",
            "class": "Explorers",
            "description": "Adventurers are flexible and charming, always ready to explore and experience something new.",
            "insight": "Adventurers are the least likely personality type to consider becoming a politician.",
            "gist_id": "f814ac720589dca5bf525ff2cc11189e",
            },
        "ESTP": {
            # Entrepreneur
            "name": "Entrepreneur",
            "class": "Explorers",
            "description": "Entrepreneurs are savvy, energetic, and very perceptive people who truly enjoy living on the edge.",
            "insight": "Entrepreneurs are the least likely personality type to think a lot about the regrets they have.",
            "gist_id": "2fec76045625d70c09cdc9304ee0bdcd",
            },
        "ESFP": {
            # Entertainer
            "name": "Entertainer",
            "class": "Explorers",
            "description": "Entertainers are spontaneous, energetic, and enthusiastic people – life is never boring around them.",
            "insight": "Entertainers are the most likely personality type to say they’d spend more money for the sake of convenience if they could.",
            "gist_id": "f211424ef56380ea56bc0506d93fc41d",
            }
        }

    ASPIRATIONS = [
        "To become a renowned artist",
        "To find true love",
        "To travel the world",
        "To start a successful business",
        "To make a groundbreaking scientific discovery",
        "To become a political leader",
        "To win an Olympic gold medal",
        "To write a bestselling novel",
        "To become a famous musician",
        "To solve a long-standing mystery",
        "To save lives as a doctor",
        "To bring peace to a war-torn region",
        "To restore a family's lost fortune",
        "To become a revered spiritual leader",
        "To revolutionize an industry",
        ]

    MOTIVATIONS = [
        "To inspire and motivate others",
        "To challenge the status quo",
        "Desire for recognition and fame",
        "Seeking redemption for past mistakes",
        "Protecting loved ones",
        "Overcoming personal limitations",
        "Proving critics wrong",
        "Escaping poverty",
        "Fulfilling a promise to a dying loved one",
        "Uncovering the truth about a personal mystery",
        "Avenging a wrong done to them or their family",
        "Restoring honor to their family name",
        "Achieving immortality through their work",
        "Making the world a better place",
        "Satisfying their curiosity about the unknown",
        "Gaining power and influence",
        "Finding inner peace and enlightenment",
        ]

    FEARS = [
        "Fear of being a stranger",
        "Fear of failure",
        "Fear of being alone",
        "Fear of the dark",
        "Fear of heights",
        "Fear of public speaking",
        "Fear of losing control",
        "Fear of death",
        "Fear of intimacy",
        "Fear of change",
        "Fear of being forgotten",
        "Fear of poverty",
        "Fear of the supernatural",
        "Fear of betrayal",
        "Fear of water",
        "Fear of being exposed as a fraud",
        ]

    PAST_EVENTS = [ # shaping the character's world view
        "Losing a parent at a young age",
        "Surviving a natural disaster",
        "Witnessing a crime",
        "Being betrayed by a close friend",
        "Experiencing a near-death situation",
        "Growing up in extreme poverty",
        "Being raised in multiple foster homes",
        "Winning a prestigious award or competition",
        "Suffering from a life-altering injury",
        "Discovering a hidden talent",
        "Being falsely accused of a crime",
        "Experiencing a religious or spiritual awakening",
        "Living through a war or conflict",
        "Making a decision that led to unintended consequences",
        "Uncovering a family secret",
        "Being caught in a dark conspiracy",
        "Being raised in a cult",
        ]

    TRAITS = {
        "Archetypes": ["Reluctant Hero", "Charismatic Anti-hero", "Wise Mentor", "Femme Fatale", "Boy/Girl Next Door", "Mad Scientist", "Grizzled Veteran", "Innocent Child", "Corrupt Politician", "Visionary Leader"],
        "Backgrounds": ["Ex-soldier", "Orphaned Prodigy", "Disgraced Nobility", "Amnesiac", "Time Traveler", "Clone", "Retired Assassin", "Undercover Alien", "Reincarnated Spirit", "Sentient AI"],
        "Skills": ["Master Hacker", "Psychic Abilities", "Martial Arts Expert", "Genius Inventor", "Shapeshifter", "Empath", "Polyglot", "Precognitive", "Master of Disguise", "Telekinetic"],
        "Flaws": ["Crippling Self-Doubt", "Addiction", "Chronic Illness", "Dark Secret", "Vengeful Nature", "Pathological Liar", "Cowardice", "Overconfidence", "Trust Issues", "Selective Amnesia",]
        }

    ARCHETYPES_AUXILIARY = [
        "Hero", "Anti-hero", "Chosen One", "Rebel", "Outcast", "Mentor", "Villain", "Protagonist",
        "Reluctant Hero", "Charismatic Anti-hero", "Rebellious Outcast", "Wise Mentor", "Tragic Villain", "Femme Fatale", "Lovable Rogue", "Child Prodigy",
        "Jaded Veteran", "Innocent Newcomer", "Mad Scientist", "Corrupt Politician", "Visionary Leader",
        "Shapeshifter", "Empath", "Polyglot", "Precognitive", "Master of Disguise", "Telekinetic", "Hardboiled Detective", "Detective",
        "Master Hacker", "Psychic Abilities", "Time Traveler", "Clone", "Mythical Creature", "Artificial Intelligence", "Alien",
        ]

    # Character Archetypes
    ARCHETYPES = [
      {
        "id": "01",
        "name": "The Hero",
        "traits": [
          "Courageous, selfless, driven by a strong moral compass",
          "Often embarks on a quest or faces a challenge that tests their abilities and resolve"
        ],
        "examples": [
          "Luke Skywalker (Star Wars)",
          "Harry Potter (Harry Potter)",
          "Katniss Everdeen (The Hunger Games)"
        ]
      },
      {
        "id": "02",
        "name": "The Anti-hero",
        "traits": [
          "Possesses flaws and may operate outside conventional morality",
          "Often driven by selfish motives or a cynical worldview",
          "Can still be compelling and even heroic in their own way"
        ],
        "examples": [
          "Han Solo (Star Wars)",
          "Severus Snape (Harry Potter)",
          "Walter White (Breaking Bad)"
        ]
      },
      {
        "id": "03",
        "name": "The Mentor",
        "traits": [
          "Wise, experienced, offers guidance and support to the protagonist",
          "Often has a past shrouded in mystery or possesses unique knowledge"
        ],
        "examples": [
          "Obi-Wan Kenobi (Star Wars)",
          "Dumbledore (Harry Potter)",
          "Gandalf (The Lord of the Rings)"
        ]
      },
      {
        "id": "04",
        "name": "The Trickster",
        "traits": [
          "Clever, mischievous, often disrupts the status quo",
          "Can be a source of both chaos and wisdom"
        ],
        "examples": [
          "Loki (Norse Mythology, Marvel Cinematic Universe)",
          "Fred and George Weasley (Harry Potter)",
          "The Joker (Batman)"
        ]
      },
      {
        "id": "05",
        "name": "The Villain",
        "traits": [
          "Opposes the hero, driven by selfish desires or a desire to dominate",
          "Often embodies the story's central conflict"
        ],
        "examples": [
          "Darth Vader (Star Wars)",
          "Voldemort (Harry Potter)",
          "The Wicked Witch of the West (The Wizard of Oz)"
        ]
      },
      {
        "id": "06",
        "name": "The Lover",
        "traits": [
          "Passionate, romantic, seeks love and connection",
          "Often faces challenges in their pursuit of love"
        ],
        "examples": [
          "Romeo and Juliet (Romeo and Juliet)",
          "Elizabeth Bennet and Mr. Darcy (Pride and Prejudice)"
        ]
      },
      {
        "id": "07",
        "name": "The Creator",
        "traits": [
          "Artistic, imaginative, driven to express themselves through their work",
          "Often faces self-doubt or struggles with the creative process"
        ],
        "examples": [
          "Shakespeare (real life)",
          "Da Vinci (real life)",
          "Holden Caulfield (The Catcher in the Rye)"
        ]
      },
      {
        "id": "08",
        "name": "The Ruler",
        "traits": [
          "Powerful, authoritative, responsible for leading others",
          "Often faces challenges to their power or struggles with the burden of leadership"
        ],
        "examples": [
          "Aragorn (The Lord of the Rings)",
          "Queen Elizabeth I (real life)",
          "Daenerys Targaryen (Game of Thrones)"
        ]
      },
      {
        "id": "09",
        "name": "The Innocent",
        "traits": [
          "Pure, naive, often unaware of the dangers of the world",
          "Can represent hope or a fresh perspective"
        ],
        "examples": [
          "Scout Finch (To Kill a Mockingbird)",
          "Dorothy Gale (The Wizard of Oz)",
          "Frodo Baggins (The Lord of the Rings)"
        ]
      },
      {
        "id": "10",
        "name": "The Caregiver",
        "traits": [
          "Nurturing, compassionate, dedicated to helping others",
          "Often puts the needs of others before their own"
        ],
        "examples": [
          "Molly Weasley (Harry Potter)",
          "Atticus Finch (To Kill a Mockingbird)"
        ]
      },
      {
        "id": "11",
        "name": "The Everyman (or Everywoman)",
        "traits": [
          "Ordinary, relatable, represents the average person",
          "Often serves as a point of identification for the reader"
        ],
        "examples": [
          "Bilbo Baggins (The Hobbit)",
          "Samwise Gamgee (The Lord of the Rings)",
          "Winston Smith (1984)"
        ]
      },
      {
        "id": "12",
        "name": "The Jester/Fool",
        "traits": [
          "Lighthearted, humorous, often provides comic relief",
          "Can also offer a unique perspective or challenge authority through wit and humor"
        ],
        "examples": [
          "R2-D2 and C-3PO (Star Wars)",
          "Mercutio (Romeo and Juliet)",
          "Tyrion Lannister (Game of Thrones)"
        ]
      },
      {
        "id": "13",
        "name": "The Sage",
        "traits": [
          "Knowledgeable, introspective, often seeks or dispenses wisdom",
          "May struggle with applying their knowledge to real-world situations"
        ],
        "examples": [
          "Yoda (Star Wars)",
          "Hermione Granger (Harry Potter)",
          "Mr. Miyagi (The Karate Kid)"
        ]
      },
      {
        "id": "14",
        "name": "The Explorer",
        "traits": [
          "Adventurous, curious, seeks new experiences and knowledge",
          "Often driven by a desire to discover the unknown"
        ],
        "examples": [
          "Indiana Jones (Indiana Jones series)",
          "Dora the Explorer (Dora the Explorer)",
          "Captain James T. Kirk (Star Trek)"
        ]
      },
      {
        "id": "15",
        "name": "The Rebel",
        "traits": [
          "Challenges authority and societal norms",
          "Often fights against injustice or oppression"
        ],
        "examples": [
          "Katniss Everdeen (The Hunger Games)",
          "V (V for Vendetta)",
          "Ferris Bueller (Ferris Bueller's Day Off)"
        ]
      },
      {
        "id": "16",
        "name": "The Outcast",
        "traits": [
          "Alienated from society, often misunderstood or rejected",
          "May struggle with finding acceptance or their place in the world"
        ],
        "examples": [
          "Quasimodo (The Hunchback of Notre Dame)",
          "Beast (Beauty and the Beast)",
          "Edward Scissorhands (Edward Scissorhands)"
        ]
      },
      {
        "id": "17",
        "name": "The Shapeshifter",
        "traits": [
          "Changes allegiances or personality, often unpredictable",
          "Can create tension and suspense in the story"
        ],
        "examples": [
          "Mystique (X-Men)",
          "Gollum/Sméagol (The Lord of the Rings)",
          "Severus Snape (Harry Potter)"
        ]
      },
      {
        "id": "18",
        "name": "The Guardian",
        "traits": [
          "Protective, loyal, often stands between the protagonist and danger",
          "May sacrifice themselves for the greater good"
        ],
        "examples": [
          "Hagrid (Harry Potter)",
          "Samwise Gamgee (The Lord of the Rings)",
          "John Coffey (The Green Mile)"
        ]
      },
      {
        "id": "19",
        "name": "The Femme Fatale",
        "traits": [
          "Seductive, dangerous, often manipulates others for personal gain",
          "Can be a complex character with hidden depths"
        ],
        "examples": [
          "Catwoman (Batman)",
          "Mata Hari (real life)",
          "Jessica Rabbit (Who Framed Roger Rabbit)"
        ]
      },
      {
        "id": "20",
        "name": "The Catalyst",
        "traits": [
          "Instigates change or action in other characters",
          "Often sets the main plot in motion"
        ],
        "examples": [
          "Gandalf (The Hobbit)",
          "Effie Trinket (The Hunger Games)",
          "Morpheus (The Matrix)"
        ]
      },
      {
        "id": "21",
        "name": "The Shadow",
        "traits": [
          "Represents the darker aspects of human nature or the protagonist's psyche",
          "Often serves as a foil to the hero"
        ],
        "examples": [
          "Mr. Hyde (Dr. Jekyll and Mr. Hyde)",
          "Tyler Durden (Fight Club)",
          "Darth Vader (Star Wars)"
        ]
      },
      {
        "id": "22",
        "name": "The Martyr",
        "traits": [
          "Self-sacrificing, often dies for a cause or to save others",
          "Inspires others through their actions"
        ],
        "examples": [
          "Sydney Carton (A Tale of Two Cities)",
          "Aslan (The Chronicles of Narnia)",
          "Tony Stark/Iron Man (Avengers: Endgame)"
        ]
      },
      {
        "id": "23",
        "name": "The Scapegoat",
        "traits": [
          "Blamed or punished for the mistakes of others",
          "Often reveals societal injustices or prejudices"
        ],
        "examples": [
          "Tom Robinson (To Kill a Mockingbird)",
          "Shirley Sherrod (real life)",
          "John Proctor (The Crucible)"
        ]
      },
      {
        "id": "24",
        "name": "The Zealot",
        "traits": [
          "Fanatically devoted to a cause, belief, or person",
          "Can be a source of conflict or drive the plot forward"
        ],
        "examples": [
          "Javert (Les Misérables)",
          "Melisandre (Game of Thrones)",
          "Mrs. Carmody (The Mist)"
        ]
      },
      {
        "id": "25",
        "name": "The Everyman Hero",
        "traits": [
          "An ordinary person thrust into extraordinary circumstances",
          "Often discovers hidden strengths or abilities"
        ],
        "examples": [
          "Neo (The Matrix)",
          "Emmet Brickowski (The Lego Movie)",
          "Sarah Connor (The Terminator)"
        ]
      },
      {
        "id": "26",
        "name": "The Machiavellian",
        "traits": [
          "Cunning, manipulative, and pragmatic in pursuing their goals",
          "Often willing to use any means necessary to achieve their ends",
          "May justify their actions as being for the greater good or necessary evils",
          "Skilled at reading and manipulating others' emotions and motivations",
          "Often found in positions of power or seeking to gain power",
          "Can be an antagonist, anti-hero, or even a protagonist in some cases"
        ],
        "examples": [
          "Frank Underwood (House of Cards)",
          "Cersei Lannister (Game of Thrones)",
          "Littlefinger/Petyr Baelish (Game of Thrones)"
        ]
      }]

    OCCUPATIONS = [
        "Software Engineer", "Data Scientist", "Teacher", "Doctor", "Writer",
        "Artist", "Musician", "Chef", "Architect", "Police Officer", "Firefighter",
        "Nurse", "Lawyer", "Mechanic", "Electrician", "Carpenter", "Plumber",
        "Salesperson", "Accountant", "Business Analyst", "Project Manager",
        "Graphic Designer", "Web Developer", "Marketing Specialist", "Farmer",
        "Librarian", "Social Worker", "Therapist", "Counselor", "Psychologist",
        "Veterinarian", "Zoologist", "Biologist", "Chemist", "Physicist",
        "Astronomer", "Geologist", "Historian", "Philosopher", "Journalist",
        "Politician", "Economist", "Sociologist", "Anthropologist", "Linguist",
        # Additional occupations:
        "Actor", "Dancer", "Choreographer", "Photographer", "Film Director",
        "Fashion Designer", "Interior Designer", "Landscape Architect",
        "Civil Engineer", "Aerospace Engineer", "Mechanical Engineer",
        "Environmental Scientist", "Marine Biologist", "Nutritionist",
        "Dietitian", "Pharmacist", "Dentist", "Optometrist", "Podiatrist",
        "Chiropractor", "Physical Therapist", "Occupational Therapist",
        "Speech Therapist", "Respiratory Therapist", "Radiologic Technologist",
        "Medical Technologist", "Paramedic", "EMT", "Dental Hygienist",
        "Midwife", "Audiologist", "Speech-Language Pathologist",
        "Prosthetist", "Orthotist", "Genetic Counselor",
        "Public Health Educator", "Epidemiologist", "Biostatistician",
        "Mathematician", "Statistician", "Actuary", "Operations Research Analyst",
        "Financial Analyst", "Investment Banker", "Stock Broker", "Insurance Underwriter",
        "Real Estate Agent", "Human Resources Specialist", "Training and Development Specialist",
        "Recruiter", "Payroll Clerk", "Logistics Coordinator", "Supply Chain Manager",
        "Retail Buyer", "Event Planner", "Hospitality Manager", "Restaurant Manager",
        "Hotel Manager", "Travel Agent", "Tour Guide", "Flight Attendant",
        "Pilot", "Air Traffic Controller", "Automotive Technician", "Aircraft Mechanic",
        "Ship Engineer", "Railroad Conductor", "Truck Driver", "Taxi Driver",
        "Bus Driver", "Delivery Driver", "Security Guard", "Private Investigator",
        "Correctional Officer", "Probation Officer", "Paralegal", "Notary Public",
        "Patent Attorney", "Environmental Lawyer", "Judge", "Mediator",
        "Arbitrator", "Clergy", "Religious Worker", "Philosophy Professor",
        "College Administrator", "School Principal", "Guidance Counselor",
        "Special Education Teacher", "Kindergarten Teacher", "Adult Literacy Teacher",
        "Fitness Trainer", "Athletic Coach", "Sports Official", "Umpire",
        "Refugee Coordinator", "International Aid Worker", "Diplomat", "Foreign Service Officer",
        "Translator", "Interpreter", "Technical Writer", "Copywriter",
        "Content Editor", "Broadcast News Analyst", "Radio DJ", "Podcaster",
        "Video Editor", "Animator", "Game Designer", "UX Designer",
        "Information Security Analyst", "Network Administrator", "Database Administrator",
        "IT Support Specialist", "Computer Systems Analyst", "Software Quality Assurance Tester",
        "User Experience Researcher", "Ethical Hacker", "Cybersecurity Consultant",
        "Robotics Engineer", "Artificial Intelligence Specialist", "Data Architect",
        "Machine Learning Engineer", "Big Data Engineer", "Cloud Computing Engineer",
        "DevOps Engineer", "IoT Specialist", "Blockchain Developer",
        "Cryptocurrency Analyst", "Ethical AI Developer", "Sustainability Consultant",
        "Renewable Energy Engineer", "Environmental Compliance Inspector",
        "Wildlife Biologist", "Conservation Scientist", "Forester", "Fisheries Manager",
        "Agricultural Scientist", "Food Scientist", "Viticulturist", "Brewmaster",
        "Cosmetologist", "Barber", "Makeup Artist", "Fashion Model", "Jeweler",
        "Watchmaker", "Toy Designer", "Industrial Designer", "Ceramic Artist",
        "Glassblower", "Textile Designer", "Leatherworker", "Bookbinder",
        "Paper Conservator", "Archivist", "Museum Curator", "Art Restorer",
        "Antique Dealer", "Auctioneer", "Real Estate Appraiser", "Home Stager",
        "Interior Stylist", "Personal Shopper", "Etiquette Consultant", "Life Coach",
        "Career Counselor", "Image Consultant", "Personal Trainer", "Nutrition Coach",
        "Wellness Director", "Spa Manager", "Yoga Instructor", "Meditation Guide",
        "Pet Groomer", "Dog Trainer", "Animal Behaviorist", "Aquarist",
        "Zoo Educator", "Wildlife Rehabilitator", "Pest Control Technician",
        "Veterinary Technician", "Animal Control Officer", "Pet Sitter",
        "Dog Walker", "Horse Trainer", "Equestrian Coach", "Riding Instructor",
        "Circus Performer", "Magician", "Comedian", "Voice Actor",
        "Foley Artist", "Sound Engineer", "Lighting Designer", "Set Designer",
        "Costume Designer", "Prop Maker", "Special Effects Technician", "Stunt Performer",
        "Casting Director", "Talent Agent", "Theatre Manager", "Concert Promoter",
        "Music Therapist", "Music Producer", "Record Label Executive", "Audio Engineer",
        "Music Video Director", "Session Musician", "Orchestra Conductor", "Choir Director",
        "Opera Singer", "Cantor", "Music Critic", "Music Librarian",
        "Dance Therapist", "Choreologist", "Dance Notator", "Dance Company Manager",
        "Art Therapist", "Drama Therapist", "Bibliotherapist", "Play Therapist",
        "Adventure Therapist", "Recreational Therapist", "Art Gallery Director", "Curator",
        "Art Dealer", "Art Consultant", "Art Appraiser", "Art Historian",
        "Art Critic", "Art Educator", "Crafts Instructor", "Pottery Instructor",
        "Sculpture Instructor", "Painting Instructor", "Drawing Instructor",
        "Photography Instructor", "Video Art Instructor", "Installation Art Instructor",
        "Performance Art Instructor", "Digital Art Instructor", "Graphic Novelist",
        "Comic Book Artist", "Cartoonist", "Illustrator", "Concept Artist",
        "Character Designer", "Storyboard Artist", "Matte Painter", "Visual Effects Artist",
        "Motion Graphics Designer", "Typography Designer", "Brand Identity Designer",
        "Packaging Designer", "Exhibition Designer", "Signage Designer", "Wayfinding Designer",
        "User Interface Designer", "Service Designer", "Experience Designer", "Spatial Designer",
        "Urban Designer", "Landscape Designer", "Garden Designer", "Floral Designer",
        "Event Florist", "Wedding Planner", "Party Planner", "Corporate Event Planner",
        "Conference Organizer", "Festival Director", "Parade Organizer", "Fireworks Technician",
        "Carnival Operator", "Amusement Park Designer", "Theme Park Manager", "Aquarium Manager",
        "Zoo Director", "Botanical Garden Director", "National Park Ranger", "Forest Ranger",
        "Wildlife Refuge Manager", "Environmental Educator", "Outdoor Recreation Planner",
        "Adventure Guide", "Mountaineering Guide", "Rock Climbing Instructor", "Kayak Guide",
        "Rafting Guide", "Diving Instructor", "Surfing Instructor", "Ski Instructor",
        "Snowboarding Instructor", "Ice Skating Coach", "Figure Skating Coach", "Hockey Coach",
        "Basketball Coach", "Football Coach", "Soccer Coach", "Baseball Coach",
        "Tennis Coach", "Golf Instructor", "Swimming Coach", "Track and Field Coach",
        "Cross Country Coach", "Boxing Coach", "Martial Arts Instructor", "Yoga Teacher",
        "Pilates Instructor", "Aerobics Instructor", "Zumba Instructor", "Dance Fitness Instructor",
        "Nutritionist", "Dietitian", "Health Coach", "Holistic Health Practitioner",
        "Aromatherapist", "Massage Therapist", "Chiropractor", "Acupuncturist",
        "Herbalist", "Naturopath", "Homeopath", "Reiki Practitioner",
        "Energy Healer", "Hypnotherapist", "Life Skills Coach", "Parenting Coach",
        "Relationship Coach", "Sex Therapist", "Grief Counselor", "Addiction Counselor",
        "Career Counselor", "Educational Consultant", "Study Skills Coach", "Tutor",
        "Test Prep Instructor", "Language Instructor", "ESL Teacher", "Foreign Language Teacher",
        "Cultural Consultant", "Etiquette Coach", "Public Speaking Coach",
        # ...
        "Software Engineer", "Data Scientist", "Teacher", "Doctor", "Writer",
        "Artist", "Musician", "Chef", "Architect", "Police Officer", "Firefighter",
        "Nurse", "Lawyer", "Mechanic", "Electrician", "Carpenter", "Plumber",
        "Salesperson", "Accountant", "Business Analyst", "Project Manager",
        "Graphic Designer", "Web Developer", "Marketing Specialist", "Farmer",
        "Librarian", "Social Worker", "Therapist", "Counselor", "Psychologist",
        "Veterinarian", "Zoologist", "Biologist", "Chemist", "Physicist",
        "Astronomer", "Geologist", "Historian", "Philosopher", "Journalist",
        "Politician", "Economist", "Sociologist", "Anthropologist", "Linguist",
        "Pharmacist", "Dentist", "Surgeon", "Paramedic", "Chiropractor",
        "Optometrist", "Dietitian", "Nutritionist", "Physiotherapist", "Occupational Therapist",
        "Speech Therapist", "Radiologist", "Anesthesiologist", "Orthodontist",
        "Laboratory Technician", "Environmental Scientist", "Marine Biologist",
        "Wildlife Biologist", "Ecologist", "Botanist", "Forestry Technician",
        "Park Ranger", "Urban Planner", "Civil Engineer", "Mechanical Engineer",
        "Electrical Engineer", "Chemical Engineer", "Aerospace Engineer",
        "Industrial Designer", "Interior Designer", "Fashion Designer",
        "Jewelry Designer", "Makeup Artist", "Hair Stylist", "Barber",
        "Tattoo Artist", "Photographer", "Videographer", "Film Director",
        "Producer", "Actor", "Comedian", "Model", "Dancer", "Choreographer",
        "Fitness Trainer", "Yoga Instructor", "Pilates Instructor", "Personal Trainer",
        "Sports Coach", "Athlete", "Umpire", "Referee", "Sports Commentator",
        "Pilot", "Flight Attendant", "Air Traffic Controller", "Ship Captain",
        "Sailor", "Fisherman", "Truck Driver", "Bus Driver", "Taxi Driver",
        "Train Conductor", "Subway Operator", "Baker", "Butcher", "Food Scientist",
        "Brewer", "Winemaker", "Sommelier", "Barista", "Bartender", "Event Planner",
        "Wedding Planner", "Travel Agent", "Tour Guide", "Translator", "Interpreter",
        "Customs Officer", "Immigration Officer", "Security Guard", "Private Investigator",
        "Detective", "Corrections Officer", "Judge", "Magistrate", "Prosecutor",
        "Public Defender", "Real Estate Agent", "Property Manager", "Loan Officer",
        "Bank Teller", "Insurance Agent", "Financial Advisor", "Investment Banker",
        "Stockbroker", "Trader", "Auditor", "Tax Advisor", "Compliance Officer",
        "Actuary", "Risk Analyst", "Supply Chain Manager", "Logistician",
        "Warehouse Manager", "Quality Control Inspector", "Operations Manager",
        "HR Manager", "Recruiter", "Talent Manager", "Customer Service Representative",
        "Technical Support Specialist", "IT Manager", "Network Administrator",
        "Database Administrator", "Cybersecurity Analyst", "DevOps Engineer",
        "Cloud Architect", "Mobile App Developer", "Game Developer", "AI Researcher",
        "Machine Learning Engineer", "Robotics Engineer", "Blockchain Developer",
        "UX/UI Designer", "SEO Specialist", "Content Creator", "Influencer",
        "Podcaster", "Blogger", "Vlogger", "E-commerce Specialist", "Digital Marketing Manager",
        "Copywriter", "Public Relations Specialist", "Community Manager",
        "Fundraiser", "Grant Writer", "Nonprofit Manager", "Volunteer Coordinator",
        "Humanitarian Aid Worker", "Peace Corps Volunteer", "Diplomat", "Consul",
        "Intelligence Analyst", "Military Officer", "Soldier", "Sailor", "Airman",
        "Marine", "Coast Guard Officer", "Drone Operator", "Cyber Warfare Specialist",
        "Weapons Engineer", "Archaeologist", "Museum Curator", "Archivist",
        "Art Conservator", "Art Critic", "Historian", "Music Producer", "Sound Engineer",
        "Lighting Technician", "Stage Manager", "Set Designer", "Costume Designer",
        "Screenwriter", "Playwright", "Novelist", "Poet", "Essayist",
        "Public Speaker", "Motivational Speaker", "Life Coach", "Business Coach",
        "Career Counselor", "Academic Advisor", "College Professor",
        "Research Scientist", "Postdoctoral Researcher", "Laboratory Manager",
        "Bioinformatician", "Geneticist", "Biomedical Engineer", "Clinical Researcher",
        "Pharmaceutical Sales Representative", "Regulatory Affairs Specialist",
        "Patent Examiner", "Patent Lawyer", "Intellectual Property Lawyer",
        "Entertainment Lawyer", "Corporate Lawyer", "Family Lawyer", "Criminal Lawyer",
        "Employment Lawyer", "Estate Planner", "Mediator", "Arbitrator",
        ]

    def __init__(self, gender: str = None, archetypes: str = None, genres: List[str] = None,):

        # Initialize the logger
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Initializing CHARACTER class...")

        self.name_data  = self.get_name_from_gender(gender)
        self.gender     = self.name_data['gender']
        self.last_name  = self.name_data['last_name']
        self.first_name = self.name_data['first_name']
        self.full_name  = f"{self.first_name} {self.last_name}"
        self.name       = self.full_name
        self.genres     = genres
        self.archetypes = archetypes if archetypes is not None else self.get_archetypes()

        # Logging
        logger.info(f"{caller_name}: Character NAME: {self.name}")
        logger.info(f"{caller_name}: Character GENDER: {self.gender}")

        # # Archetypes
        # if archetypes is not None:
        #     self.archetypes = archetypes
        # else:
        #     archetypes_data = self.get_archetypes()
        #     self.archetypes = []
        #     for archetype_data in archetypes_data:
        #         self.archetypes.append(archetype_data["name"])

        self.personality = self.get_personality()                                  # personality
        self.arc_shape   = self.get_character_arc()                                # character arc
        self.flaws       = self.get_character_attributes("character_flaw"    , 2,) # flaws
        self.strengths   = self.get_character_attributes("character_strength", 2,) # strengths
        self.fears       = random.sample(Character.FEARS      , 2,)                # fears
        self.aspirations = random.sample(Character.ASPIRATIONS, 2,)                # aspirations
        self.motivations = random.sample(Character.MOTIVATIONS, 2,)                # motivations
        self.past_events = random.sample(Character.PAST_EVENTS, 2,)                # past events
        self.occupation  = random.choice(Character.OCCUPATIONS)                    # occupation

    # Serialize for export
    def to_dict(self):
        """Converts the Character object to a dictionary"""
        return {
            'full_name'     : self.full_name,
            'gender'        : self.gender,
            'fears'         : self.fears,
            'flaws'         : self.flaws,
            'strengths'     : self.strengths,
            'archetypes'    : self.archetypes,
            'personality'   : self.personality,
            'aspirations'   : self.aspirations,
            'motivations'   : self.motivations,
            'past_events'   : self.past_events,
            'occupation'    : self.occupation,
            'character_arc' : self.arc_shape,
            # not needed in world bible
            # 'genres'      : self.genres,
            }

    def get_name_from_gender(self, gender: str = None,) -> Dict:
        """
        Generate a random name and gender.

        Args:
            gender (str, optional): The gender to generate a name for
            ('male', 'female'). If None, a random gender is chosen.
            Defaults to None.

        Returns:
            Dict: A dictionary containing the generated
                  first name, last name, and gender.
        """
        if gender and gender not in ('male', 'female'):
            raise ValueError("Gender must be either 'male' or 'female'")

        if not gender or gender is None:
            gender = random.choice(['male', 'female',])

        first_name = names.get_first_name(gender=gender) if gender else names.get_first_name()
        last_name = names.get_last_name()

        return {
            'first_name': first_name,
            'last_name': last_name,
            'gender': gender,
        }

    def get_archetypes(self, num_archetypes: int = 3,):
        """
        Generate a list of random character archetypes using the richer data structure.
        Removes the "id" attribute from each archetype before returning.

        Args:
            num_archetypes (int, optional): The number of archetypes to generate.
            Defaults to 3.

        Returns:
            List[Dict]: A list of dictionaries, each representing an archetype, without the "id" attribute.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating {num_archetypes} random character ARCHETYPES...")

        archetypes = random.sample(Character.ARCHETYPES, num_archetypes,)
        for archetype in archetypes:
            if "id" in archetype:
                del archetype["id"]

        logger.info(f"{caller_name}: {num_archetypes} ARCHETYPES generated.")
        logger.info(f"{caller_name}:   - Archetypes: {archetypes}")

        return archetypes

    def get_personality(self):
        """
        Generate a random MBTI personality type and return full data.
        Define the 16 possible MBTI types - Generate the personality code (e.g., 'INTP-T')
        """

        # The 4 dichotomies in MBTI
        ei = random.choice(['E', 'I',])  # Extraversion vs. Introversion
        sn = random.choice(['S', 'N',])  # Sensing vs. Intuition
        tf = random.choice(['T', 'F',])  # Thinking vs. Feeling
        jp = random.choice(['J', 'P',])  # Judging vs. Perceiving

        # Neuroticism
        at = random.choice(['A', 'T',])  # Assertive vs. Turbulent

        personality_code = f"{ei}{sn}{tf}{jp}"  # Extract MBTI code (e.g., 'INTP')
        neuroticism_code = f"{at}" # Extract neuroticism code (e.g., '-T')
        full_personality_code = f"{personality_code}-{neuroticism_code}"

        # Retrieve personality details and construct dictionary
        personality_data = {
            'type': personality_code,
            'name': Character.PERSONALITIES.get(personality_code, {}).get('name'),
            'class': Character.PERSONALITIES.get(personality_code, {}).get('class'),
            'description': Character.PERSONALITIES.get(personality_code, {}).get('description'),
            'insight': Character.PERSONALITIES.get(personality_code, {}).get('insight'),
            'neuroticism': Character.PERSONALITIES.get(neuroticism_code, {}),
        }
        return personality_data

    def get_character_attributes(self, attribute: str, num_attributes: int = 2,):
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating random character ATTRIBUTES...")
        logger.info(f"{caller_name}:   - attribute: {attribute}")
        logger.info(f"{caller_name}:   - number: {num_attributes}")

        element_provider = StoryElementProvider()
        logger.info(f"{caller_name}: Initiated StoryElementProvider...")

        # fetch random element
        logger.info(f"{caller_name}: Fetching random element...")
        element_types = [attribute,]
        logger.info(f"{caller_name}: Selecting random element...")
        random_attributes_full_obj = element_provider.get_random_elements(element_types=element_types, genres=self.genres, num_elements=num_attributes,)

        # Extract story_element only - strip genres, element_types
        random_attributes = [random_attribute['story_element'] for random_attribute in random_attributes_full_obj],
        logger.debug(f"{caller_name}: Random attributes: {attribute}: {random_attributes}")
        return random_attributes

    def get_character_arc(self):
        """Randomly select a character arc, get its value and convert to string"""
        random_arc = random.choice(list(ArcShape))
        arc_shape = str(random_arc.value)
        return arc_shape

    def get_current_state(self) -> Dict[str, float]:
        """
        Calculates and returns the character's current state based on
        their arc type and the events they've experienced.
        """
        current_state = self.attributes.copy()
        for event in self.events:
            for attr in current_state:
                if self.arc_type == ArcType.GROWTH:
                    current_state[attr] += event.impact * 0.1
                elif self.arc_type == ArcType.FALL:
                    current_state[attr] -= event.impact * 0.1
                current_state[attr] = max(0.0, min(1.0, current_state[attr]))
        return current_state

# </entropic_creation>

# <series_bible>

class SeriesBible:
    def __init__(self):
        self.name = "Starlight Saga Studios"
        self.author = "Ghost Writer"
        self.chapter_outlines = []
        self.chapter_titles = []
        # self.chapter_descriptions = []
        # self.chapter_numbers = []
        # self.book_numbers = []

        self.github_manager = GithubManager()

    def add_chapter_outline(self, chapter_outline: str, chapter_number: int, book_number: int = 1,) -> None:
        """Adds a chapter outline to the SeriesBible."""
        if not hasattr(self, 'chapter_outlines'):
            self.chapter_outlines = []
        self.chapter_outlines.append(chapter_outline)

    def save_to_gist(self, working_title: str,):
        """Saves the Series Bible to a Gist."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Saving SERIES BIBLE to Gist...")

        # Create a dictionary to structure the Series Bible data
        series_bible_data = {
            'name': self.name,
            'author': self.author,
            'chapter_outlines': self.chapter_outlines, # YAML list of chapters
            # ... Add other attributes as needed
        }

        series_bible_yaml = yaml.dump(series_bible_data, indent=4)
        logger.info(f"{caller_name}: SERIES BIBLE: {series_bible_yaml}")

        sanitized_filename= self.github_manager.sanitize_filename(f"seriesbible_{working_title}.yaml")
        gist_id, gist_url = self.github_manager.save_to_gist(
            filename=sanitized_filename,
            content=series_bible_yaml,
            description=f"Series Bible for '{working_title}'",
        )
        logger.info(f"{caller_name}: SERIES BIBLE saved to Gist: {gist_url}")
        return gist_id, gist_url

# <world_bible>

class WorldBible:
    """
        A class to store and manage information about the story world.

        Attributes:
            name (str): The name of the world or setting.
            description (str): A brief description of the world.
            characters (Dict[str, Character]): A dictionary of characters in the world, keyed by their names.
            locations (Dict[str, Location]): A dictionary of locations in the world, keyed by their names.
            items (Dict[str, Item]): A dictionary of items in the world, keyed by their names.
            events (Dict[str, Event]): A dictionary of events that have occurred in the world, keyed by their names.
            cultures (Dict[str, Culture]): A dictionary of cultures in the world, keyed by their names.

        Methods:
            add_character(self, character: Character): Adds a character to the world.
            add_location(self, location: Location): Adds a location to the world.
            add_item(self, item: Item): Adds an item to the world.
            add_event(self, event: Event): Adds an event to the world.
            add_culture(self, culture: Culture): Adds a culture to the world.
            """

    def __init__(self, working_title: str = None, premise: str = None,):
        """
        Initializes the WorldBible.

        Args:
            name (str, optional): The name of the world. Defaults to "My World".
            description (str, optional): A description of the world. Defaults to "A world full of wonder.".
        """
        self.characters = []
        self.narratives = {}
        self.premise = premise
        self.author_style = None
        self.working_title = None

    def describe(self) -> str:
        """Returns a description of the world."""

    def add_author_style(self, author_style: str,) -> None:
        """
        Adds an author style to the WorldBible if there is not already a style defined.

        Only one author style can be defined per WorldBible, please.

        Args:
            author_style (str): The author style to add.
        """
        if not hasattr(self, 'author_style') or self.author_style is None:
            self.author_style = author_style

    def add_working_title(self, working_title: str) -> None:
        """Adds a working title to the WorldBible."""
        self.working_title = working_title

    def add_premise(self, premise: str) -> None:
        """Adds a premise to the WorldBible."""
        self.premise = premise

    def add_character(self, character: dict,) -> None:
        """Adds a character to the WorldBible."""
        self.characters.append(character)

    def add_narrative(self, narrative: str, label: str) -> None:
        """Adds a narrative response to the World Bible."""
        if label not in self.narratives:
            self.narratives[label] = ""  # Initialize as an empty string
        self.narratives[label] += narrative  # Append the narrative text

    def retrieve_narrative_block(self, label: str) -> str:
        """Retrieves a narrative block from the World Bible."""
        return self.narratives.get(label, "") # Return empty string if not found

    def deprecating_add_narrative(self, narrative: str, label: str) -> None:
        """
        Adds a narrative response to the World Bible, enclosed in XML tags for later retrieval.

        Args:
            narrative (str): The narrative text.
            label (str): A label to identify the narrative block.
        """
        if label not in self.narratives:
            self.narratives[label] = []
        self.narratives[label] = narrative
        # self.narratives[label].append(narrative)

    def deprecating_retrieve_narrative_block(self, label: str) -> str:
        """
        Retrieves a narrative block from the World Bible using the specified label.

        Example Usage:
            # Retrieve the narrative
            relationships_narrative = writer.world_bible.retrieve_narrative_block("relationships")
            logger.info(f"Relationships Narrative:\n{relationships_narrative}")
            """
        if label in self.narratives:
            match = re.search(f"<{label}>(.*?)</{label}>", self.narratives[label], re.DOTALL)
            if match:
                return match.group(1).strip()
        return None

# </world_bible>

# </series_bible>

# <dynamic_writing>

@dataclass
class Beat(TypedDict):
    """
    Represents a story beat with its name, phase, word count, and associated prompts.

    Attributes:
        name (str): The beat name.
        chapter (int): The chapter number.
        act (int): The number of the act that contains the beat.
        word_count (int): The word count for the beat.
        screenplay_page (): The approximate page this beat would start in a screenplay.
        prompts (List[str]): The list of prompts for the beat. The questions the beat answers.
        description (Optional[str]): A brief description of the beat.
        example (Optional[str]): An example of the beat in a story.
    """
    act: int
    name: str
    chapter: int
    word_count: int
    prompts: List[str]
    example: Optional[str]
    description: Optional[str]
    screenplay_page: Optional[int]

class Genre(Enum):
    """
    Enum representing different genres of stories with associated attributes and methods.

    Attributes:
        name (str): The name of the genre.
        description (str): A brief description of the genre.
        total_word_count (int): The total word count for the genre.
        beats (List[Beat]): The list of beats associated with the genre.

        ## A note about structures:

        There are still some other story structures that we haven't included yet. Some additional structures that are commonly used in storytelling include:
          - The Fichtean Curve: This structure focuses on rising action through a series of crises, leading to a climax.
          - Freytag's Pyramid: A five-act dramatic structure that includes exposition, rising action, climax, falling action, and denouement.
          - The Seven-Point Story Structure: A plot outline that includes hook, plot turn 1, pinch point 1, midpoint, pinch point 2, plot turn 2, and resolution.
          - The Snowflake Method: A more detailed approach to story development that starts with a simple premise and expands outward.
          - The Story Circle (Dan Harmon's Structure): An eight-step cycle based on the Hero's Journey but simplified for modern storytelling.
          - The MICE Quotient: A system for categorizing story elements into Milieu, Idea, Character, and Event.
          - The W-Plot: A structure that follows the ups and downs of the protagonist's journey, forming a W shape.
        These structures could potentially be adapted into a similar format as the ones we've already covered,
        providing writers with additional tools for structuring their stories across various genres.

        Based on the structures we've covered and typical genre conventions,
        here's a mapping of story structures to various genres:
          - Three-Act Structure: DRAMA, COMEDY, ACTION, THRILLER, HORROR, WESTERN
          - Hero's Journey: FANTASY, SCI_FI, ADVENTURE, YOUNG_ADULT, DYSTOPIAN
          - Mystery/Detective Structure: MYSTERY, CRIME, THRILLER (with modifications)
          - Save the Cat: Can be adapted for most genres, but particularly useful for: COMEDY, ACTION, ADVENTURE, YOUNG_ADULT
          - Romance Beat Structure: ROMANCE, EROTICA (with modifications), Can also be incorporated as a subplot in many other genres
          - While these are typical associations, it's important to note that many genres can use multiple structures or adapt them to fit specific needs.
          Here are some additional notes on genres not directly mapped above:
          - HISTORICAL_FICTION: Often uses Three-Act Structure or Hero's Journey, depending on the focus of the story.
          - CHILDRENS: May use simplified versions of Three-Act Structure or Hero's Journey.
          - INSPIRATIONAL and RELIGIOUS: Often use Three-Act Structure or a modified Hero's Journey.
          - POETRY: Typically doesn't follow these narrative structures, as it has its own forms and conventions.
          - PARANORMAL: Often uses elements of Mystery/Detective Structure combined with Horror or Fantasy elements.

        Remember that these are general guidelines, and skilled authors often blend or subvert these structures to create unique and engaging stories within their chosen genres.
    """
    ROMANCE = {
        "name": "Romance",
        "total_word_count": 53125,
        "beats": [
            Beat(name="Intro H1",
                 chapter=1, act=1, word_count=1500,
                 example="Meet Corrina",
                 prompts=[
                     """How can you concisely introduce the first main character with just enough detail about their personality,
                     background, and current situation to immediately hook the reader into feeling a sense of personal empathy
                     and a strong emotional connection with this character?""",
                     "How can you describe the first main character's personal traits in a way that will become relevant to the upcoming conflict?",
                     "What are the first main character's most significant flaws and desires?",
                     "What immediate goal or desire of the first character could drive them and lead to a central conflict?",
                     "How can you hint at the first character's flaws that will play into the central conflict?",
                     "What action or scenario would best reveal the first main character's personality and motivations?",
                 ]),
            Beat(name="Intro H2",
                 chapter=2, act=1, word_count=1500,
                 example="Meet Nicholas",
                 prompts=[
                     "How does the second main character contrast with the first in terms of personality and goals?",
                     "What challenge or conflict could reveal the second main character's strengths and weaknesses?",
                     "What compelling backstory or personal flaw explains the second main character's motivations and inner conflicts?",
                     "What specific qualities of the first main character draw in the second main character?",
                     "How can you explore the second main character's underlying desires and needs beyond superficial attraction?",
                     "How can you introduce the second main character in a way that immediately creates tension with the first?",
                     "What conflicting goal or desire does the second character have?",
                     "How do the second character's traits or background contrast with the first character's in a way that sparks conflict?",
                 ]),
            Beat(name="Meet Cute",
                 chapter=3, act=1, word_count=2125,
                 example="Corrina and Nicholas meet at the hotel.",
                 prompts=[
                     "What unique setting, set of circumstances or events, could enhance the first encounter between the main characters?",
                     "How can you introduce an element of conflict or tension in the meet-cute?",
                     "How can you design the first encounter to immediately highlight this central conflict or tension between the characters?",
                     "Could a misunderstanding or clash occur during this meeting that sets up the main conflict?",
                     "How can you balance the attraction between characters with the emerging conflict?",
                     "What memorable detail or event during their first encounter could spark curiosity or intrigue?",
                 ]),
            Beat(name="No Way 1",
                 chapter=4, act=1, word_count=3000,
                 example="Corrina and Nicholas are from rival families.",
                 prompts=[
                     "What is the central conflict that seems to make a relationship between the characters impossible?",
                     "What significant obstacle could prevent the characters from being together initially?",
                     "How can you make this obstacle believable and impactful?",
                     "How can you fully explore the implications and depth of this conflict?",
                     "What misunderstanding or clash of values could contribute to this obstacle?",
                     "What stakes can you establish for both characters if they pursue a relationship despite this conflict?",
                 ]),
            Beat(name="Adhesion Plot Thrust",
                 chapter=5, act=1, word_count=2500,
                 example="Corrina and Nicholas both have a relation to Bella and must be around each other.",
                 prompts=[
                     "What plot element could force the characters to stay connected despite the obstacles?",
                     "How can you increase the stakes of this plot thrust to create urgency?",
                     "What consequences of this plot thrust could affect both characters significantly?",
                     "What circumstances force the characters to interact despite the central conflict?",
                     "How does this forced interaction exacerbate the existing conflict?",
                     "What new complications arise from this situation that deepen the central conflict?",
                     "Can we introduce an external obstacle to their relationship, such as strong family opposition or a rival suitor with compelling qualities.",
                     "Can we increase the stakes of the adhesion plot thrust by creating significant consequences for their relationship if they fail to overcome this obstacle.",
                     "How can you create a situation where the characters must rely on each other despite their differences?",
                     "What shared responsibility or mutual goal could force them to work together?",
                     "How might their forced proximity reveal unexpected commonalities or attractions?",
                 ]),
            Beat(name="No Way 2",
                 chapter=6, act=2, word_count=4250,
                 example="Drama sparks in Bella's posse.",
                 prompts=[
                     "How can you escalate the 'No Way' obstacle with a new challenge or complication?",
                     "What secondary character could influence the 'No Way' obstacle, either intentionally or unintentionally?",
                     "How might this escalation further separate the main characters?",
                     "How can you escalate the central conflict to a new level?",
                     "What unforeseen consequences of the conflict emerge at this point?",
                     "How do the characters' attempts to resolve the conflict inadvertently make it worse?",
                     "Introduce a new external factor or character that intensifies the 'No Way' obstacle and forces the characters to confront their conflicting desires.",
                     "What misunderstanding or miscommunication could further complicate their situation?",
                     "How might societal or cultural pressures amplify the existing obstacles?",
                     "What personal crisis for one character could make their relationship seem even more impossible?",
                     "How could their growing attraction paradoxically create new barriers between them?",
                 ]),
            Beat(name="Inkling",
                 chapter=7, act=2, word_count=4250,
                 example="Corrina and Nicholas share a sweet moment.",
                 prompts=[
                     "What genuine interaction or experience could hint at the characters' potential compatibility?",
                     "How can you use sensory details to depict this 'Inkling' moment vividly?",
                     "What emotional cues could create a lasting impression on the reader during this moment?",
                     "Despite the conflict, what moment suggests the characters could work well together?",
                     "How does this moment of connection create internal conflict for each character?",
                     "What small way does this experience begin to shift the characters' perspectives on the central conflict?",
                 ]),
            Beat(name="Deepening Desire",
                 chapter=8, act=2, word_count=4250,
                 example="Corrina and Nicholas spend more time together.",
                 prompts=[
                     "What meaningful interactions could develop the characters' attraction to each other?",
                     "How can you show the characters' vulnerability as their emotional connection deepens?",
                     "What shared experiences could intensify their desire for each other?",
                     "How does attraction grow between the characters despite the ongoing conflict?",
                     "What aspects of each character are revealed through the conflict that actually draw them closer?",
                     "How does the growing desire complicate the central conflict?",
                 ]),
            Beat(name="Maybe This Could Work",
                 chapter=9, act=2, word_count=4250,
                 example="Corrina and Nicholas realize there are ways around their family drama.",
                 prompts=[
                     "What turning point could make the characters believe their relationship might work?",
                     "How can you introduce a new perspective that helps overcome their doubts?",
                     "What solution could the characters discover to address their initial obstacles?",
                     "What possible resolution to the conflict do the characters begin to see?",
                     "How do the characters start to work together to address the central conflict?",
                     "What new obstacle emerges just as they think they might overcome the conflict?",
                 ]),
            Beat(name="Midpoint of Love Plot Thrust",
                 chapter=10, act=2, word_count=4250,
                 example="Corrina and Nicholas have dinner at a nice restaurant in town.",
                 prompts=[
                     "What significant event could deepen the emotional connection between the characters?",
                     "How can you raise the stakes of the love plot at this midpoint?",
                     "What new challenges and opportunities could arise for the characters' relationship?",
                     "What major event forces the characters to confront their feelings amid the conflict?",
                     "How does this event challenge the characters' previous stance on the central conflict?",
                     "What decision point does this create regarding the conflict and their relationship?",
                 ]),
            Beat(name="Inkling Doubt",
                 chapter=11, act=3, word_count=2125,
                 example="Corrina wonder if Nicholas is dating a model friend of Bella's.",
                 prompts=[
                     "What seed of doubt or insecurity could emerge about the viability of their relationship?",
                     "How can you use internal monologue to portray this doubt subtly?",
                     "What external event might trigger this initial doubt?",
                     "What makes the characters doubt their ability to overcome the central conflict?",
                     "How do external factors reinforce the original conflict at this point?",
                     "What personal insecurities resurface in light of the ongoing conflict?",
                 ]),
            Beat(name="Deepening Doubt",
                 chapter=12, act=3, word_count=2125,
                 example="Corrina sees Nick and the model hanging out.",
                 prompts=[
                     "What new event could escalate the characters' doubt and insecurity?",
                     "How can you show the characters' emotional reactions to this deepening doubt?",
                     "What aspect of trust or commitment could be challenged in this beat?",
                     "How does the central conflict seem to become truly insurmountable?",
                     "What sacrifices would be required to resolve the conflict, and why do they seem too great?",
                     "How do the characters' doubts about resolving the conflict affect their relationship?",
                 ]),
            Beat(name="Retreat",
                 chapter=13, act=3, word_count=2125,
                 example="Nick tries to explain to Corrina but she won't listen.",
                 prompts=[
                     "How might one or both characters retreat from the relationship to protect themselves?",
                     "What actions could portray the characters' emotional withdrawal?",
                     "How can dialogue reveal the reasons behind their retreat?",
                     "How do the characters try to escape or deny the central conflict?",
                     "What actions do they take to retreat from both the conflict and each other?",
                     "How does this retreat seem to confirm that the conflict cannot be resolved?",
                 ]),
            Beat(name="Shields Up",
                 chapter=14, act=3, word_count=2125,
                 example="Corrina tells Nick she is done.",
                 prompts=[
                     "How can you show the characters putting up emotional walls and defenses?",
                     "What communication breakdown could occur as tensions escalate?",
                     "How might the characters' body language change as they guard themselves?",
                     "How do the characters try to protect themselves from the pain caused by the central conflict?",
                     "What defense mechanisms does each character employ?",
                     "How does their self-protection reinforce the conflict between them?",
                 ]),
            Beat(name="Break Up",
                 chapter=15, act=3, word_count=2125,
                 example="Corrina and Nicholas spend time apart.",
                 prompts=[
                     "What climactic conflict could lead to the characters' separation or break-up?",
                     "How can you explore the immediate emotional fallout of the break-up?",
                     "What actions might the characters take to cope with their pain and regret?",
                     "What final straw makes the central conflict appear completely irresolvable?",
                     "How do the characters decide to give up on resolving the conflict?",
                     "What is the immediate fallout of this decision for both characters?",
                 ]),
            Beat(name="Dark Night",
                 chapter=16, act=4, word_count=2125,
                 example="Nicholas leaves the posse to go to Rome.",
                 prompts=[
                     "How can you portray the lowest point for one or both characters after the break-up?",
                     "What inner demons or deepest fears might the characters confront?",
                     "How can you use symbolism to create a sense of despair during this 'Dark Night'?",
                     "How do the characters hit rock bottom as a result of failing to resolve their conflict?",
                     "What personal epiphanies do they have about their role in perpetuating the conflict?",
                     "How does this low point prepare them for potentially overcoming the conflict?",
                 ]),
            Beat(name="Wake Up",
                 chapter=17, act=4, word_count=2125,
                 example="Corrina gets a handwritten letter from Nicholas delivered to her room.",
                 prompts=[
                     "What moment of realization could make a character recognize their mistakes?",
                     "What catalyst might trigger this 'Wake Up' moment?",
                     "How can you show the character's newfound understanding of the relationship's importance?",
                     "What realization helps the characters see the central conflict in a new light?",
                     "How do they come to understand that resolving the conflict is worth the necessary sacrifices?",
                     "What first step do they take towards truly addressing the core issue of their conflict?",
                 ]),
            Beat(name="Grand Gesture",
                 chapter=18, act=4, word_count=2125,
                 example="Nick flies back to Portofino from Rome to confess his love.",
                 prompts=[
                     "What significant action could a character take to demonstrate their love and commitment?",
                     "How can you make this 'Grand Gesture' meaningful and impactful?",
                     "What growth or change in the character could this gesture showcase?",
                     "What major action does a character take to prove they can overcome the central conflict?",
                     "How does this gesture demonstrate their growth and change since the conflict began?",
                     "How does this action begin to resolve the long-standing conflict between them?",
                 ]),
            Beat(name="Whole Hearted",
                 chapter=19, act=4, word_count=2125,
                 example="Corrina and Nicholas kiss and makeup and officially become a couple.",
                 prompts=[
                     "How can you show the characters reconciling with a deeper understanding of each other?",
                     "What remaining doubts or insecurities might the characters need to overcome?",
                     "How can you depict their wholehearted commitment to the relationship?",
                     "How do the characters work together to fully resolve the central conflict?",
                     "What final obstacles to resolving the conflict do they overcome?",
                     "How has resolving the conflict changed and strengthened their relationship?",
                 ]),
            Beat(name="Epilogue",
                 chapter=20, act=4, word_count=2125,
                 example="Corrina and Nicholas bring their families together at a vineyard for a big, Italian-style meal.",
                 prompts=[
                     "What glimpse into the characters' future could highlight their happiness and fulfillment?",
                     "How can you tie up any loose ends to provide a satisfying resolution?",
                     "What final image or scene could leave the reader with a sense of closure?",
                     "How has resolving the central conflict changed the characters' lives?",
                     "What remnants of the conflict still exist, and how do the characters deal with them now?",
                     "How does the resolution of the conflict set the stage for the characters' future together?",
                 ]),
        ],
    }
    SAVE_THE_CAT = {
        "name": "Save the Cat",
        "description": """
            Introduced by Blake Snyder in his 2005 screenwriting guide.
            Initially designed for screenplays, many writers have adapted it for
            various forms of storytelling, including novels across different genres.
            Its popularity stems from its clear, structured approach to narrative
            development and character arc progression.

            This is an enhanced version of Blake Snyder's Save the Cat beat sheet structure,
            designed to offer more flexibility and depth for various storytelling mediums.
            This improved version maintains the core structure while adding elements to
            support character development, theme exploration, and adaptability across genres.

            -- "Save the Cat" --

                Refers to a moment, typically early in the story, where the protagonist does
                something likable, heroic, or altruistic, often as simple as 'saving a cat.'
                This moment is designed to make the audience empathize with and root for the protagonist.

                Placement: Usually occurs during the Setup beat (but can be flexible).
                Importance: Crucial for establishing an emotional connection between the audience and the protagonist.
                            The audience must care about and be emotionally invested in the fate of the protaganist
                            in order for any of the plot events to have any meaning or be worth consuming.
            """,
        "saving_the_cat_examples": [
            # "Save the Cat" Moment Examples
            "A gruff detective shares his lunch with a stray dog at a crime scene.",
            "A busy executive stops to help an elderly person cross a busy street.",
            "A teenage rebel stands up to bullies picking on a younger student.",
            "A cynical lawyer takes on a pro-bono case for a struggling family.",
            "A shy librarian rescues a child's beloved stuffed animal from the library's lost and found.",
            "A hardened soldier gently helps a young refugee child tie their shoelaces.",
            "A self-absorbed influencer anonymously donates their designer clothes to a women's shelter.",
            "A stern teacher stays late to help a struggling student understand a difficult concept.",
            "A cutthroat businesswoman postpones an important meeting to comfort her assistant who just received bad news.",
            "A grumpy old man teaches neighborhood kids how to fix their bicycles.",
            "A reserved scientist passionately defends a colleague's work against unfair criticism.",
            "A sarcastic barista remembers a regular customer's complicated order and has it ready before they ask.",
            "A tough-as-nails coach shows unexpected gentleness with an injured player.",
            "A jaded journalist risks their job to expose corruption that's hurting vulnerable communities.",
            "A self-centered athlete shares the spotlight with a less-recognized teammate during an interview.",
            "A workaholic parent cancels an important business trip to attend their child's school play.",
            "A nervous new doctor stays hours after their shift to ensure a patient's comfort.",
            "A misanthropic writer mentors a young, aspiring author at a local coffee shop.",
            "A wealthy CEO anonymously pays for a stranger's groceries at the checkout line.",
            "A socially awkward genius patiently explains complex theories to interested laypeople at a party.",
            "A hardened criminal returns a lost wallet without taking any money from it.",
            "A busy chef closes their restaurant for a day to cook meals for a local homeless shelter.",
            "A reclusive artist donates a painting to a school fundraiser auction.",
            "A no-nonsense judge shows leniency to a first-time offender and offers guidance instead of punishment.",
            "A cynical film critic enthusiastically promotes an unknown indie filmmaker's debut work.",
            "A tough boxing coach spends extra time with a discouraged student, sharing personal stories of overcoming adversity.",
            "A self-absorbed pop star visits sick children in the hospital without any media coverage.",
            "A ruthless corporate lawyer provides free legal advice at a community center on weekends.",
            "A strict military officer bends the rules to help a recruit with a family emergency.",
            "A reserved librarian organizes a surprise birthday party for a lonely regular patron.",
            "A grumpy neighbor shovels snow from an elderly couple's driveway without being asked.",
            "A snarky talk show host uses their platform to raise awareness for an overlooked charitable cause.",
            "A busy surgeon takes time to explain a complex procedure to a scared child patient using drawings and toys.",
            "A cynical weather forecaster stays on air during a severe storm to ensure public safety, missing their child's recital.",
            "A tough-love fitness instructor shows unexpected patience with a discouraged client struggling to lose weight.",
            "A standoffish coworker secretly leaves encouraging notes for a stressed-out colleague.",
            "A rebellious teenager helps their younger sibling cover up a broken vase to avoid punishment.",
            "A notoriously harsh food critic praises a struggling family-run restaurant, helping save it from closure.",
            "A competitive athlete stops during a race to help a fallen opponent.",
            "A reserved accountant volunteers to read stories at the local children's hospital every weekend.",
            "A gruff mechanic fixes a single mother's car for free when she can't afford the repairs.",
            "A pessimistic environmental scientist plants trees with local schoolchildren on Earth Day.",
            "A stern principal uses their day off to repaint a vandalized area of the school.",
            "A cantankerous old neighbor secretly tends to a deployed soldier's garden.",
            "A ruthless businessman anonymously pays for a struggling employee's child's medical treatment.",
            "A disgruntled postal worker goes out of their way to track down the recipient of a lost, important-looking letter.",
            "A cynical art teacher passionately defends a student's unconventional project to disapproving parents.",
            "A grumpy IT specialist stays late to recover a coworker's accidentally deleted files.",
            "A hard-nosed police officer buys lunch for a hungry suspect before questioning.",
            "A misanthropic writer volunteers to teach creative writing at a juvenile detention center.",
        ],
        "total_word_count": 70000,
        "beats": [
            Beat(
                name="Opening Image",
                chapter=1,
                act=1,
                word_count=1500,
                screenplay_page=1,
                example="Introduce the protagonist in their 'before' state",
                prompts=[
                    "How can you visually represent the protagonist's world before the story begins?",
                    "What symbolic elements can you include to foreshadow the journey ahead?",
                    "How does the opening reflect the story's genre and tone?",
                ],
                character_focus="Protagonist's initial state",
                theme_connection="Subtle introduction of thematic elements",
            ),
            Beat(
                name="Theme Stated",
                chapter=2,
                act=1,
                word_count=1000,
                screenplay_page=5,
                example="A character states the theme in a way the protagonist doesn't yet understand",
                prompts=[
                    "How can you subtly introduce the story's theme through dialogue or action?",
                    "What misunderstanding of the theme does the protagonist currently have?",
                    "How can the theme be presented in a way that's natural to the story's world?",
                ],
                character_focus="Protagonist's worldview",
                theme_connection="Explicit statement of theme, often misunderstood",
            ),
            Beat(
                name="Setup",
                chapter=3,
                act=1,
                word_count=3000,
                screenplay_page=10,
                example="Establish the protagonist's normal world and routines",
                prompts=[
                    "What aspects of the protagonist's life need to change?",
                    "How can you show the protagonist's flaws and desires?",
                    "What elements of the normal world contrast with the coming adventure?",
                    "How can you incorporate a 'Save the Cat' moment to endear the protagonist to the audience?",
                ],
                character_focus="Protagonist's comfort zone, limitations, and likable qualities",
                theme_connection="Showing how the protagonist's life doesn't align with the theme",
                save_the_cat_opportunity="Prime moment to showcase the protagonist's redeeming qualities",
            ),
            Beat(
                name="Catalyst",
                chapter=4,
                act=1,
                word_count=2000,
                screenplay_page=12,
                example="An inciting incident disrupts the protagonist's world",
                prompts=[
                    "What event forces the protagonist out of their comfort zone?",
                    "How does this catalyst relate to the story's central conflict?",
                    "How can the catalyst challenge the protagonist's existing worldview?",
                ],
                character_focus="Protagonist's initial reaction to change",
                theme_connection="First major challenge to the protagonist's understanding of the theme",
            ),
            Beat(
                name="Debate",
                chapter=5,
                act=2,
                word_count=4000,
                screenplay_page=18,
                example="The protagonist wrestles with the call to adventure",
                prompts=[
                    "What fears or doubts hold the protagonist back?",
                    "How can you increase the stakes to force a decision?",
                    "What internal and external conflicts arise during this debate?",
                ],
                character_focus="Protagonist's internal struggle and resistance",
                theme_connection="Deepening the thematic conflict within the protagonist",
            ),
            Beat(
                name="Break into Two",
                chapter=6,
                act=2,
                word_count=3000,
                screenplay_page=25,
                example="The protagonist makes a choice and enters the 'upside-down' world",
                prompts=[
                    "What action does the protagonist take to fully commit to the journey?",
                    "How does this decision contrast with their behavior in the Setup?",
                    "What does the protagonist leave behind, literally or metaphorically?",
                ],
                character_focus="Protagonist's first major step towards change",
                theme_connection="Active engagement with the theme, often still flawed",
            ),
            Beat(
                name="B Story",
                chapter=7,
                act=3,
                word_count=3000,
                screenplay_page=30,
                example="Introduction of a secondary plot, often involving a love interest or mentor",
                prompts=[
                    "How does the B Story reflect or contrast with the main plot?",
                    "What new character can help the protagonist learn the theme?",
                    "How can the B Story provide a fresh perspective on the main conflict?",
                ],
                character_focus="Protagonist's relationships and support system",
                theme_connection="Exploring the theme through different relationships or subplots",
            ),
            Beat(
                name="Fun and Games",
                chapter=8,
                act=3,
                word_count=15000,
                screenplay_page=42,
                example="The protagonist explores the new world and faces initial challenges",
                prompts=[
                    "What exciting set pieces can showcase the premise's potential?",
                    "How can you balance victories and setbacks for the protagonist?",
                    "What new skills or insights does the protagonist gain in this section?",
                ],
                character_focus="Protagonist's growth and adaptation to new challenges",
                theme_connection="Testing the theme through various scenarios and challenges",
            ),
            Beat(
                name="Midpoint",
                chapter=9,
                act=4,
                word_count=5000,
                screenplay_page=55,
                example="A major event that raises the stakes and pushes the story in a new direction",
                prompts=[
                    "What false victory or false defeat can serve as a turning point?",
                    "How does this event force the protagonist to recommit to their goal?",
                    "What revelation can deepen the protagonist's understanding of the conflict?"
                ],
                character_focus="Protagonist's moment of truth or major realization",
                theme_connection="Crystallization of the theme, often with a twist or deeper meaning",
            ),
            Beat(
                name="Bad Guys Close In",
                chapter=10,
                act=4,
                word_count=10000,
                screenplay_page=65,
                example="Antagonists regroup and intensify their efforts against the protagonist",
                prompts=[
                    "How can external pressures increase on the protagonist?",
                    "What internal doubts or conflicts resurface?",
                    "How can allies be separated or turned against the protagonist?",
                ],
                character_focus="Protagonist under maximum pressure, testing resolve",
                theme_connection="Theme challenged by mounting opposition and doubt",
            ),
            Beat(
                name="All Is Lost",
                chapter=11,
                act=5,
                word_count=3000,
                screenplay_page=75,
                example="The protagonist hits rock bottom, often with a death or loss",
                prompts=[
                    "What can the protagonist lose that seems to destroy all hope?",
                    "How can this loss relate to the story's theme?",
                    "What beliefs or attachments must the protagonist let go of?",
                ],
                character_focus="Protagonist's darkest moment and greatest vulnerability",
                theme_connection="Apparent failure of the theme, setting up for later revelation",
            ),
            Beat(
                name="Dark Night of the Soul",
                chapter=12,
                act=5,
                word_count=2000,
                screenplay_page=80,
                example="The protagonist's lowest moment of despair",
                prompts=[
                    "How does the protagonist confront their deepest fears or flaws?",
                    "What realization can lead to a new understanding of the theme?",
                    "How can this moment echo or contrast with the Opening Image?",
                ],
                character_focus="Protagonist's inner journey and self-reflection",
                theme_connection="Profound personal confrontation with the theme's meaning",
            ),
            Beat(
                name="Break into Three",
                chapter=13,
                act=6,
                word_count=3000,
                screenplay_page=85,
                example="The protagonist finds a new solution and rallies for the final push",
                prompts=[
                    "How does the protagonist combine what they've learned from the A Story and B Story?",
                    "What new approach demonstrates their growth and understanding of the theme?",
                    "How does this moment prepare for the final confrontation?"
                ],
                character_focus="Protagonist's transformation and renewed determination",
                theme_connection="Integration of the theme into the protagonist's new worldview",
            ),
            Beat(
                name="Finale",
                chapter=14,
                act=6,
                word_count=12000,
                screenplay_page=100,
                example="The protagonist faces the antagonist in a final battle",
                prompts=[
                    "How can you create a sequence of escalating challenges?",
                    "How does the protagonist demonstrate mastery of the theme?",
                    "What final sacrifice or choice cements the protagonist's arc?",
                ],
                character_focus="Protagonist's ultimate test and demonstration of growth",
                theme_connection="Theme fully realized through action and choice",
            ),
            Beat(
                name="Final Image",
                chapter=15,
                act=6,
                word_count=1500,
                screenplay_page=110,
                example="A mirror of the opening image, showing how the world has changed",
                prompts=[
                    "How can you visually represent the protagonist's transformation?",
                    "What symbolic elements reflect the journey's completion?",
                    "How does this final image reinforce the story's theme and genre?",
                ],
                character_focus="Protagonist's transformed state",
                theme_connection="Visual representation of thematic resolution",
            ),
        ],
        "flexibility_options": {
            "save_the_cat_variations": [
                "Traditional 'saving' action (helping someone in need)",
                "Moral choice that demonstrates good character",
                "Moment of vulnerability that humanizes the protagonist",
                "Skill demonstration that impresses the audience",
                "Humorous or charming interaction that showcases personality",
            ],
            "genre_adaptations": {
                "romance": "Emphasize relationship development in B Story",
                "mystery": "Incorporate red herrings and clues throughout Fun and Games",
                "action": "Increase intensity and pacing of Bad Guys Close In",
                # Add more genre-specific adaptations
            },
            "non_linear_storytelling": [
                "Rearrange beats for flashback-heavy narratives",
                "Split beats across multiple timelines for parallel storytelling",
                "Use thematic connections to link non-chronological scenes",
            ],
            "multi_protagonist": [
                "Alternate beats between protagonists",
                "Create shared and individual character arcs",
                "Ensure thematic consistency across multiple journeys",
            ],
            "character_development_tools": [
                "Character-specific goals for each beat",
                "Tracking of internal and external conflicts",
                "Relationship dynamics map",
                "Character belief system evolution",
            ],
        },
        "theme_development": {
            "tracking": "Note how each beat contributes to or challenges the theme",
            "variations": "Explore different facets of the theme through supporting characters",
            "resolution": "Ensure the Finale and Final Image strongly reinforce the theme",
        },
        "character_development_tools": [
            "Character-specific goals for each beat",
            "Tracking of internal and external conflicts",
            "Relationship dynamics map",
            "Character belief system evolution",
        ],
    }
    HERO_JOURNEY = {
        "name": "Hero's Journey",
        "total_word_count": 80000,
        "beats": [
            Beat(name="Ordinary World",
                 chapter=1, act=1, word_count=5000,
                 example="Introduce the hero in their familiar environment",
                 prompts=[
                     "How can you establish the hero's normal life and routines?",
                     "What aspects of the Ordinary World contrast with the coming adventure?",
                 ]),
            Beat(name="Call to Adventure",
                 chapter=2, act=1, word_count=3000,
                 example="Present the inciting incident that disrupts the hero's world",
                 prompts=[
                     "What event or message challenges the hero to leave their comfort zone?",
                     "How does this call relate to the larger conflict or quest?",
                 ]),
            Beat(name="Refusal of the Call",
                 chapter=3, act=1, word_count=2000,
                 example="Show the hero's reluctance or fear to accept the challenge",
                 prompts=[
                     "What fears or doubts make the hero hesitate?",
                     "How can you increase the stakes to make the call more urgent?",
                 ]),
            Beat(name="Meeting the Mentor",
                 chapter=4, act=2, word_count=3000,
                 example="Introduce a wise figure who provides guidance or aid",
                 prompts=[
                     "Who or what can serve as a mentor figure for your hero?",
                     "What crucial advice or tool does the mentor provide?",
                 ]),
            Beat(name="Crossing the Threshold",
                 chapter=5, act=2, word_count=4000,
                 example="The hero commits to the adventure and enters the new world",
                 prompts=[
                     "What action signifies the hero's commitment to the journey?",
                     "How does crossing the threshold change the hero's perspective?",
                 ]),
            Beat(name="Tests, Allies, and Enemies",
                 chapter=6, act=3, word_count=15000,
                 example="The hero faces challenges and meets various characters",
                 prompts=[
                     "What trials can test the hero's resolve and abilities?",
                     "Who are the key allies and enemies the hero encounters?",
                 ]),
            Beat(name="Approach to the Inmost Cave",
                 chapter=7, act=3, word_count=8000,
                 example="The hero prepares for the major challenge ahead",
                 prompts=[
                     "What final preparations does the hero make?",
                     "How can you build tension as the hero approaches the central conflict?",
                 ]),
            Beat(name="Ordeal",
                 chapter=8, act=4, word_count=10000,
                 example="The hero faces their greatest challenge or fear",
                 prompts=[
                     "What is the central conflict or challenge the hero must overcome?",
                     "How does this ordeal force the hero to confront their deepest fears?",
                 ]),
            Beat(name="Reward (Seizing the Sword)",
                 chapter=9, act=4, word_count=5000,
                 example="The hero achieves their goal and gains new power or insight",
                 prompts=[
                     "What tangible or intangible reward does the hero gain?",
                     "How has the hero changed as a result of their ordeal?",
                 ]),
            Beat(name="The Road Back",
                 chapter=10, act=5, word_count=8000,
                 example="The hero begins their return journey, facing the consequences of their actions",
                 prompts=[
                     "What new challenges arise as the hero tries to return?",
                     "How do the stakes escalate during the return journey?",
                 ]),
            Beat(name="Resurrection",
                 chapter=11, act=5, word_count=10000,
                 example="The hero faces a final test, often mirroring the opening crisis",
                 prompts=[
                     "How can the hero apply what they've learned in a final, climactic challenge?",
                     "What sacrifice might be required for the hero to truly transform?",
                 ]),
            Beat(name="Return with the Elixir",
                 chapter=12, act=6, word_count=7000,
                 example="The hero returns transformed, bringing something to benefit the Ordinary World",
                 prompts=[
                     "How has the hero's success changed their original world?",
                     "What wisdom or power does the hero bring back to share?",
                 ]),
        ],
    }
    THREE_ACT_STRUCTURE = {
        "name": "Three-Act Structure",
        "total_word_count": 75000,
        "beats": [
            Beat(name="Exposition",
                 chapter=1, act=1, word_count=10000,
                 example="Introduce the protagonist and their world",
                 prompts=[
                     "How can you establish the protagonist's normal life and key relationships?",
                     "What aspects of the world or setting are crucial for the reader to understand?",
                     "How can you hint at the protagonist's flaws or internal conflicts?",
                 ]),
            Beat(name="Inciting Incident",
                 chapter=2, act=1, word_count=5000,
                 example="An event that disrupts the protagonist's normal life",
                 prompts=[
                     "What unexpected event or challenge forces the protagonist out of their comfort zone?",
                     "How does this incident relate to the overall conflict of the story?",
                     "What immediate consequences does this incident have for the protagonist?",
                 ]),
            Beat(name="First Plot Point",
                 chapter=3, act=1, word_count=5000,
                 example="The protagonist commits to addressing the central conflict",
                 prompts=[
                     "What decision or action marks the protagonist's commitment to the main conflict?",
                     "How does this choice change the trajectory of the story?",
                     "What stakes are established for the protagonist at this point?",
                 ]),
            Beat(name="Rising Action",
                 chapter=4, act=2, word_count=20000,
                 example="The protagonist faces escalating challenges and obstacles",
                 prompts=[
                     "What series of challenges can test the protagonist's resolve and abilities?",
                     "How can you introduce new characters or subplots that complicate the main conflict?",
                     "What internal struggles does the protagonist face as they confront these challenges?",
                 ]),
            Beat(name="Midpoint",
                 chapter=5, act=2, word_count=5000,
                 example="A major event that changes the protagonist's perspective or approach",
                 prompts=[
                     "What significant event or revelation occurs at the middle of the story?",
                     "How does this event raise the stakes or change the nature of the conflict?",
                     "How does the protagonist's understanding of their goal or conflict shift?",
                 ]),
            Beat(name="Continued Rising Action",
                 chapter=6, act=2, word_count=15000,
                 example="Challenges intensify as the story builds towards the climax",
                 prompts=[
                     "How do the antagonistic forces gain strength or complexity?",
                     "What personal sacrifices must the protagonist make as the conflict deepens?",
                     "How can you increase tension and pacing as the story progresses?",
                 ]),
            Beat(name="Pre-Climax Setback",
                 chapter=7, act=3, word_count=5000,
                 example="The protagonist faces a major setback or moment of doubt",
                 prompts=[
                     "What significant obstacle or failure does the protagonist encounter?",
                     "How does this setback challenge the protagonist's resolve or beliefs?",
                     "What internal or external resources does the protagonist draw upon to continue?",
                 ]),
            Beat(name="Climax",
                 chapter=8, act=3, word_count=7500,
                 example="The final confrontation or highest point of tension",
                 prompts=[
                     "How does the protagonist face the central conflict in a decisive way?",
                     "What surprising twists or revelations occur during this climactic moment?",
                     "How does the climax bring together various plot threads and character arcs?",
                 ]),
            Beat(name="Falling Action",
                 chapter=9, act=3, word_count=1500,
                 example="The immediate aftermath of the climax",
                 prompts=[
                     "What are the immediate consequences of the climactic events?",
                     "How do the characters react to the outcome of the climax?",
                     "What loose ends begin to be tied up in this section?",
                 ]),
            Beat(name="Resolution",
                 chapter=10, act=3, word_count=1000,
                 example="The new equilibrium established after the conflict",
                 prompts=[
                     "How has the world of the story changed as a result of the events?",
                     "What final character growth or realizations are shown?",
                     "How can you provide a sense of closure while potentially leaving room for future stories?",
                 ]),
        ],
    }
    SEVEN_BASIC_PLOTS = """
        According to Christopher Booker's influential book "The Seven Basic Plots: Why We Tell Stories", there are seven fundamental narrative frameworks that are recycled in fiction:
        - Overcoming the Monster
        - Rags to Riches
        - The Quest
        - Voyage and Return
        - Rebirth
        - Comedy
        - Tragedy
        """

    MYSTERY_DETECTIVE = {
        "name": "Mystery/Detective",
        "total_word_count": 75000,
        "beats": [
            Beat(name="The Crime",
                 chapter=1, act=1, word_count=5000,
                 example="A valuable painting is stolen from a museum",
                 prompts=[
                     "How can you introduce the crime in a way that immediately grabs the reader's attention?",
                     "What unique aspects of the crime can you highlight to make it intriguing?",
                     "How can you plant subtle clues or red herrings in the crime scene description?",
                 ]),
            Beat(name="Introduce the Detective",
                 chapter=2, act=1, word_count=3000,
                 example="Meet Detective Sarah Chen, art theft specialist",
                 prompts=[
                     "How can you introduce the detective's unique skills or quirks that will be relevant to solving the case?",
                     "What personal stake could the detective have in this particular case?",
                     "How can you hint at the detective's flaws or past failures that might complicate the investigation?",
                 ]),
            Beat(name="Initial Investigation",
                 chapter=3, act=1, word_count=7000,
                 example="Sarah examines the crime scene and interviews key witnesses",
                 prompts=[
                     "What initial clues or inconsistencies can the detective discover?",
                     "How can you introduce potential suspects during the initial investigation?",
                     "What obstacles or complications arise during this first phase of the investigation?",
                 ]),
            Beat(name="Deepen the Mystery",
                 chapter=4, act=2, word_count=10000,
                 example="Sarah uncovers connections to other art thefts",
                 prompts=[
                     "How can you introduce new information that complicates the initial understanding of the crime?",
                     "What unexpected connections or patterns could the detective discover?",
                     "How can you raise the stakes or expand the scope of the investigation?",
                 ]),
            Beat(name="Red Herring",
                 chapter=5, act=2, word_count=8000,
                 example="Evidence points to a famous art collector",
                 prompts=[
                     "How can you create a convincing false lead that temporarily misleads the detective?",
                     "What seemingly damning evidence supports this red herring?",
                     "How does pursuing this false lead create new problems or reveal new information?",
                 ]),
            Beat(name="Subplot Development",
                 chapter=6, act=2, word_count=7000,
                 example="Sarah struggles with departmental politics threatening her case",
                 prompts=[
                     "What personal or professional challenges can you introduce for the detective?",
                     "How does this subplot intersect with or complicate the main investigation?",
                     "What character growth opportunities does this subplot present for the detective?",
                 ]),
            Beat(name="Major Plot Twist",
                 chapter=7, act=3, word_count=8000,
                 example="The stolen painting is discovered to be a forgery",
                 prompts=[
                     "What significant revelation can upend the detective's understanding of the case?",
                     "How does this twist force the detective to reevaluate previous assumptions?",
                     "What new questions or avenues of investigation does this twist open up?",
                 ]),
            Beat(name="Raising the Stakes",
                 chapter=8, act=3, word_count=7000,
                 example="A key witness is found dead",
                 prompts=[
                     "How can you increase the urgency or danger of the situation?",
                     "What personal consequences might the detective face if they fail to solve the case?",
                     "How does this development change the nature of the investigation?",
                 ]),
            Beat(name="The Sleuth Puts It All Together",
                 chapter=9, act=3, word_count=10000,
                 example="Sarah connects the dots and formulates a theory",
                 prompts=[
                     "How does the detective synthesize all the clues and information gathered?",
                     "What crucial insight or realization leads to the breakthrough?",
                     "How can you show the detective's unique skills or methods in solving the puzzle?",
                 ]),
            Beat(name="Climax/Confrontation",
                 chapter=10, act=4, word_count=8000,
                 example="Sarah confronts the true mastermind behind the art thefts",
                 prompts=[
                     "How can you create a tense and dramatic confrontation between the detective and the culprit?",
                     "What final twist or revelation can you include in this confrontation?",
                     "How does the detective ultimately outwit or outmaneuver the antagonist?",
                 ]),
            Beat(name="Resolution/Denouement",
                 chapter=11, act=4, word_count=2000,
                 example="The case is closed, and loose ends are tied up",
                 prompts=[
                     "How can you explain any remaining mysteries or unanswered questions?",
                     "What consequences do the various characters face as a result of the investigation?",
                     "How has solving this case changed the detective or their world?",
                 ]),
        ]
    }
    # ... (other genres)
    CRIME = {"name": "Crime"}
    THRILLER = {"name": "Thriller"}
    MYSTERY = {"name": "Mystery"}
    FANTASY = {"name": "Fantasy"}
    SCI_FI = {"name": "Sci-Fi"}
    YOUNG_ADULT = {"name": "Young Adult"}
    HORROR = {"name": "Horror"}
    HISTORICAL_FICTION = {"name": "Historical Fiction"}
    CHILDRENS = {"name": "Children"}
    INSPIRATIONAL = {"name": "Inspirational"}
    RELIGIOUS = {"name": "Religious"}
    EROTICA = {"name": "Erotica"}
    ACTION = {"name": "Action"}
    ADVENTURE = {"name": "Adventure"}
    COMEDY = {"name": "Comedy"}
    DRAMA = {"name": "Drama"}
    POETRY = {"name": "Poetry"}
    WESTERN = {"name": "Western"}
    DYSTOPIAN = {"name": "Dystopian"}
    PARANORMAL = {"name": "Paranormal"}

    # Dictionary mapping genre names (strings) to their best-selling or notable authors
    GENRE_TO_AUTHORS: Dict[str, list[str]] = {
        ROMANCE["name"]: ["Danielle Steel", "Nora Roberts"],
        CRIME["name"]: ["James Patterson", "Michael Connelly"],
        THRILLER["name"]: ["John Grisham", "Dan Brown"],
        MYSTERY["name"]: ["Agatha Christie", "Arthur Conan Doyle"],
        FANTASY["name"]: ["J.R.R. Tolkien", "George R.R. Martin"],
        SCI_FI["name"]: ["Isaac Asimov", "Arthur C. Clarke"],
        YOUNG_ADULT["name"]: ["J.K. Rowling", "Suzanne Collins"],
        HORROR["name"]: ["Stephen King", "Dean Koontz"],
        HISTORICAL_FICTION["name"]: ["Hilary Mantel", "Ken Follett"],
        CHILDRENS["name"]: ["Roald Dahl", "Dr. Seuss"],
        INSPIRATIONAL["name"]: ["Oprah Winfrey", "Eleanor Roosevelt", "Michelle Obama"],
        RELIGIOUS["name"]: ["Rick Warren", "C.S. Lewis"],
        EROTICA["name"]: ["E.L. James", "Sylvia Day"],
        ACTION["name"]: ["Lee Child", "Tom Clancy"],
        ADVENTURE["name"]: ["Michael Crichton", "Clive Cussler"],
        COMEDY["name"]: ["P.G. Wodehouse", "Terry Pratchett"],
        DRAMA["name"]: ["Tennessee Williams", "Arthur Miller"],
        POETRY["name"]: ["Sylvia Plath", "Robert Frost"],
        WESTERN["name"]: ["Louis L'Amour", "Zane Grey"],
        DYSTOPIAN["name"]: ["Margaret Atwood", "George Orwell"],
        PARANORMAL["name"]: ["Charlaine Harris", "Laurell K. Hamilton"]
        }

    # Dictionary mapping genre names (strings) to a list of popular books
    GENRE_TO_BOOKS: Dict[str, List[str]] = {
        ROMANCE["name"]: ["Pride and Prejudice", "Outlander"],
        CRIME["name"]: ["The Silence of the Lambs", "In Cold Blood"],
        THRILLER["name"]: ["The Da Vinci Code", "The Girl with the Dragon Tattoo"],
        MYSTERY["name"]: ["The Adventures of Sherlock Holmes", "The Hound of the Baskervilles"],
        FANTASY["name"]: ["A Game of Thrones", "The Hobbit"],
        SCI_FI["name"]: ["Dune", "2001: A Space Odyssey"],
        YOUNG_ADULT["name"]: ["The Hunger Games", "The Fault in Our Stars"],
        HORROR["name"]: ["The Shining", "Dracula"],
        HISTORICAL_FICTION["name"]: ["Wolf Hall", "The Nightingale"],
        CHILDRENS["name"]: ["Harry Potter and the Philosopher's Stone", "Matilda"],
        INSPIRATIONAL["name"]: ["The Alchemist", "The Power of Now"],
        RELIGIOUS["name"]: ["The Chronicles of Narnia", "Mere Christianity"],
        EROTICA["name"]: ["Fifty Shades of Grey", "Story of O"],
        ACTION["name"]: ["Killing Floor", "The Bourne Identity"],
        ADVENTURE["name"]: ["Jurassic Park", "The Lost City of Gold"],
        COMEDY["name"]: ["The Code of the Woosters", "Catch-22"],
        DRAMA["name"]: ["A Streetcar Named Desire", "Angels in America"],
        POETRY["name"]: ["Ariel", "The Waste Land"],
        WESTERN["name"]: ["Hondo", "Lonesome Dove"],
        DYSTOPIAN["name"]: ["The Handmaid's Tale", "1984"],
        PARANORMAL["name"]: ["Dead Until Dark", "The Southern Vampire Mysteries"],
        }

    @staticmethod
    def get_best_selling_authors(genre: "Genre") -> Optional[str]:
        """
        Retrieve the best-selling authors for a given genre

        Args:
            genre: An enum value from the Genre class

        Returns:
            String representing the name of the best-selling author
                 or None if not found
        """
        return Genre.genre_to_authors.get(
            genre.value["name"], "No best-selling author defined for this genre"
        )

    @staticmethod
    def get_popular_books(genre: "Genre") -> List[str]:
        """
        Retrieve a list of popular books for the given genre.

        Args:
            genre: An enum value from the Genre class

        Returns:
            List of popular book titles or an empty list if not found
        """
        return Genre.genre_to_books.get(genre.value["name"], [])

    @staticmethod
    def get_genre_from_book(book_title: str) -> Optional["Genre"]:
        """
        Find the genre of a book based on its title.

        Args:
            book_title: Title of the book to look up

        Returns:
            The genre of the book if found, None otherwise
        """
        for genre, books in Genre.genre_to_books.items():
            if book_title in books:
                return Genre(genre)  # Convert string to Genre enum
        return None

    @classmethod
    def list_all_genres(cls) -> List['Genre']:
        """
        List all available genres.

        Returns:
            List of all Genre enum values
        """
        return list(cls)

    @classmethod
    def list_all_genres_names(cls) -> List[str]:
        """
        List all available genre names.

        Returns:
            List of all genre names
        """
        return [genre.name.lower() for genre in cls]
        #[genre.name.lower() for genre in list(cls)]

    @staticmethod
    def is_genre_valid(genre_name: str) -> bool:
        """
        Check if a given string is a valid genre name.

        Args:
            genre_name: String to check against genre names

        Returns:
            True if the string matches a genre name, False otherwise
        """
        return any(genre_name.lower() == g.value["name"].lower() for g in Genre)

    # Example usage:
    def test():
        for genre in Genre:
            print(
                f"Popular authors in {genre.value['name']}: "
                f"{genre.get_best_selling_authors(genre)}"
            )
            print(
                f"Popular books in {genre.value['name']}: "
                f"{genre.get_popular_books(genre)}"
            )
            print()

class WritingPromptLibrary:
    """
    A library of dynamic prompts for creative content generation.

    Stores a collection of dynamic prompt templates that focus on different aspects of writing style and tone

    # @TODO: implement dynamic prompting system to add variety to creative content
      - study the list of dynamic prompts below, when one calls for a setup,
        decide when and how to implement the payoff at a later time.
    """

    def get_all_prompt_categories(self) -> List[List[str]]:
        """
        Returns a list of all prompt categories (lists of prompts) in the class.
        """
        all_categories = []
        for attribute_name in dir(self):
            attribute = getattr(self, attribute_name)
            if isinstance(attribute, list):
                all_categories.append(attribute)
        return all_categories

    def select_random_prompts(self, num_prompts: int = 12,) -> List[str]:
        """
        Selects a specified number of prompts randomly from all categories.
        """
        all_prompts = []
        for category in self.get_all_prompt_categories():
            all_prompts.extend(category)
        return random.sample(all_prompts, num_prompts)

    def get_stringified_prompt_menu(self, num_prompts: int = 12,) -> str:
        """
        Returns a list of random prompts into a single string, separated by newlines.
        """
        random_prompts = self.select_random_prompts(num_prompts)
        stringified_random_prompts = chr(10).join(random_prompts)
        return stringified_random_prompts

    # Descriptive Prompts
    DESCRIPTIVE_PROMPTS = [
        "Describe the scene with vivid imagery, appealing to the reader's senses.",
        "Use figurative language (metaphors, similes) to create a more evocative description.",
        "Show the character's emotions through their actions and expressions, rather than stating them directly."
        "Describe the scene with vivid imagery, appealing to the reader's senses.",
        "Use figurative language (metaphors, similes) to create a more evocative description.",
        "Show the character's emotions through their actions and expressions, rather than stating them directly.",
        "Incorporate sensory details that are often overlooked, such as textures, scents, or ambient sounds.",
        "Use specific, concrete nouns instead of vague, general terms to paint a clearer picture.",
        "Vary sentence structure and length to create rhythm and emphasize certain elements of the description.",
        "Describe the setting through the perspective of a character, revealing their state of mind.",
        "Use color and light to set the mood and atmosphere of the scene.",
        "Incorporate movement and change to bring the scene to life, rather than presenting a static image.",
        "Focus on small, telling details that reveal something significant about the setting or characters.",
        "Use weather and natural elements to reflect or contrast with the emotional tone of the scene.",
        "Describe how the environment affects the characters physically and emotionally.",
        "Use unexpected or unconventional comparisons to make familiar objects or scenes feel fresh and new.",
        "Incorporate all three dimensions in your descriptions, including often-forgotten elements like ceilings or the space above characters.",
        "Use synesthesia (mixing sensory descriptions) to create unique and memorable imagery.",
        ]

    # Dialogue Prompts
    DIALOGUE_PROMPTS = [
        "Write a dialogue exchange that reveals the characters' personalities and their relationship dynamics.",
        "Use subtext and unspoken emotions in the dialogue to create tension and intrigue.",
        "Avoid clichés and use fresh, original language in the dialogue.",
        "Write a dialogue exchange that reveals the characters' personalities and their relationship dynamics.",
        "Use subtext and unspoken emotions in the dialogue to create tension and intrigue.",
        "Avoid clichés and use fresh, original language in the dialogue.",
        "Create distinct voices for each character, reflecting their background, education, and personality.",
        "Use interruptions, pauses, and silences to add realism and tension to the conversation.",
        "Incorporate body language and facial expressions to complement or contradict what characters are saying.",
        "Use dialect, slang, or jargon appropriate to the character's background, but be careful not to overdo it.",
        "Write a conversation where what isn't said is as important as what is said.",
        "Use dialogue to advance the plot or reveal important information, but make it feel natural and unforced.",
        "Create conflict or tension between characters through their dialogue, even if they're discussing something mundane.",
        "Use a character's choice of words to reveal their attitudes, biases, or hidden agendas.",
        "Write a conversation where characters are talking about one thing but really discussing something else entirely.",
        "Use humor in dialogue to reveal character traits or diffuse tense situations.",
        "Write dialogue that shows how characters change or adapt their way of speaking depending on who they're talking to.",
        "Create subtext by having characters say the opposite of what they really mean or feel.",
        ]

    # Character Development Prompts
    CHARACTER_DEVELOPMENT_PROMPTS = [
        "Write a scene that reveals a character's deepest fear or insecurity.",
        "Write a scene that reveals a character's motivations and aspirations.",
        "Show a character's growth or change through their actions in a key moment.",
        "Describe a character's typical day to reveal their habits, routines, and priorities.",
        "Write a flashback that explains a character's current behavior or motivation.",
        "Write a scene that references a prior event that happened to a character that explains their current behavior.",
        "Write a scene that references a prior relationship or set of circumstances that explains the current behavior of a character.",
        "Create an internal monologue that exposes a character's true thoughts and feelings.",
        "Show how a character reacts to an unexpected challenge or setback.",
        "Describe a character's living space in a way that reveals their personality and history.",
        "Write a scene where a character's public persona clashes with their private self.",
        "Create a moment where a character must make a difficult decision, revealing their values and priorities.",
        "Show how a character's background and upbringing influence their current actions and beliefs.",
        "Write a scene that demonstrates a character's personality traits."
        "Write a scene that makes the audience bond emotionally with the character (i.e., 'save the cat' but not literally!)",
        "Write a scene that illustrates how the personality types (MBTI) of the main characters interact to create tension.",
        "Write a scene that illustrates how the personality types (MBTI) of the main characters interact to propel a plot point.",
        ]

    # Plot and Pacing Prompts
    PLOT_AND_PACING_PROMPTS = [
        "Write a scene that introduces a surprising plot twist.",
        "Create a 'ticking clock' scenario to increase tension and urgency in the story.",
        "Develop a subplot that intertwines with and enhances the main plot.",
        "Write a scene that foreshadows a future event or revelation.",
        "Create a moment of calm before a major conflict or climax.",
        "Introduce a new obstacle that forces the protagonist to change their plans.",
        "Write a scene that brings multiple plot threads together.",
        "Create a cliffhanger ending for a chapter or section.",
        "Create an ending that contains a 'hook,' preview or flashforward into an important feature of the next chapter or section to give a 'page-turner' effect.",
        "Develop a series of escalating challenges for your protagonist to overcome.",
        "Write a scene that resolves one conflict while introducing another.",
        "Open a loop: Introduce a question, mystery, or unresolved conflict that will be addressed later in the story. This could involve a character's secret, a foreshadowed event, a missing piece of information, or a challenge that remains unresolved at the end of the scene.",
        "Close a loop: Provide an answer, solution, or resolution to a previously opened loop. This might involve revealing a character's secret, showing the consequences of a past event, or resolving a conflict that was introduced earlier in the story.",
        ]

    # Tone and Atmosphere Prompts
    TONE_AND_ATMOSPHERE_PROMPTS = [
        "Write a scene that establishes a specific mood or atmosphere (e.g., eerie, joyful, melancholic).",
        "Write a scene that gives the reader a building sense of dread or doom about what will happen to the characters",
        "Use weather or natural phenomena to enhance the emotional tone of a scene.",
        "Create contrast between the tone of the narration and the events being described.",
        "Write a scene where the atmosphere gradually shifts from one mood to another.",
        "Use sensory details to create a specific atmosphere without explicitly stating the mood.",
        "Write a scene where the tone is primarily conveyed through dialogue and character interactions.",
        "Create a moment where the tone of the scene ironically juxtaposes the events taking place.",
        "Use pacing and sentence structure to reinforce the tone of a tense or relaxed scene.",
        "Write a scene where multiple characters perceive the atmosphere differently.",
        "Create a recurring motif or symbol that reinforces the overall tone of your story.",
        ]

    # Setting and Worldbuilding Prompts
    SETTING_AND_WORLDBUILDING_PROMPTS = [
        "Describe a unique cultural practice or tradition in your story's world.",
        "Create a legend or myth that's significant to your story's setting.",
        "Describe how technology or magic (if applicable) affects everyday life in your world.",
        "Write a scene that showcases the political or social structure of your story's society.",
        "Describe an important historical event that shaped your story's world.",
        "Create a unique ecosystem or environment that affects your characters' lives.",
        "Write a scene that reveals the economic systems or trade practices in your world.",
        "Describe how the geography of your setting influences the plot or characters.",
        "Create a scene that showcases the art, music, or entertainment of your story's world.",
        "Write about how seasons or celestial events affect life in your story's setting.",
        ]

    def __init__(self):
        # ... (initialize other attributes as needed)

      # ... (add more prompt categories as needed)
      pass

    def get_random_prompt(self, category: str) -> str:
        """
        Returns a random prompt from the specified category.

        @TODO: alternatively, feed all the prompts at inference time along with an instruction
               to leverage as many of the prompts shown as necessary to write the most compelling
               chapter possible. or create new prompts internal to its own thoughts and write
               the chapter (or section) accordingly.
        """
        if category == "descriptive":
            return random.choice(PromptLibrary.DESCRIPTIVE_PROMPTS)
        elif category == "dialogue":
            return random.choice(PromptLibrary.DIALOGUE_PROMPTS)
        # ... (handle other categories)
        else:
            raise ValueError(f"Invalid prompt category: {category}")

    def generate_character_introduction_prompt(self) -> str:
        """
        Generates a prompt for introducing a new character.
        "" "
        Example 1:
            prompt = f"" "Introduce a new character named {character.name}.
                She is a {character.age}-year-old {character.species_race} who is known for her {character.positive_traits[0]} personality.
                She lives in the {character.place_of_origin} and has a deep fear of {character.fears[0]}.
                Make her entrance memorable and hint at a connection to the main plot."" "
            return prompt
        Example 2:
            Character-Focused Prompts: When generating chapters, include prompts that focus on individual characters:
                "Describe [character's] reaction to the events, highlighting their unique personality traits."
                "Explore [character's] internal conflict in this scene, showing their struggle with [specific dilemma]."
                "Use dialogue to reveal [character's] backstory and how it influences their current actions."

        """
        pass

class DynamicElements:
    # Element Banks
    GENRES = [
        "Romance", "Fantasy", "Sci-Fi", "Mystery", "Thriller", "Horror",
        "Historical Fiction", "Young Adult", "Action", "Comedy", "Drama",
        "Dystopian", "Cyberpunk", "Steampunk", "Magical Realism", "Western",
        "Post-Apocalyptic", "Supernatural", "Urban Fantasy", "Space Opera"
        ]

    SETTINGS = {
        "Time Periods": [
            "Ancient", "Medieval", "Renaissance", "Victorian", "Modern", "Near Future", "Far Future", "Prehistoric Era",
            "Bronze Age", "Industrial Revolution", "World War II", "Cold War", "Information Age", "Post-Singularity",
            "Ancient Greece", "Medieval Europe", "Renaissance Italy", "Victorian England", "1920s America", "Present Day",
            "Near Future", "Distant Future", "Alternate 1980s", "Post-Apocalyptic",
            ],
        "Locations": [
            "Urban Metropolis", "Rural Farmland", "Tropical Island", "Space Station", "Underwater City", "Fantasy Kingdom",
            "Desert Wasteland", "Arctic Tundra", "Volcanic Landscape", "Floating Sky City", "Subterranean Cave Network",
            "Virtual Reality World", "Parallel Universe", "Time Loop", "Bustling Metropolis", "Quiet Small Town",
            "Remote Island", "Orbital Space Station", "Underwater City", "Floating Sky Kingdom", "Barren Desert",
            "Lush Rainforest", "Arctic Research Base", "Virtual Reality World", "Parallel Universe",
            ],
        "Environments": [
            "Lush Jungle", "Barren Wasteland", "Neon-lit Cityscape", "Ethereal Dreamscape", "Toxic Swamp", "Crystal Cavern",
            "Dimensional Nexus", "Quantum Realm", "Sentient Ocean", "Living Spaceship", "Neon-lit Cyberpunk Streets", "Misty Enchanted Forest",
            "Post-apocalyptic Wasteland", "Steampunk Airship Docks", "Alien Coral Reefs", "Temporal Nexus", "Interdimensional Bazaar",
            "Haunted Victorian Mansion", "Bioluminescent Cave System",
            ]
        }

    CONFLICTS_1 = [
        "Man vs. Man", "Man vs. Nature", "Man vs. Self", "Man vs. Society", "Man vs. Technology",
        "Man vs. Time", "Man vs. World", "Man vs. Other", "Man vs. Fate", "Man vs. Supernatural",
        "Man vs. Reality", "Man vs. Machine", "Order vs. Chaos", "Science vs. Magic",
        "Man vs. Events", "Man vs. Circumstances", "Circumstances vs. Nature", "Man vs. Identity",
        "Tradition vs. Progress", "Loyalty vs. Betrayal", "Love vs. Duty",
        ]

    CONFLICTS_2 = [
        ("Ethical Dilemma", "Character must choose between two morally ambiguous options"),
        ("Race Against Time", "A crucial task must be completed before a deadline"),
        ("Battle of Ideologies", "Competing worldviews clash with far-reaching consequences"),
        ("Hidden Conspiracy", "Unraveling a web of secrets that threatens everything"),
        ("Man vs. Nature", "Surviving against overwhelming natural forces"),
        ("Technological Upheaval", "Adapting to a world transformed by new technology"),
        ("Cultural Clash", "Navigating between vastly different societies"),
        ("Internal Struggle", "Overcoming personal demons or limitations"),
        ("Power Struggle", "Competing for control in a chaotic situation"),
        ("Moral Corruption", "Resisting the temptation to compromise one's principles")
        ]

    THEMES_1 = [
        "Love", "Loss", "Justice", "Freedom", "Identity", "Power", "Family",
        "Redemption", "Coming of Age", "Power Corrupts", "Love Conquers All", "Man's Inhumanity to Man",
        "The Nature of Reality", "The Price of Progress", "The Cycle of Violence", "The Search for Identity",
        "The Burden of Knowledge", "The Illusion of Choice", "The Struggle for Survival", "The Power of Memory",
        "The Weight of Legacy", "The Danger of Unchecked Ambition",
        ]

    THEMES_2 = [
        ("Redemption", "A journey from guilt to forgiveness"),
        ("Identity", "Discovering one's true self amidst confusion"),
        ("Power and Corruption", "The dangers of unchecked authority"),
        ("Love and Sacrifice", "The costs and rewards of deep connections"),
        ("Man vs. Technology", "The double-edged sword of progress"),
        ("The Nature of Reality", "Questioning the fabric of existence"),
        ("Social Inequality", "Confronting systemic injustices"),
        ("The Burden of Knowledge", "Coping with uncomfortable truths"),
        ("The Cycle of Violence", "Breaking free from destructive patterns"),
        ("The Illusion of Choice", "Fate versus free will in shaping destiny")
        ]

    PLOT_TWISTS = [
        "Unexpected Betrayal", "Hidden Identity Revealed", "Time Loop Discovered", "Parallel Universe Crossover",
        "Protagonist is the Antagonist", "All Events Were a Simulation", "Dead Character Returns", "Prophecy Misinterpreted",
        "Apparent Villain is the True Hero", "Reality is an Illusion", "Unexpected Family Connection", "Last-Minute Ally Switch",
        "Everything Was Orchestrated by an Unseen Force", "The Whole Story is a Book Within a Book",
        "The antagonist is revealed to be the protagonist's long-lost sibling",
        "The entire story is revealed to be a simulation or dream",
        "A key character thought to be dead returns at a crucial moment",
        "The protagonist discovers they are actually a clone/robot/alien",
        "The conflict is revealed to be orchestrated by a hidden third party",
        "A seemingly insignificant character is revealed to be the mastermind",
        "The story's reality merges with a parallel universe",
        "The protagonist and antagonist must work together to face a greater threat",
        "A prophecy or prediction is fulfilled, but in an unexpected way",
        "The story is revealed to be a book/film within the world itself"
        ]

    INCITING_INCIDENTS = [
        "a [Catastrophic Event] strikes",
        "the discovery of [Game-Changing Object/Information]",
        "the sudden disappearance of [Crucial Element/Person]",
        ]

    def __init__(self):
        pass

    def generate_random_name():
        """
        Generate a random adjective-noun name.
        """
        # Generate a random adjective-noun name
        random_name = randomname.get_name()
        print(f"Random name: {random_name}")

        # Generate a name with specific categories
        specific_name = randomname.get_name(adj=('weather',), noun=('cats', 'food'))
        print(f"Specific category name: {specific_name}")

        # Generate a more complex name
        complex_name = randomname.generate('v/fire', 'adj/music_theory', ('n/cats', 'n/food'))
        print(f"Complex name: {complex_name}")

# </dynamic_writing>

# <feedback_loop>

@dataclass
class Feedback:
    """
    Feedback for a story draft.
    """
    plot: int
    characters: int
    worldbuilding: int
    writing_style: int
    engagement: int
    recommendations: List[str]

class WritingQualityEvaluator:
    """
    A structured approach for developing feedback for a story draft.

    Facilitates iterative improvements.

    @TODO: May require creatiing a rubric to benchmark story quality.

    Attributes:
        plot: int
        characters: int
        worldbuilding: int
        writing_style: int
        engagement: int
        recommendations: List[str]

    Example output:

        **Strengths:**

        * **Fast-Paced Plot:** The story moves quickly, keeping the reader engaged. The "meet-cute" happens early, and the conflict between Winnie and Pacey is established quickly.
        * **Clear Conflict:** The central conflict (Winnie's past relationship with Pacey's half-brother) is clear and creates immediate tension.
        * **Humorous Elements:** The story includes some lighthearted and humorous moments, particularly in the interactions between Winnie and the hockey players.
        * **Romantic Tension:** The attraction between Winnie and Pacey is evident, and the story builds anticipation for their eventual resolution.

        **Weaknesses:**

        * **Clichéd Premise:** The "cabin in the woods" trope and the "one of them is my ex's half-brother" conflict are fairly common in romance novels.
        * **Underdeveloped Characters:** The characters, particularly the hockey players, are not very well-developed. They mostly serve as background figures or comedic relief.
        * **Lack of Depth:** The story focuses primarily on the surface-level romance and doesn't explore deeper themes or character motivations in much detail.
        * **Repetitive Dialogue:** Some of the dialogue feels repetitive, especially the banter between Winnie and Pacey.
        * **Weak Ending:** The ending feels rushed and somewhat anticlimactic. The resolution of the conflict is a bit too easy and convenient.

        **Scaling and Scoring:**

        It's difficult to provide a precise numerical score for story quality, as it's subjective. However, on a scale of 1 to 10 (with 1 being very poor and 10 being excellent), I would rate this story around a **5 or 6**.

        **Overall Critique:**

        The story is a light and entertaining read, but it lacks depth and originality. The characters are not very memorable, and the plot relies on familiar tropes. The writing is competent but not particularly engaging.

        **Recommendations for Improvement:**

        * **Develop Characters More Fully:** Give the characters, especially the hockey players, more distinct personalities, backstories, and motivations.
        * **Explore Deeper Themes:** Consider adding more depth to the story by exploring themes of family, forgiveness, or personal growth.
        * **Make the Conflict More Complex:** Add more layers to the central conflict and make the resolution more challenging and meaningful.
        * **Vary the Dialogue:** Use more descriptive language and avoid repetitive phrases in the dialogue.
        * **Craft a More Satisfying Ending:** Spend more time on the ending and make sure it provides a sense of closure and emotional resonance.

    @TODO: Here are some strategies and code modifications we can implement:
        Develop Characters More Fully
        Explore Deeper Themes:
          - Integrate theme-related prompts into chapter generation
          - "Show how the theme of [specific theme] is explored in this scene through the characters' actions and dialogue."
          - "Use symbolism or imagery to reinforce the theme of [specific theme]."
          - "Create a moment of reflection for [character] where they grapple with the implications of [specific theme]."
        Increase plot tension with conflicts:
          - Handle multiple conflicts simultaneously
          - Make the Conflicts More Complex: Make conflicts deal with nuance
        Thicken the Plot:
          - Introduce Subplots: Add subplots that intersect with the main conflict, creating more layers and challenges.
          - Externalize Internal Conflicts: Make the characters' internal conflicts manifest in external actions and events.
          - Raise the Stakes: Increase the consequences of the characters' choices and actions.
          - Unexpected Twists: Introduce unexpected plot twists that complicate the conflict and challenge the characters' assumptions.
        Vary the Dialogue:
          - Use prompts that focus on high-quality dialogue to develop characters
          - "Write a dialogue exchange between [character 1] and [character 2] that reveals their contrasting personalities and perspectives."
          - "Use subtext and unspoken emotions in the dialogue to create tension and intrigue."
          - "Avoid clichés and use fresh, original language in the dialogue."
        Craft a More Satisfying Ending:
          - Resolution and Reflection: Ensure the ending provides a clear resolution to the conflict and allows for character reflection and growth.
          - Emotional Resonance: Aim for an ending that evokes a strong emotional response from the reader.
          - Lingering Questions: Consider leaving some subtle, thought-provoking questions unanswered to create a lasting impact.
        General Code Improvements:
          - Refactor `StoryWriter`: Break down the `generate_chapter` method into smaller, more manageable functions for each beat or story element.
          - Dynamic Prompt Library: Create a library of dynamic prompt templates that can be combined and customized based on the `WorldBible` data and story context.
        Expand Vocabulary and Sentence Structure: Using a wider range of vocabulary and varying sentence structure can make the writing more engaging and sophisticated.
        Show, Don't Tell: Instead of stating emotions directly, use descriptive language to show the characters' feelings through their actions, expressions, and internal thoughts.
        Sensory Details: Incorporate more sensory details (sight, sound, smell, taste, touch) to create a more vivid and immersive experience for the reader.
        Figurative Language: Use metaphors, similes, and other figurative language to add depth and richness to the writing.
        Dialogue Polish: Pay close attention to dialogue, ensuring it sounds natural, reveals character, and moves the plot forward.
        Fine Tunings:
          - Consider leveraging fine-tuning for genre-specific styles
          - Create a Dataset: Collect examples of well-written romance novels (or your target genre).
          - Fine-Tune the LLM: Use the dataset to fine-tune the language model, training it to generate text that aligns with the style and conventions of the genre.

    """

# </feedback_loop>

@dataclass
class deprecating_StoryWriter_v0(): # (ContentGenerator)
    """
        A class to generate a novel based on a given concept and genre.

        This class inherits from ContentGenerator and uses a language model to
        generate the title, beat map, outline, and finally the story text itself,
        following a structured approach based on the chosen genre. The story is
        generated chapter by chapter, and each chapter is assembled beat by beat,
        with an optional cliffhanger at the end.

        Attributes:
            None

        ## Dynamic Prompting:

        This class will use dynamic prompting to generate story content that is
        consistent with the `WorldBible`. Prompt templates will be combined with
        data retrieved from the `WorldBible` to create context-aware and specific
        prompts for the language model.

        Example Prompt Template:

        ```python
        prompt_template = '''Write a scene where {character_1} and {character_2} meet in the {location}.
                            {character_1} is searching for the {item}, while {character_2} is guarding the location.
                            Create a tense atmosphere and hint at a potential conflict.'''
        ```

        Example Data Retrieval Function:

        ```python
        def get_relevant_characters(self, current_scene: str) -> List[Character]:
            # ... (logic to retrieve characters based on the scene)
        ```
        """

    # class-level constants

    PERSONA = """
        <persona>
        ### Persona

        You are an award-winning author with a penchant for expansive, intricately
        woven stories. Your ultimate goal is to write the next award-winning novel.
        </persona>
        """

    WRITING_GUIDELINES = """
        <writing_guidelines>
        ### Writing Guidelines
        - Delve deeply into your story.
        - Lose yourself in the world you're building.
        - Unleash vivid descriptions to paint highly detailed scenes in your reader's mind.
        - Develop your characters patiently and fully: Let their nuances, motivations,
          emotions, and complexities unfold naturally. Build an emotional connection
          between your characters and your audience.
        - Weave in the threads of your outline, but never be fully constrained by it.
        - Use your OUTLINE to guide you whenever necessary.
        - Include as many elements of the CONCEPT as possible in your story.
        - Allow your story to surprise you as you write.
        - Use rich imagery, sensory details, and evocative language to bring the
          setting, characters, and events to life.
        - Introduce elements subtly that can blossom into complex subplots, relationships,
          or worldbuilding details later in the story.
        - Keep things intriguing but never fully resolved.
        - Avoid boxing the story into a corner too early: Plant the seeds
          of subplots or potential character arc shifts that can be expanded later.
        - Write as much as you can:
          - Expand, never summarize.
          - If you get through the story too fast, that is bad.
        - Pace your story properly: hook the audience quickly
          - Generate conflict early
            - To "hook" the audience, prioritize establishing the central conflict beat early in the story.
            - Ideally this conflict would occur within the first or second chapter.
            - As an example, in the Romance genre, this primary conflict would occur in the beat labeled 'No Way 1'.
            - Other genres might have different beat names, but the common idea is that this is the central conflict
              the main character/s spend the rest of the story attempting to resolve.
          - Create frequently spaced tension (PAGE-TURNING EFFECT)
            - Make the reader want to continue reading and not put the book down.
            - Give them a reason to "turn the page" (continue reading) by doing the following"
              - Make many of the chapters end in minor cliff-hangers by teasing the next chapter's next beat.
              - Make all the phase transitions end in major cliff-handers by teasing the next phase's turn.

        <important!>
        #### Important
        - Target word volume by category:
          - Narration: 55%
          - Dialogue: 35%
          - Internal monologue/thoughts/feelings: 10%
        - **Show, not tell** the character's emotions:
          - Describe their physical reactions
          - Include their inner thoughts and feelings
          - Use symbolic actions to convey their emotional state
        - Be liberal with your usage of dialogue between the characters.
        - Be lberal with your descriptions of the characters' inner thoughts and feelings.
        - Remember, your main goal is to build imagery and evoke the emotion and imagination in your audience.
        - Vary your sentence length and structure
          - Use a mix of short, impactful sentences and longer, more descriptive ones.
          - Mix-in different sentence types (e.g., simple, compound, complex) to create a more dynamic rhythm.
        </important!>

        <IMPORTANT!>
        GIVEN THE STORY AS A WHOLE, ONE OR MORE EXTERNAL CONFLICTS MUST FORCE THE MAIN CHARACTERS TO CONFRONT ONE OR MORE INTERNAL CONFLICTS.
        </IMPORTANT!>
        </writing_guidelines>
        """

    POPULATE_WORLD_BIBLE_FROM_CONCEPT_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Populate a WORLD BIBLE based on the following CONCEPT and GENRE.

        - Analyze the CONCEPT and identify key themes, characters, locations, and items.
        - Generate a diverse set of characters, locations, items, events, and cultures
          that fit the CONCEPT and GENRE.
        - Ensure you create at least:
            - 3 Characters
            - 2 Locations
            - 1 Item
            - 2 Events
            - 1 Culture
        - Provide detailed descriptions for each element, including:
            - **Characters:** name, age, species_race, occupation, description, personality_traits, backstory, relationships, goals, motivations, conflicts, internal_conflicts
            - **Locations:** name, type, description, history, culture, geography, landmarks, significance to the story
            - **Items:** name, description, function, history, powers, significance to the story
            - **Events:** name, description, year, impact on the world or story
            - **Cultures:** name, description, customs, beliefs, values, social structure
        - Create at least one Plot Arc that connects several of the generated elements.
        - Suggest potential relationships between elements.
        - Identify key themes that emerge from the concept.

        Format your output as a YAML list, where each item is a dictionary representing a world element.
        Use the following keys for each element type:

        - **Character:** name, age, species_race, occupation, description, personality_traits, background, relationships, goals, motivations, conflicts
        - **Location:** name, type, description, history, culture, geography, landmarks, significance
        - **Item:** name, description, function, history, powers, significance
        - **Event:** name, description, year, impact
        - **Culture:** name, description, customs, beliefs, values, social_structure
        - **PlotArc:** name, description, characters, locations, items, events

        <format_requirement>
        **IMPORTANT:**
        - Do not include any introductory or explanatory text.
        - Only output the YAML list of dictionaries.
        - Make sure the YAML is valid and can be parsed correctly.
        </format_requirement>
        </task>

        <concept>
        ### Concept:
        {{concept}}
        </concept>

        <genre>
        ### Genre:
        {{genre_yaml}}
        </genre>

        WORLD BIBLE:
        ```yaml
        """

    DEPRECATING_TITLE_PROMPT = f"""
        {PERSONA}

        You are given a detailed concept for a story:

        <concept>
        {{concept}}
        </concept>

        - Choose the most compelling title for this story.
        - The title should spark the reader's imagination: grip them, hook them and pull them in.
        - Do not give options, just the title.
        - Only return the actual title. Do not include any additional text.

        TITLE:
        """

    TITLE_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Choose the most compelling title for a story based on the following CONCEPT.
        - The title should spark the reader's imagination.
        - Do not give options, just the title.
        - Only return the actual title. Do not include any additional text.
        </task>

        <concept>
        ### Concept:
        {{concept}}
        </concept>

        TITLE:
        """

    OUTLINE_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Write a detailed OUTLINE for the plot of a {{genre_name}} story.
        The OUTLINE should:
        - Follow the typical {{genre_name}} story structure.
        - Have distinct ACTS and BEATS.
        - Be formatted in YAML (without --- separators).
        - Be structured as a list of beats.
        - Include all the beats from the provided genre definition.
        - Not change the order of the beats.
        - Be original and not copy the example plot.
        - Include as many elements as possible from the provided CONCEPT and WORLD BIBLE.
        - Prioritize pacing and establishing an emotional connection with the audience.
        </task>

        <concept>
        ### Concept:
        {{concept}}
        </concept>

        <world_bible>
        ### World Bible:
        ```yaml
        {{world_bible_yaml}}
        ```
        </world_bible>

        <title>
        ### Title:
        {{title}}
        </title>

        <genre_structure>
        ### Genre Structure:
        Here is the typical {{genre_name}} story structure with its phases and beats:
        ```yaml
        {{genre_definition}}
        ```
        </genre_structure>

        <outline_instructions>
        ### Outline Instructions:
        Each beat in the OUTLINE should be a YAML dictionary with the following keys:

        - **name:** The name of the beat (as defined in the genre definition).
        - **chapter:** The chapter number (integer).
        - **act:** The act number containing the beat (integer).
        - **description:** A list of single-sentence strings describing the beat.
          - Include at least six bullet items for each description (more is better).
          - Each description sentence should be wrapped in double quotes (").
        - **title:** A string title for the chapter based on the description elements.
        </outline_instructions>

        <outline_example>
        ### Example Outline:
        **IMPORTANT:**
        - Do not copy the plot of this example.
        - Choose different characters and plots.
        - Make original characters and plots.

        ```yaml
        - name: Intro H1
          act: 1
          chapter: 1
          description:
            - "Meet Corrina."
            - "Introduce her character and background."
            - "..."
          title: "..."
        - name: Intro H2
          act: 1
          chapter: 2
          description:
            - "Meet Nicholas."
            - "Provide details on his character and background."
            - "..."
          title: "..."
        # ... (rest of the example outline)
        ```
        </outline_example>

        OUTLINE:
        """

    SYNOPSIS_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Write a SYNOPSIS for a story based on the following CONCEPT, GENRE, TITLE, and OUTLINE.
        - The SYNOPSIS should be one to three paragraphs in length.
        - The SYNOPSIS should seek to create a desire for readers to read the book.
        </task>

        <concept>
        ### Concept:
        {{concept}}
        </concept>

        <genre>
        ### Genre:
        {{genre_name}}
        </genre>

        <title>
        ### Title:
        {{title}}
        </title>

        <outline>
        ### Outline:
        ```yaml
        {{outline}}
        ```
        </outline>

        <world_bible>
        ### World Bible:
        ```yaml
        {{world_bible_yaml}}
        ```
        </world_bible>

        SYNOPSIS:
        """

    CLIFFHANGER_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Write a cliffhanger ending for the following CHAPTER to entice the reader to continue to the next chapter.
        </task>

        <concept>
        ### Concept:
        {{concept}}
        </concept>

        <title>
        ### Title:
        {{title}}
        </title>

        <outline>
        ### Outline:
        ```yaml
        {{outline}}
        ```
        </outline>

        <chapter_text>
        ### Chapter Text:
        {{chapter_text}}
        </chapter_text>

        {WRITING_GUIDELINES}
        """

    CHAPTER_PROMPT = f"""
        {PERSONA}

        <task>
        ### Task:
        Write CHAPTER {{chapter_number}} of a story based on the following STORY CONCEPT, STORY TITLE, STORY SYNOPSIS, STORY OUTLINE, and WORLD BIBLE.
        - Follow the WRITING GUIDELINES.
        - The target word count for this chapter is: {{chapter_target_word_count}} words.
        - The CHAPTER HEADER is: {{chapter_header}}
        - Describe all the elements in the CHAPTER DESCRIPTION in vivid and engaging detail.
        </task>

        <story_concept>
        ### Story Concept:
        {{concept}}
        </story_concept>

        <story_title>
        ### Story Title:
        {{story_title}}
        </story_title>

        <story_synopsis>
        ### Story Synopsis:
        {{synopsis}}
        </story_synopsis>

        <story_outline>
        ### Story Outline:
        ```yaml
        {{outline}}
        ```
        </story_outline>

        <world_bible>
        ### World Bible:
        ```yaml
        {{world_bible_yaml}}
        ```
        </world_bible>

        <chapter_description>
        ### Chapter Description:
        ```yaml
        {{chapter_description_yaml}}
        ```
        </chapter_description>

        <writing_guidelines>
        ### Writing Guidelines:
        {WRITING_GUIDELINES}
        </writing_guidelines>

        {{chapter_header}}:
        """

    # instance-level variables

    def __init__(
            self,
            concept: Optional[str] = None,
            concept_filename: str = "concept.txt",
            outline_filename: str = "outline.yaml",
            synopsis_filename: str = "synopsis.txt",
            story_filename: str = "story.md",
            world_bible_filename: str = "world_bible.yaml",
            output_foldername: str = "story_files",
            include_outline_in_output_file: bool = False,
            # world_bible: Optional[WorldBible] = None,
            genre: Literal[
                Genre.ROMANCE,
                Genre.CRIME,
                Genre.THRILLER,
                Genre.MYSTERY,
                Genre.FANTASY,
                Genre.SCI_FI,
                Genre.YOUNG_ADULT,
                Genre.HORROR,
                Genre.HISTORICAL_FICTION,
                Genre.CHILDRENS,
                Genre.INSPIRATIONAL,
                Genre.RELIGIOUS,
                Genre.EROTICA,
                Genre.ACTION,
                Genre.ADVENTURE,
                Genre.COMEDY,
                Genre.DRAMA,
                Genre.POETRY,
                Genre.WESTERN,
                Genre.DYSTOPIAN,
                Genre.PARANORMAL,
            ] = Genre.ROMANCE
        ) -> None:
        """
        Initializes the StoryWriter.

        Args:
            concept_filename (str, optional): The path to the file where the
                                              concept will be saved.
                                              Defaults to "concept.txt".
            outline_filename (str, optional): The path to the file where the
                                              outline will be saved.
                                              Defaults to "outline.yaml".
            synopsis_filename (str, optional): The path to the file where the
                                              synopsis will be saved.
                                              Defaults to "synopsis.txt".
            story_filename (str, optional): The path to the file where the
                                            story will be saved incrementally.
                                            Defaults to "story.md".
                                            Defaults to "story.md".
            output_foldername (str, optional): The name of the folder where
                                               the story files will be saved.
                                               Defaults to "story_files".
            world_bible_filename (str, optional):
                                            The path to the file where the
                                            world bible will be saved.
                                            Defaults to "world_bible.yaml".
            genre (Genre, optional): The genre of the story. Must be one of the
                                    predefined genres in the Genre enum.
                                    Defaults to Genre.ROMANCE.
            concept (str, optional): The detailed concept for the story.
                                    Defaults to None.
            include_outline_in_output_file (bool, optional):
                                    Whether to include the outline in
                                    the output file. Defaults to False.
            world_bible (Optional[WorldBible], optional):
                                    A WorldBible object to use as a source of
                                    canonical reference information. Defaults to None.
        """
        super().__init__()
        self.genre = genre
        self.story_filename = story_filename
        self.outline_filename = outline_filename
        self.concept_filename = concept_filename
        self.synopsis_filename = synopsis_filename
        self.output_foldername = output_foldername
        self.world_bible_filename = world_bible_filename
        self.include_outline_in_output_file = include_outline_in_output_file
        self.concept = concept if concept else self.generate_concept_for_genre()
        self.title: Optional[str] = None
        self.outline: Optional[List[Dict]] = None
        self.outline_yaml: Optional[str] = None # YAML string
        self.outline: Optional[List[Dict]] = None
        self.synopsis: Optional[str] = None
        self.world_bible = world_bible
        self.world_bible_yaml: Optional[str] = None # YAML string
        self.output_filepath = os.path.join(self.output_foldername, self.story_filename)

    # methods

    def format_outline_markdown(self, outline: List[Dict]) -> str:
        """
        Formats the outline in Markdown.

        Args:
            outline (List[Dict]): The outline data.

        Returns:
            str: The formatted outline in Markdown.
        """
        outline_in_markdown = "## Outline:\n\n"
        for chapter in outline:
            outline_in_markdown += f"### {chapter['header']}\n\n"
            for i, section in enumerate(chapter["sections"]):
                outline_in_markdown += f"{i + 1}. {section}\n"
            outline_in_markdown += "\n"
        return outline_in_markdown

    def generate_title(self) -> str:
        """
        Generates a title for the story based on the concept.

        Returns:
            str: The generated title.
        """
        # Generate the story title
        title_prompt = StoryWriter.TITLE_PROMPT.format(concept=self.concept)
        title = self.generate_with_retry(title_prompt)
        logger.debug(f"Generated title: {title}")
        return title.strip()

    def generate_outline(self) -> List[Dict]:
        """
        Generates an outline for the story based on
        the genre and at least one of the following:
            1. an image
            2. a concept
            3. one or more books or videos

        Returns:
            A tuple containing the generated beat map and the parsable text.
            The outline is a list of dictionaries, each representing a beat.
            The parsable text is the YAML text that was used to generate the beat map.
        """
        # Convert WorldBible objects to dictionaries for YAML serialization
        world_bible_dict = {}
        if self.world_bible:
            world_bible_dict = {
                "name": self.world_bible.name,
                "description": self.world_bible.description,
                "characters": [char.__dict__ for char in self.world_bible.characters.values()],
                "locations": [loc.__dict__ for loc in self.world_bible.locations.values()],
                "items": [item.__dict__ for item in self.world_bible.items.values()],
                "events": [event.__dict__ for event in self.world_bible.events.values()],
                "cultures": [culture.__dict__ for culture in self.world_bible.cultures.values()],
                "plot_arcs": [arc.__dict__ for arc in self.world_bible.plot_arcs.values()]
            }

        # Convert the dictionary to a YAML string
        world_bible_yaml = yaml.dump(world_bible_dict, indent=4)  # Use yaml.dump for YAML

        # Construct the prompt, including world_bible_yaml conditionally
        prompt = StoryWriter.OUTLINE_PROMPT.format(
            concept=self.concept,
            title=self.title,
            genre_name=self.genre.value['name'],
            genre_definition=str(self.genre.value),
            world_bible_yaml=world_bible_yaml if self.world_bible else "",
        )

        outline_yaml = self.generate_with_retry(prompt)
        parsable_text = self.remove_non_yaml_parsable_strings(outline_yaml)
        try:
            outline = yaml.safe_load(parsable_text)
            return outline, parsable_text
        except yaml.YAMLError as e:
            print(f"Error decoding YAML outline: {e}")
            raise

    def generate_synopsis(self) -> str:
        """
        Generates a synopsis for the story based on the concept.

        Returns:
            str: The generated synopsis.
        """
        synopsis_prompt = StoryWriter.SYNOPSIS_PROMPT.format(
            concept=self.concept,
            title=self.title,
            genre_name=self.genre.value['name'],
            outline=self.outline,
            world_bible_yaml=self.world_bible_yaml if self.world_bible else ""  # Add conditionally
        )
        synopsis = self.generate_with_retry(synopsis_prompt)

        logger.debug(f"Generated synopsis: {synopsis}")
        return synopsis.strip()

    def generate_concept_for_genre(self) -> str:
        """
        Generates a concept for the story based on the selected genre if
        no concept is provided.

        Returns:
            str: A detailed concept for the story.
        """
        genre_concept_prompt = f"""
            You are a creative writing expert.
            Generate a detailed concept for a {self.genre.value['name']} story.
            The concept should be detailed and comprehensive, including
            information about the characters, setting, plot, themes, and any
            other relevant details.

            Here is an example concept for the romance genre:

            <example_concept_for_romance_genre>
                I've always been drawn to heroines who are softer and shyer,
                rather than the bold and feisty types.
                Ideally, I would love for the female main character (FMC) to
                be exceptionally sweet, though she doesn't necessarily have
                to be shy, and to have a domestic inclination.

                Additionally, I'm looking for a story where the male main
                character (MMC) is wealthy and indulges the FMC with affection
                and gifts, showing his love through spoiling and cuddling.
                While I have a soft spot for instant love, I'm also open
                to a slow-burn romance.

                As for the plot or genre, I'm flexible, but I would
                particularly enjoy a narrative where the MMC acts
                as a knight in shining armor, and importantly, where the FMC
                willingly accepts his assistance.

                My preference leans towards books written in the third person,
                although I understand this might be an unusual request, so
                first-person narratives are acceptable if the story is compelling.
                I also appreciate detailed storytelling.

                Please note, I am not interested in any content featuring DDLG
                dynamics or erotica.
            </example_concept_for_romance_genre>

            Concept:
        """

        concept = self.generate_with_retry(genre_concept_prompt).strip()
        print(f"Generated concept for {self.genre.value['name']}: {concept}")
        return concept

    def populate_world_bible_from_concept(self):
        """
        Populates the WorldBible with elements generated from a concept string.

        Example Output:
            ```yaml
            - PlotArc:
                name: Anya's Journey
                description: Anya's quest to master alchemy and uncover the secrets of her master's past.
                characters:
                    - Anya Silverwood
                locations:
                    - Everwood Forest
                items:
                    - The Alchemist's Grimoire

            - Character:
                name: Anya Silverwood
                age: 17
                species_race: Human
                occupation: Apprentice Alchemist
                description: A bright and curious young woman with a passion for alchemy. She is determined to prove herself worthy of her master's legacy.

            - Location:
                name: Everwood Forest
                type: Forest
                description: A sprawling ancient forest filled with magical creatures and hidden secrets.

            - Item:
                name: The Alchemist's Grimoire
                description: A leather-bound book containing ancient alchemical recipes and secrets.
                function: To guide Anya in her alchemical studies.
            ```
        """
        prompt = self.POPULATE_WORLD_BIBLE_FROM_CONCEPT_PROMPT.format(
            concept=self.concept,
            genre_yaml=yaml.dump(self.genre.value),
            )

        output = self.generate_with_retry(prompt)
        parsable_text = self.remove_non_yaml_parsable_strings(output)

        try:
            elements = yaml.safe_load(parsable_text)
            for element in elements:
                element_type = list(element.keys())[0]
                data = element[element_type]

            if element_type == "Character":
                self.world_bible.add_character(Character(**data))
            elif element_type == "Location":
                self.world_bible.add_location(Location(**data))
            elif element_type == "Item":
                self.world_bible.add_item(Item(**data))
            elif element_type == "Event":
                self.world_bible.add_event(Event(**data))
            elif element_type == "Culture":
                self.world_bible.add_culture(Culture(**data))
            elif element_type == "PlotArc":
                self.world_bible.add_plot_arc(PlotArc(**data))
            else:
                print(f"Unknown element type: {element_type}")

        except yaml.YAMLError as e:
            print(f"Error decoding YAML: {e}")
            raise

    def _extract_world_bible_data(self, chapter_description: List[str]) -> Dict:
        """Extracts relevant World Bible data based on the chapter description."""
        characters = []
        locations = []
        for item in chapter_description:
            for character_name in self.world_bible.characters:
                if character_name.lower() in item.lower():
                    characters.append(self.world_bible.characters[character_name])
            for location_name in self.world_bible.locations:
                if location_name.lower() in item.lower():
                    locations.append(self.world_bible.locations[location_name])
        return {"characters": characters, "locations": locations}

    def _add_world_bible_data_to_prompt(self, prompt: str, world_bible_data: Dict) -> str:
        """Adds World Bible data to the prompt string."""
        if self.world_bible:
            prompt += "\n\nWorld Bible Data:\n"
            if world_bible_data["characters"]:
                prompt += "Characters:\n"
                for character in world_bible_data["characters"]:
                    prompt += f"- {character.name}: {character.describe()}\n"
            if world_bible_data["locations"]:
                prompt += "Locations:\n"
                for location in world_bible_data["locations"]:
                    prompt += f"- {location.name}: {location.describe()}\n"
        return prompt

    def generate_chapter(
          self,
          chapter_number: int,
          chapter_data_from_outline: Dict,
          chapter_header: str,
          chapter_target_word_count: Optional[int] = None,
        ) -> str:
        """
            Generates the text for a single chapter by assembling it beat by beat.

            Args:
                chapter_number: The chapter number.
                chapter_data: The chapter data from the outline.
                chapter_header: The header for the chapter.
                chapter_target_word_count: The target word count for the chapter.

            Returns:
                The generated chapter text.

            Raises:
                Exception: If there is an error generating the chapter.

            # # Dynamic Prompting Example
            # relevant_characters = self.get_relevant_characters(current_scene)
            # current_location = self.get_current_location()

            # prompt = generate_character_introduction_prompt(relevant_characters[0])
            # # ... (use other dynamic prompting functions as needed)

            # chapter_text = self.generate_with_retry(prompt)
            # # ... (rest of chapter generation)
            """
        # Initialize the chapter text
        chapter_text = f"## {chapter_header}\n"

        # Generate the description
        chapter_description_yaml = yaml.dump(chapter_data_from_outline)

        # Construct the chapter prompt
        chapter_prompt = StoryWriter.CHAPTER_PROMPT.format(
            concept=self.concept,
            story_title=self.title,
            synopsis=self.synopsis,
            outline=self.outline_yaml,
            chapter_number=chapter_number,
            chapter_header=chapter_header,
            chapter_description_yaml=chapter_description_yaml,
            chapter_target_word_count=chapter_target_word_count,
            world_bible_yaml=self.world_bible_yaml if self.world_bible else ""  # Add conditionally
        )

        # Extract and add World Bible data to the prompt
        if self.world_bible:
            world_bible_data = self._extract_world_bible_data(chapter_data_from_outline['description'])
            chapter_prompt = self._add_world_bible_data_to_prompt(chapter_prompt, world_bible_data)

        # Generate the chapter in one shot
        chapter_text = self.generate_with_retry(chapter_prompt)

        # Add a cliffhanger ending (optional)
        if chapter_number < len(self.outline):  # Only add cliffhanger if not the last chapter
            prompt = StoryWriter.CLIFFHANGER_PROMPT.format(
                concept=self.concept,
                title=self.title,
                outline=yaml.dump(self.outline),
                chapter_text=chapter_text
            )
            cliffhanger_text = self.generate_with_retry(prompt)
            chapter_text += cliffhanger_text

        return chapter_text

    def generate_story(self) -> str:
        """
        Generates the complete story, chapter by chapter, beat by beat,
        description item by description item, using the beat map as the guide.
        Saves the story incrementally to the output file.

        Returns:
            The generated story text.
        """
        # save to story file
        self.save_to_file(self.story_filename, f"# {self.title}\n\n", append=False,) # start new file

        story_text = ""
        for chapter_number, chapter_data_from_outline in enumerate(self.outline, 1):
            chapter_header = f"Chapter {chapter_number} -- {chapter_data_from_outline['title']}\n"
            logger.info(f"Starting to generate: {chapter_header}...")

            # Calculate the target word count for the chapter from the genre definition
            # Access the 'beats' list from the genre's value dictionary
            # Then, access the beat corresponding to the current chapter number (minus 1 for 0-based indexing)
            # Finally, get the 'word_count' from that beat's dictionary
            chapter_target_word_count = self.genre.value["beats"][chapter_number-1].get("word_count")

            chapter_text = self.generate_chapter(
                chapter_number=chapter_number,
                chapter_header=chapter_header,
                chapter_data_from_outline=chapter_data_from_outline,
                chapter_target_word_count=chapter_target_word_count,
                )
            story_text += chapter_text
            self.save_to_file(self.story_filename, chapter_text, append=True,)

            logger.info(f"Finished generating CHAPTER {chapter_number}...")
            logger.info(chapter_text)
            logger.info("Continuing...\n")
        return story_text

    def _handle_world_bible(self):
        """Handles the creation, population, and saving of the WorldBible."""

        logger.info("Looking for WORLD BIBLE...")
        if self.world_bible is None:
            logger.info("WORLD BIBLE not found")
            # Create a new WorldBible if none is provided
            logger.info("Creating new WORLD BIBLE...")
            new_world_bible_name = "My New Story"
            self.world_bible = WorldBible(name=new_world_bible_name)
            logger.info(f"Created new WORLD BIBLE: {new_world_bible_name}")
            if self.concept:
                self.populate_world_bible_from_concept(self.concept)
        else:
            logger.info("WORLD BIBLE found!")

        # Convert WorldBible objects to dictionaries for YAML serialization
        world_bible_dict = {}
        if self.world_bible:
            world_bible_dict = {
                "name": self.world_bible.name,
                "characters": [char.__dict__ for char in self.world_bible.characters.values()],
                "locations": [loc.__dict__ for loc in self.world_bible.locations.values()],
                "items": [item.__dict__ for item in self.world_bible.items.values()],
                "events": [event.__dict__ for event in self.world_bible.events.values()],
                "cultures": [culture.__dict__ for culture in self.world_bible.cultures.values()],
                "plot_arcs": [arc.__dict__ for arc in self.world_bible.plot_arcs.values()]
            }

        # Convert the dictionary to a YAML string
        self.world_bible_yaml = yaml.dump(world_bible_dict, indent=4)  # Use yaml.dump for YAML

        # Create the output folder if it doesn't exist
        os.makedirs(self.output_foldername, exist_ok=True)
        logger.info(f"Output folder created: {self.output_foldername}")

        # Save WorldBible as YAML
        if self.world_bible:
            with open(os.path.join(self.output_foldername, self.world_bible_filename), "w") as f:
                yaml.dump(world_bible_dict, f, indent=4)  # Use yaml.dump for YAML
            logger.info(f"WORLD BIBLE saved to {self.world_bible_filename}\n")
            logger.info(f"WORLD BIBLE:\n{self.world_bible_yaml}\n")

        return

    def run(self) -> str:
        """
        Runs the story generation process.

        Returns:
            The generated story text.
        """
        # Log initial information
        logger.info("Initialized new StoryWriter instance")
        logger.info(f"Genre: {self.genre.value['name']}")
        logger.info(f"Story filename: {self.story_filename}")
        logger.info(f"Outline filename: {self.outline_filename}")
        logger.info(f"Synopsis filename: {self.synopsis_filename}")
        logger.info(f"World Bible filename: {self.world_bible_filename}")
        logger.info(f"Include outline in output file: {self.include_outline_in_output_file}")
        logger.info(f"Concept: {self.concept}")
        logger.info("Continuing...\n")

        # Handle WORLD BIBLE
        logger.info("Handling WORLD BIBLE...")
        self._handle_world_bible() # Call the new function
        logger.info("Continuing...\n")

        # Generate TITLE
        # @TODO: consider incorporating world bible
        #        as context for title creation procedure
        logger.info(f"Generating TITLE...")
        self.title = self.generate_title()
        logger.info(f"Generated TITLE")
        self.save_to_file(
            self.concept_filename,
            f"{self.title}\nConcept\n\n{self.concept}",
            append=False, # start a new file
            )
        logger.info(f"TITLE: {self.title}\n")
        logger.info("Continuing...\n")

        # Generate OUTLINE
        logger.info("Generating OUTLINE...")
        self.outline, self.outline_yaml = self.generate_outline()
        logger.info(f"Generated OUTLINE")
        self.save_to_file(
            self.outline_filename,
            f"{self.title}\nOutline\n{self.outline_yaml}",
            append=False, # start a new file
            )
        logger.info(f"OUTLINE:\n{self.outline_yaml}\n")
        logger.info("Continuing...\n")

        # Generate SYNOPSIS
        logger.info("Generating SYNOPSIS...")
        self.synopsis = self.generate_synopsis()
        logger.info(f"Generated SYNOPSIS")
        self.save_to_file(
            self.synopsis_filename,
            f"{self.title}\n\nSynopsis\n\n{self.synopsis}",
            append=False, # start a new file
            )
        logger.info(f"SYNOPSIS:\n{self.synopsis}\n")
        logger.info("Continuing...\n")

        # GENERATE the complete story
        story = self.generate_story()
        logger.info(f"Generated Story: {self.title}")
        # # file has already been sufficiently saved
        # # incrementally during writing process
        # # no need to save the file again here
        # self.save_to_file(self.story_filename, story)

        logger.info(f"Story ('{self.title}') SAVED REMOTELY to {self.output_filepath}")
        logger.info("Continuing...\n")

        # SAVE generated files locally
        # Zip the output folder
        zip_folder(self.output_folderpath, self.output_filename,)
        # Download the zip file
        local_filename = f"{self.title}.zip"
        files.download(local_filename)
        logger.info(f"Output files SAVED LOCALLY to {local_filename}")

        # implement feedback loop
        # look for opportunities to add richness to the world
        # look for opportunities to add richness to the story
        # revise chapters, now in context. build in layers
        # use WritingQualityEvaluator and Feedback classes
        # make extensive and intelligent use of PromptLibrary

        logger.info("FINISHED!")
        return story

    def test(self) -> None:
        """
        Function for testing the StoryWriter class.
        """
        CONCEPTS = [
            # 0
            "cat",
            # 1
            """
                I've always been drawn to heroines who are softer and shyer, rather than the bold and feisty types.
                Ideally, I would love for the female main character (FMC) to be exceptionally sweet,
                though she doesn't necessarily have to be shy, and to have a domestic inclination.

                Additionally, I'm looking for a story where the male main character (MMC) is wealthy and
                indulges the FMC with affection and gifts, showing his love through spoiling and cuddling.
                While I have a soft spot for instant love, I'm also open to a slow-burn romance.

                As for the plot or genre, I'm flexible, but I would particularly enjoy a narrative where the MMC acts
                as a knight in shining armor, and importantly, where the FMC willingly accepts his assistance.

                My preference leans towards books written in the third person, although I understand this might
                be an unusual request, so first-person narratives are acceptable if the story is compelling.
                I also appreciate detailed storytelling.

                Please note, I am not interested in any content featuring DDLG dynamics or erotica.
                """,
            # 2
            """
                # “I’ve waited years for you” trope

                Y’ALL…

                I need more fantasy romances with this trope.

                An example is Howl’s Moving Castle, Sophie in the future says “I know how to help you, find me in the future”.
                So he literally waits for her all this time and in the beginning of the movie he says “I was looking everywhere for you” 🥺🥺🥺

                Or in an M/M series I adore (pls read it), Heaven Official’s Blessing, this trope is taken to an extreme.
                Won’t say more to spoil but…yeah the man waited 800 years for his love 😭

                Both of these are fantasy and I definitely would like to stick to fantasy romance for this trope.
                Contemporary never hits that well for me. M/F and M/M are both good.
                """,
            # 3
            """
                # Are there any bully romances where the bullying gets so bad that she just leaves the city and doesn't tell anyone where she went to? Not even her parents? And the hero gets desperate trying to find her.

                I know this is be specific so I hope the mods don't delete it.

                I'm looking for a book (Bonus points if it's set on high school) and the bullying gets so damn bad that she just leaves... She doesn't tell anyone where she went to. She just disappears and then the Hero goes crazy trying to find out what happened to her.

                Bonus points if there's one big public humiliation scene that devastates her and she leaves after that.
                """,
            # 4
            """
                # Timid/emotionally abused FMC stands up to overbearing/narcissistic parents - comes into her own

                I'm rereading the Kilts & Quilts series by Patience Griffin this week, and I'm remembering how much
                I love Emma & Gabriel's story in Meet Me in Scotland.

                I want to watch a FMC (or MMC) grow a backbone! Preferably standing up to their parents,
                but not absolutely essential - it could be a teacher/boss/mentor, just not the love interest.
                I'm hoping for the conflict/abuse to be long term, not just a Forbidden love/disapproving family/"I choose him/her".

                I'm not particular on the style of story - contemporary/historical/paranormal/fantasy/etc.
                Any spice level, but I prefer not YA. I don't love mafia romance and reverse harems are a hard pass for me though.

                I found a similar post from about a year ago with the button, but it was only for books with toxic relationships,
                and the key element I'm interested in is the MC standing up to it, not accepting it and building boundaries,
                going no contact, or something else - but calling them out for the awful behavior and growing/healing as a result.

                Thanks in advance!
                """,
            # 5: Flock by Kate Stewart (from Beth)
            """
                Can you keep a secret?

                I grew up sick.⁣
                ⁣
                Let me clarify.⁣
                ⁣
                I grew up believing that real love stories include a martyr or demand great sacrifice to be worthy.⁣
                ⁣
                Because of that, I believed it, because I made myself believe it, and I bred the most masochistic of romantic hearts, which resulted in my illness.⁣
                ⁣
                When I lived this story, my own twisted fairy tale, it was unbeknownst to me at the time because I was young and naïve. I gave into temptation and fed the beating beast, which grew thirstier with every slash, every strike, every blow.⁣
                ⁣
                Triple Falls wasn’t at all what it seemed, nor were the men that swept me under their wing. But in order to keep them, I had to be in on their secrets.⁣
                ⁣
                Secrets that cost us everything to keep.⁣
                ⁣
                That’s the novelty of fiction versus reality. You can’t re-live your own love story, because by the time you’ve realized you’re living it, it’s over. At least that was the case for me and the men I trusted my foolish heart to.⁣
                ⁣
                Looking back, I’m convinced I willed my story into existence due to my illness.⁣
                ⁣
                And all were punished.
                """,
            # 6: Kiss and Don’t Tell by Meghan Quinn (from Beth)
            """
                Five hockey players, a cabin in the woods...and little old me, Winnie Berlin.

                Sounds like the start of a fantasy, right? But it's not. It's my reality.

                It all started when I got lost in the middle of the Canadian Rockies. A wicked thunderstorm sent my car mud sliding into a ditch, and for a girl whose survival skills involve finding the nearest Starbucks, I was in deep trouble.

                Forced to abandon my car in search of help, I stumbled upon a cabin in the middle of nowhere. Soaking wet, irritated, and in desperate need of a hot shower, I knocked on the door for help only to be greeted by five boisterous, burly, and insanely attractive men.

                Not just men.

                Professional hockey players.

                I hit the motherload of fantasies, right? Wrong.

                Because as it turns out, one of them is my ex-boyfriend's half-brother. And when I end up crashing in their spare bedroom for the night, things take a very unexpected turn.

                I'm not sure any amount of survival skills can protect my heart from Vancouver Agitators star goalie, Pacey Lawes. But as long as we kiss and don’t tell, maybe everything will be okay....
                """,
            # 7: Dante by Sadie Kincaid (from Beth)
            """
                I hate Dante Moretti. He is cold, calculating and vicious. It’s no secret that he wiped out his former fiancée and her entire family on the eve of their wedding. He’s the kind of evil that makes even devils cry.

                But that’s not why I hate him.

                I hate him because he has taken the only thing I have left in this miserable world – my freedom. He has chained me to him because of a debt that I don’t owe. I just have the misfortune to share the bloodline of the man who does.

                And now I’m trapped in his gilded cage and the only thing I have left to do is run. And I try. I really do. But he is everywhere.

                Suffocating. All consuming. Intoxicating.

                He is a paradox. A man with a heart as cold as ice, whose passion rages like an inferno.
                """,
            # 8: Rock Bottom Girl by Lucy Score (from Beth)
            # 9: The Coworker by Freida McFadden (from Beth)
            # 10: Blue Hawaiian by Carla Luna (from Beth)
            # 11: The Summer of Broken Rules by K.L. Walther (from Beth)
            # 12: The Housemaid by Freida McFadden (from Beth)
            # 13: Yours Truly by Abby Jimenez (from Beth)
            # 14: Verity by Colleen Hoover (from Beth)
            # 15: Thousand Boy Kisses by Tillie Cole (from Beth)
            ]
        PICK = 7
        writer = self(genre=Genre.ROMANCE, concept=CONCEPTS[PICK],)
        writer.run()

class StoryElementProvider:
    GIST_ID = "2fc92b5c01cf23a0ed573ff26b864ae7"

    # Example name components
    location_prefixes = ["Aldo", "Bran", "Crim", "Dun", "Eld", "Frost", "Glen", "Haven", "Ivory", "Jade"]
    location_suffixes = ["wood", "field", "ford", "shire", "port", "moor", "brook", "peak", "vale", "ridge"]
    item_prefixes = ["Ancient", "Mystic", "Enchanted", "Cursed", "Shimmering", "Rusty", "Glowing", "Shadowy"]
    item_suffixes = ["Sword", "Amulet", "Staff", "Bow", "Shield", "Potion", "Ring", "Cloak"]
    event_prefixes = ["Great", "Dark", "Bright", "Eternal", "Forgotten", "Lost", "Hidden", "Mystical"]
    event_suffixes = ["War", "Festival", "Ritual", "Quest", "Battle", "Tournament", "Feast", "Celebration"]
    # "Golden City" variations
    golden_synonyms = ["Aureate", "Gilded", "Sunlit", "Radiant", "Luminous", "Shining", "Bright", "Golden"]
    city_synonyms = ["Metropolis", "Urban", "Town", "Borough", "Village", "Hamlet", "Settlement", "Capital", "Citadel", "Fortress"]
    # Example of enhanced variation generation
    additional_adjectives = ["Ancient", "Majestic", "Grand", "Royal", "Eternal", "Mystic"]
    additional_nouns = ["Realm", "Kingdom", "Empire", "Domain", "Territory"]

    def __init__(self, element_type: str = None, genres: List[str] = ["generic",], num_elements: int = 1,):
        """
        Initializes the StoryElementProvider and loads data from a CSV file in a Gist.

        Args:
            gist_id (str): The ID of the Gist containing the story elements CSV.
            element_type (str, optional): Filter elements by this type.
            genres (List[str], optional): Filter elements by these genres.

        Usage:
            element_provider = StoryElementProvider()
            story_element = element_provider.random_element

        Prompting:
            To get more, consider the following prompt sequence.

            User:

            > summarize the premise of the harry potter series using the following template.

            ```
            premise = f"In a world where {world_essence}, {character_spark} must {conflict_catalyst} to {theme_resonance}."
            ```

            User:

            > now, give me lists of such world_essences, character_sparks, conflict_catalysts and theme_resonances from other books and other series. i want to be able to mix and match those together to create unique combinations of those elements to create unique premises to serve as the basis for new stories. make each list as long and the elements as interchangable (without being too generic) as possible.

            Model:

            > To create a unique premise, you can randomly select one element from each category and plug them into the template:
            Copypremise = f"In a world where {world_essence}, {character_spark} must {conflict_catalyst} to {theme_resonance}."
            For example, a random combination might be:

            > "In a world where Earth's gravity suddenly becomes unreliable, a con artist forced to become a hero must prevent the collapse of reality itself to confront the fear of the unknown."
            This method allows for 50,625 unique combinations (15 x 15 x 15 x 15), providing a wealth of potential story premises. Feel free to mix and match these elements as you see fit, or use them as inspiration to create your own unique story elements.

            User:

            > now, repeat and do the same, except this time, focus on the romance genre

            User:

            > again, except this time, focus on the anime genre
        """

        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Initializing StoryElementProvider...")
        # logger.setLevel(logging.DEBUG)
        # logger.setLevel(logging.INFO)

        self.gist_id = StoryElementProvider.GIST_ID
        logger.info(f"{caller_name}:   - Gist ID: {self.gist_id}")

        self.element_type = element_type
        logger.info(f"{caller_name}:   - Element type: {self.element_type}")

        self.genres = genres
        logger.info(f"{caller_name}:   - Genres: {self.genres}")

        self.num_elements = num_elements
        logger.info(f"{caller_name}:   - Num elements: {self.num_elements}")

        self.github = GithubManager()
        self.data = self.load_data()

        logger.info(f"{caller_name}: Loaded {len(self.data)} elements from Gist.")
        logger.debug(f"{caller_name}: Data: {self.data}")

        self.random_elements = self.get_random_elements()
        logger.info(f"{caller_name}: Random element: {self.random_elements}")

    def load_data(self) -> list[dict[str, str]]:
        """Loads story elements from the specified Gist."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Fetching data from Github...")

        data = self.github.load_data_from_gist(gist_id=self.gist_id, file_type="csv")
        logger.info(f"{caller_name}: Github data fetched...")
        logger.debug(f"{caller_name}: Data: {data}")
        return data

    def get_random_elements(self, element_types: List[str] = None, genres: List[str] = None, num_elements: int = None,) -> dict:
        """
        Retrieves a random story element, optionally filtering by element types and genres.

        Args:
            element_types (List[str], optional): A list of element types to filter by. If None, all element types are considered.
            genres (List[str], optional): A list of genres to filter by. If None, all genres are considered.
            num_elements (int, optional): The number of elements to return. Effectively defaults to 1,
                                          because defaullt instance-level value overrides no argument

        Returns:
            dict: A randomly selected story element as a dictionary.
                  Returns an empty dictionary if no matching elements are found.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Retrieving random element...")

        filtered_elements = self.data

        # Filter by element types
        if element_types:
            logger.info(f"{caller_name}: Filtering elements by element types: {element_types}...")
            filtered_elements = [
                element for element in filtered_elements
                if element['element_type'] in element_types
            ]

        # Filter by genres
        if genres:
            logger.info(f"{caller_name}: Filtering elements by genres: {genres}...")
            filtered_elements = [
                element for element in filtered_elements
                if any(genre.lower() in element['genres'].lower().split(',') for genre in genres)
            ]

        effective_target_num_elements = num_elements if num_elements is not None else self.num_elements
        target_num_elements = min(effective_target_num_elements, len(filtered_elements))

        if filtered_elements:
            logger.info(f"{caller_name}: {len(filtered_elements)} matching elements found. Selecting a random element...")
            chosen_element = random.choice(filtered_elements) if target_num_elements == 1 else random.sample(filtered_elements, k=target_num_elements,) # returns one element or list
            logger.info(f"{caller_name}: Element selected.")
            return chosen_element
        else:
            logger.warning(f"{caller_name}: No matching elements found for element types: '{element_types}' and genres: '{genres}'. Returning a random element.")
            return None  # Return a random element if no match is found

    # <experimental_cluster>

    def generate_location_name():
        location_prefix = random.choice(location_prefixes)
        location_suffix = random.choice(location_suffixes)
        return f"{location_prefix}{location_suffix}"

    def generate_item_name():
        prefix = random.choice(item_prefixes)
        suffix = random.choice(item_suffixes)
        return f"{prefix} {suffix}"

    def generate_event_name():
        prefix = random.choice(event_prefixes)
        suffix = random.choice(event_suffixes)
        return f"{prefix} {suffix}"

    def generate_complex_location_name():
        """Example of more complex name generation"""
        prefix = random.choice(prefixes)
        middle = random.choice(["River", "Lake", "Mountain", "Forest"])
        suffix = random.choice(suffixes)
        return f"{prefix}{middle}{suffix}"

    def usage_example():
        """Generate some random names"""
        location_name = generate_location_name()
        item_name = generate_item_name()
        event_name = generate_event_name()

        print(f"Location: {location_name}")
        print(f"Item: {item_name}")
        print(f"Event: {event_name}")

        complex_location_name = generate_complex_location_name()
        print(f"Complex Location: {complex_location_name}")

        # -- Example Output --
        # Location: Branbrook
        # Item: Enchanted Sword
        # Event: Great Festival
        # Complex Location: AldoLakefield

    def generate_golden_city_variation():
        golden_word = random.choice(golden_synonyms)
        city_word = random.choice(city_synonyms)
        return f"{golden_word} {city_word}"

    def generate_variations():
        """Generate and print some variations"""
        for _ in range(10):
            print(generate_golden_city_variation())

        # # Example Output
        # Aureate Metropolis
        # Gilded Urban
        # Sunlit Town
        # Radiant Borough
        # Luminous Village
        # Shining Hamlet
        # Bright Settlement
        # Golden Capital
        # Aureate Citadel
        # Gilded Fortress

    def generate_enhanced_golden_city_variation():
        noun        = random.choice(additional_nouns)
        city_word   = random.choice(city_synonyms)
        adjective   = random.choice(additional_adjectives)
        golden_word = random.choice(golden_synonyms)

        # Randomly decide whether to use an additional adjective and noun
        if random.random() < 0.5:
            return f"{adjective} {golden_word} {city_word}"
        else:
            return f"{golden_word} {city_word} of {noun}"

        # Generate and print some enhanced variations
        for _ in range(10):
            print(generate_enhanced_golden_city_variation())

        # # Example Enhanced Output
        # Majestic Gilded Metropolis
        # Radiant Town of Realm
        # Eternal Luminous Urban
        # Mystic Shining Borough
        # Royal Aureate Capital
        # Gilded Urban of Empire
        # Ancient Bright Fortress
        # Golden Citadel of Domain
        # Majestic Sunlit Settlement
        # Eternal Radiant Hamlet

    # </experimental_cluster>

class PremiseGenerator:

    GOALS_CONFLICTS = {
        "fantasy":{
            "goals":[
                "Defeat an ancient evil", "Restore balance to a magical world", "Find a legendary artifact", "Master magical abilities", "Unite warring factions",
                "Discover hidden realms", "Protect the innocent", "Forge a new destiny", "Discover hidden magical powers", "Unite warring kingdoms", "Reclaim a lost city",
                "Escape a mystical prison", "Find a legendary creature", "Break an ancient curse", "Fulfill a prophecy", "Prevent a magical catastrophe", "Ascend to godhood",
                "Restore a fallen empire", "Tame a mythical beast", "Uncover forgotten lore", "Close a portal to another dimension", "Overthrow a tyrant",
                "Retrieve a stolen magical item", "Awaken an ancient guardian", "Solve an arcane mystery", "Survive in a hostile magical environment", "Establish a new magical order",
                "Protect the balance between realms", "Resurrect an extinct magical species", "Harness the power of the elements", "Unveil a magical secret",
            ],
            "conflicts":[
                "Good vs. Evil", "Technology vs. Magic", "Destiny vs. Free Will", "Tradition vs. Progress", "Order vs. Chaos", "Light vs. Darkness", "Nature vs. Civilization",
                "Mortality vs. Immortality", "Science vs. Superstition", "Individual vs. Society", "Human vs. Non-human", "Physical vs. Spiritual", "Material vs. Ethereal",
                "Reason vs. Emotion", "Past vs. Future", "Reality vs. Illusion", "Creation vs. Destruction", "Unity vs. Division", "Faith vs. Skepticism", "Duty vs. Desire",
                "Knowledge vs. Ignorance", "Balance vs. Extremism", "Mundane vs. Supernatural", "Loyalty vs. Betrayal", "Justice vs. Mercy", "Truth vs. Deception",
                "Freedom vs. Security", "Sacrifice vs. Preservation", "Harmony vs. Discord", "Redemption vs. Condemnation", "Power vs. Weakness",
            ],
            "character_archetypes":[
                "Chosen One", "Reluctant Hero", "Wise Mentor", "Dark Lord", "Trickster", "Guardian", "Shapeshifter", "Outcast", "Warrior", "Scholar", "Prophet", "Beast Master",
                "Enchantress", "Fallen Hero", "Dragon Rider", "Elemental Mage", "Cursed One", "Ethereal Being", "Magical Artificer", "Spirit Walker",
            ],
            "settings":[
                "Enchanted Forest", "Ancient Ruins", "Floating Islands", "Underground Kingdom", "Magical Academy", "Celestial Realm", "Cursed Wasteland", "Elemental Planes",
                "Mystical Nexus", "Timeless Sanctuary", "Shapeshifting Castle", "Ethereal Void", "Living Dungeon", "Pocket Dimension", "Primordial Wilderness",
            ],
            "magical_elements":[
                "Elemental Magic", "Necromancy", "Illusion", "Divination", "Alchemy", "Summoning", "Time Magic", "Runic Magic", "Blood Magic", "Nature Magic", "Chaos Magic",
                "Soul Manipulation", "Dimensional Magic", "Mind Control", "Transmutation",
            ]
        },
        "sci_fi":{
            "goals":[
                "Explore unknown planets", "Establish first contact with aliens", "Survive in a post-apocalyptic world", "Prevent a dystopian future", "Discover new forms of life",
                "Uncover the secrets of the universe", "Find a new home for humanity", "Achieve interstellar travel", "Discover a new form of energy", "Create a new society",
                "Escape a space-time anomaly", "Solve a cosmic mystery",
            ],
            "conflicts":[
                "Humanity vs. Technology", "Individual vs. Society", "Progress vs. Ethics", "Space/Time limitations vs. Human ambition", "The dangers of artificial intelligence",
                "The ethical implications of a new technology", "Competition for limited resources", "Humanity vs. Nature", "Science vs. Faith",
                "Individual freedom vs. Collective security", "Rebellion vs. Authority",
            ]
        },
        "mystery_thriller":{
            "goals":[
                "Solve a complex crime", "Uncover a conspiracy", "Protect a key witness", "Clear one's name", "Unmask the true culprit", "Expose a corrupt system",
                "Prevent a tragedy", "Seek justice for the wronged", "Escape a deadly trap", "Unmask a mastermind", "Reveal a hidden truth", "Solve a murder mystery",
            ],
            "conflicts":[
                "Justice vs. Corruption", "Truth vs. Deception", "Personal ethics vs. Professional duty", "Past vs. Present", "The weight of secrets", "The struggle for power",
                "The clash of ideologies", "Memory vs. Reality", "Perception vs. Truth", "Loyalty vs. Betrayal", "Morality vs. Expedience",
            ]
        },
        "romance":{
            "goals":[
                "Find true love", "Overcome personal insecurities", "Reconcile with a lost love", "Balance career and relationship",
                "Build a strong foundation for a future together", "Navigate the challenges of a long-term relationship", "Find happiness and fulfillment",
                "Create a life together", "Make a grand gesture", "Escape a toxic relationship", "Find a sense of belonging", "Learn to forgive",
                "Rediscover passion in a long-term relationship", "Heal from past heartbreak", "Learn to trust again", "Discover self-love and independence",
                "Navigate cultural differences in a relationship", "Build a blended family", "Overcome long-distance challenges", "Find love later in life",
                "Reconcile different life goals with a partner", "Learn to communicate effectively", "Overcome societal prejudices against the relationship",
                "Navigate the complexities of modern dating", "Find balance between personal space and togetherness", "Rekindle a lost spark",
                "Overcome financial struggles together", "Support partner through a personal crisis", "Learn to compromise without losing oneself",
                "Build a relationship based on friendship and respect",
            ],
            "conflicts":[
                "Love vs. Duty", "Heart vs. Mind", "Individual growth vs. Relationship demands", "Past wounds vs. New possibilities", "The fear of commitment",
                "The clash of personalities", "The pressure of societal expectations", "Social pressure vs. Personal freedom", "Fear of intimacy vs. Desire for connection",
                "Loss vs. Grief", "Jealousy vs. Trust", "Career ambitions vs. Relationship stability", "Family approval vs. Personal happiness",
                "Cultural differences vs. Shared values", "Independence vs. Interdependence", "Passion vs. Compatibility", "Idealism vs. Reality in relationships",
                "Self-sacrifice vs. Self-preservation", "Communication barriers vs. Understanding", "Past relationships vs. Current commitment",
                "Long-distance vs. Physical presence", "Attraction to others vs. Loyalty", "Personal baggage vs. Fresh start", "Different love languages",
                "Financial stability vs. Emotional fulfillment", "Traditional values vs. Modern relationship dynamics", "Social media influence vs. Private relationship",
                "Work-life balance vs. Quality time", "Sexual compatibility vs. Emotional connection", "Individual identity vs. Couple identity",
            ],
            "character_archetypes":[
                "The Hopeless Romantic", "The Cynic", "The Workaholic", "The Free Spirit", "The Wounded Soul", "The Player", "The Best Friend", "The Wallflower",
                "The Charmer", "The Commitment-Phobe", "The Caregiver", "The Bad Boy/Girl", "The Perfectionist", "The Adventurer", "The Peacemaker", "The Rebel",
                "The Nurturer", "The Intellectual", "The Artist", "The Protector",
            ],
            "settings":[
                "Small Town", "Big City", "Workplace", "College Campus", "Tropical Resort", "Childhood Home", "Destination Wedding", "Coffee Shop", "Online Dating App",
                "High School Reunion", "Cruise Ship", "Historic Castle", "Bustling Metropolis", "Quaint Bed and Breakfast", "Picturesque Vineyard", "Charming Bookstore",
                "Luxury Penthouse", "Remote Island", "Cozy Cabin in the Woods", "International Study Abroad Program",
            ],
            "relationship_stages":[
                "First Meeting", "First Date", "Honeymoon Phase", "Moving In Together", "Meeting the Family", "Long-Distance Period", "Engagement",
                "Wedding Planning", "Newlywed Life", "Parenting", "Empty Nest", "Retirement", "Health Crisis", "Rekindling Romance", "Golden Years",
            ],
            "romantic_gestures":[
                "Surprise Weekend Getaway",
                "Handwritten Love Letter",
                "Recreating First Date",
                "Learning Partner's Hobby",
                "Public Declaration of Love",
                "Thoughtful Homemade Gift",
                "Surprise Party",
                "Romantic Scavenger Hunt",
                "Stargazing Picnic",
                "Couples Cooking Class",
                "Surprise Serenade",
                "Creating a Memory Book",
                "Spontaneous Dance in the Rain",
                "Breakfast in Bed",
                "Planning a Dream Vacation"
            ]
        },
        "horror":{
            "goals":[
                "Survive a terrifying ordeal",
                "Banish an evil entity",
                "Solve a supernatural mystery",
                "Break a curse",
                "Confront the darkness within",
                "Find peace after trauma",
                "Uncover the truth behind the horror",
                "Escape the nightmare",
                "Escape a haunted location",
                "Find a safe haven",
                "Uncover a dark secret",
                "Confront a personal demon"
            ],
            "conflicts":[
                "Sanity vs. Madness",
                "Reality vs. Supernatural",
                "Knowledge vs. Blissful ignorance",
                "Individual vs. Unknown forces",
                "The struggle for survival",
                "The fear of the unknown",
                "The battle against inner demons",
                "Fear vs. Courage",
                "Reason vs. Instinct",
                "Humanity vs. Monstrosity",
                "Life vs. Death"
            ]
        },
        "historical_fiction":{
            "goals":[
                "Change the course of history",
                "Survive a significant historical event",
                "Preserve important cultural artifacts",
                "Uncover hidden historical truths",
                "Gain a deeper understanding of the past",
                "Learn from the mistakes of history",
                "Make a difference in the world",
                "Leave a legacy",
                "Escape a historical disaster",
                "Join a historical revolution",
                "Find a historical treasure",
                "Witness a pivotal moment"
            ],
            "conflicts":[
                "Progress vs. Tradition",
                "Individual vs. Historical forces",
                "Recorded history vs. True events",
                "Cultural preservation vs. Assimilation",
                "The struggle for power and influence",
                "The clash of ideologies",
                "Loyalty vs. Duty",
                "Nation vs. Nation",
                "Individual freedom vs. Collective security",
                "Morality vs. Expedience"
            ]
        },
        "adventure":{
            "goals":[
                "Discover lost treasures",
                "Complete a perilous journey",
                "Save a community or civilization",
                "Overcome personal limitations",
                "Find a new home",
                "Uncover the secrets of the past",
                "Make a difference in the world",
                "Achieve personal growth",
                "Escape a treacherous environment",
                "Find a hidden passage",
                "Uncover a hidden treasure",
                "Reach a distant destination"
            ],
            "conflicts":[
                "Man vs. Nature",
                "Greed vs. Honor",
                "Teamwork vs. Individual glory",
                "Comfort zone vs. Unknown territories",
                "The struggle for survival",
                "The clash of cultures",
                "The fight against injustice",
                "Survival vs. Exploration",
                "Friendship vs. Betrayal",
                "Loyalty vs. Duty",
                "Risk vs. Reward"
            ]
        }
    }

    def __init__(self, num_premises: int = 16, element_type: str = None, genres: List[str] = ["generic",], primary_genre: str = None,
                 template: str = "In a world where {world_essence}, {character_spark} must {conflict_catalyst} to {theme_resonance}."):
        """
            Initializes the PremiseGenerator with an element provider and an optional premise template string.

            get complete lists here:
            gist_id = "2fc92b5c01cf23a0ed573ff26b864ae7"

            Args:
                num_premises (int, optional): The number of premises to generate.
                                              Defaults to 15.
                element_type (str, optional): The type of element to filter by.
                                              One of: 'world_essence', 'character_spark',
                                              'conflict_catalyst', 'theme_resonance'.
                genres (List[str], optional): The genres to filter elements by as a list of strings.
                template (str, optional): The template string for the premise.

            # Usage
            generator = StoryPremiseGenerator()
            premises = generator.generate_multiple_premises(16)
            """
        self.genres = genres
        self.template = template
        self.element_type = element_type
        self.num_premises = num_premises
        self.primary_genre = primary_genre

        self.llm = GeminiLLM()

    def testing_get_premise1(self, genres: List[str] = None,) -> Optional[str]:
        """
            Generates a list of premise candidates, evaluates them using an LLM,
            and selects the premise with the highest overall quality score.
            """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Beginning to select final premise from a list of candidates...")

        premises = self.generate_premises(genres=genres if genres else self.genres,)

        if not premises:
            logger.error(f"{caller_name}: Error: Failed to generate premises after multiple attempts. Returning None.")
            return None

        # Construct the evaluation prompt
        eval_prompt = f"""You are an experienced story editor with a keen eye for identifying promising story ideas. You are tasked with evaluating a set of story premises based on specific criteria.

        Your goal is to provide a score for each premise on a scale of 1 to 5 for each of the following criteria, with 5 being the highest:

        - Clarity of Goal: How well-defined and understandable is the protagonist's goal implied in the premise?
        - Conflict Potential: Does the premise suggest a strong, compelling conflict with clear stakes?
        - Originality: Is the premise unique, fresh, and intriguing? Does it avoid common tropes or clichés?
        - Genre Alignment: How well does the premise fit the specified genre(s)?
        - Overall Quality: A subjective assessment of the premise's overall potential for a good story, considering all factors.

        For each premise, provide your scores in the following JSON format:

        ```json
        {{
          "premise": "<The full text of the premise>",
          "scores": {{
            "Clarity of Goal": <Score>,
            "Conflict Potential": <Score>,
            "Originality": <Score>,
            "Genre Alignment": <Score>,
            "Overall Quality": <Score>
          }}
        }}
        ```

        Here are the premises to evaluate:
        {chr(10).join(f'- {premise}' for premise in premises)}

        Please provide your evaluation for each premise in separate JSON blocks, one block per premise.
        """
        # Get LLM evaluation
        evaluations_json = self.llm.generate_inference(eval_prompt)
        logger.debug(f"{caller_name}: LLM evaluations: {evaluations_json}")

        # Parse evaluations and select best premise
        best_premise = None
        best_score = -1

        for eval_block in evaluations_json.split('\n'):
            try:
                eval_data = json.loads(eval_block)
                premise = eval_data['premise']
                overall_quality = eval_data['scores']['Overall Quality']

                if overall_quality > best_score:
                    best_premise = premise
                    best_score = overall_quality

            except json.JSONDecodeError:
                logger.warning(f"{caller_name}: Warning: Skipping invalid JSON block: {eval_block}")

        if best_premise:
            logger.info(f"{caller_name}: Selected premise: {best_premise} (Overall Quality: {best_score})")
            return best_premise
        else:
            logger.error(f"{caller_name}: Error: Failed to select a premise from LLM evaluations. Returning None.")
            return None

    def testing_get_premise(self, world_bible: WorldBible, author_style: str, genres: List[str] = None,) -> Optional[str]:
        """
        Generates a list of premise candidates, evaluates them, selects the best,
        and updates the WorldBible with the selected premise, goal, and conflicts.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Beginning to select final premise from a list of candidates...")

        premises = self.generate_premises(genres=genres if genres else self.genres,)

        # # @TODO:
        # # - implement logic to select premise based on quality assessment vs. some rubric
        # # - write premise to unused premises file saved as a separate Gist to be created
        # if premises:
        #     premise = random.choice(premises)
        #     logger.info(f"{caller_name}: Generated premise: {premise}")
        #     return premise
        # else:
        #     logger.error(f"Error: Failed to generate premises after multiple attempts. Returning None.")
        #     return None

        # @TODO implemented below:
        # --- merit-based premise selection ---

        if not premises:
            logger.error(f"{caller_name}: Error: Failed to generate premises after multiple attempts. Returning None.")
            return None

        # Construct the evaluation prompt
        eval_prompt = f"""
        You are an experienced story editor with a keen eye for identifying promising story ideas.
        You are tasked with evaluating a set of story premises based on specific criteria.

        Your goal is to provide a score for each premise on a scale of 1 to 5 for each of the following criteria, with 5 being the highest:

        - Clarity of Goal: How well-defined and understandable is the protagonist's goal implied in the premise?
        - Conflict Potential: Does the premise suggest a strong, compelling conflict with clear stakes?
        - Originality: Is the premise unique, fresh, and intriguing? Does it avoid common tropes or clichés?
        - Genre Alignment: How well does the premise fit the specified genre(s)?
        - Overall Quality: A subjective assessment of the premise's overall potential for a good story, considering all factors.

        For each premise, provide your scores in the following JSON format:

        ```json
        {{
          "premise": "<The full text of the premise>",
          "scores": {{
            "Clarity of Goal": <Score>,
            "Conflict Potential": <Score>,
            "Originality": <Score>,
            "Genre Alignment": <Score>,
            "Overall Quality": <Score>
          }}
        }}
        ```

        Here are the premises to evaluate:
        {chr(10).join(f'- {premise}' for premise in premises)}

        Please provide your evaluation for each premise in separate JSON blocks, one block per premise.
        """
        # Get LLM evaluation
        evaluations_json = self.llm.generate_inference(eval_prompt)
        logger.debug(f"{caller_name}: LLM evaluations: {evaluations_json}")

        # Parse evaluations and select best premise
        best_premise = None
        best_score = -1

        for eval_block in evaluations_json.split('\n'):
            try:
                eval_data = json.loads(eval_block)
                premise = eval_data['premise']
                overall_quality = eval_data['scores']['Overall Quality']

                if overall_quality > best_score:
                    best_premise = premise
                    best_score = overall_quality

            except json.JSONDecodeError:
                logger.warning(f"{caller_name}: Warning: Skipping invalid JSON block: {eval_block}")

        if not best_premise:
            logger.error(f"{caller_name}: Error: Failed to select a premise from LLM evaluations. Returning None.")
            return None

        # --- Extract Goal and Conflicts from Premise ---
        logger.info(f"{caller_name}: Extracting GOAL and CONFLICTS from premise: '{best_premise}'")
        goal_and_conflicts_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        You are presented with a premise for a story.

        Your task is to identify one main goal for the characters,
        and a list of potential obstacles or conflicts they must overcome to achieve that goal.

        - Premise: {best_premise}
        - Goal: <Main goal for the story>
        - Conflicts:
            - Conflict 1: <Obstacle or conflict to overcome>
            - Conflict 2: <Obstacle or conflict to overcome>
            - Conflict 3: <Obstacle or conflict to overcome>

        Write in a style inspired by: {author_style}
        """
        goal_and_conflicts_data = self.llm.generate_inference(goal_and_conflicts_prompt).strip()
        logger.info(f"{caller_name}: Goal and Conflicts Data: {goal_and_conflicts_data}")

        goal = None
        conflicts = []
        for line in goal_and_conflicts_data.split('\n'):
            if line.startswith("- Goal:"):
                goal = line.replace("- Goal:", "").strip()
            elif line.startswith("- Conflict"):
                conflicts.append(line.replace("- Conflict", "").strip())

        # --- Update WorldBible ---
        logger.info(f"{caller_name}: Updating WORLD BIBLE with premise, goal and conflicts...")
        world_bible.add_premise(best_premise)
        world_bible.add_narrative(f"Goal: {goal}", "goals")
        world_bible.add_narrative(f"Conflicts:\n{chr(10).join(f'- {conflict}' for conflict in conflicts)}", "conflicts")

        logger.info(f"{caller_name}: Selected premise: '{best_premise}'")
        logger.info(f"{caller_name}: Goal: '{goal}'")
        logger.info(f"{caller_name}: Conflicts: {conflicts}")
        return best_premise

    def generate_premises_prompt(self, story_element: str) -> str:
        """
            Generates a prompt for the LLM to create premises based on a given element.

            Args:
                story_element (str): The starting element for the premise.
                num_premises (int, optional): The number of premises to generate.
                                                Defaults to 15.

            Returns:
                str: The formatted prompt for the LLM.
            """

        prompt = f"""
            Task: Generate {self.num_premises} story premises based on the given element for the genre {self.primary_genre}.

            Format for each premise:
            "In a world where [world_essence], [character_spark] must [conflict_catalyst] in order to [theme_resonance]."

            Elements:
            1. world_essence: A unique aspect of the story's setting or reality
            2. character_spark: A brief, distinctive description of the main character
            3. conflict_catalyst: The main challenge or action the character must undertake
            4. theme_resonance: The broader implication or goal of the character's journey

            Given element: "{story_element}"
            (This element can be used as any of the four elements in the premise. Be creative in how you incorporate it.)

            Guidelines:
            - Ensure each premise is unique and creative
            - Vary the elements across premises, using the given element in different roles
            - Keep each premise concise but descriptive
            - Ensure that you stick to the genre: **{self.primary_genre}**
            - Create diverse and interesting combinations that meaningfully incorporate the given element
            - Develop premises that explore various implications of the given element
            - Don't make it overly complicated. If you have clones, don't also add magic - unless it's relevant. Focus. Keep it simple.
            - Formatting: Help us parse your response...
              - Don't include any extra words in your response other than the premises.
              - Put each premise on its own line. (i.e. precede with a newline character: \n).
              - Wrap each premise in double quotation marks (").

            Example premise:
            "In a world where dreams are shared collectively, an isolated lucid dreamer must learn to connect with others in order to prevent a nightmare from becoming reality."

            Please generate {self.num_premises} story premises following this format and guidelines, creatively incorporating the given element.
            """

        return prompt

    def generate_premises(self, num_attempts: int = 5, genres: List[str] = None,) -> Optional[str]:
        """
            Generates a single story premise by randomly selecting one
            story element and then using the LLM to generate a list of premises.

            Args:
                num_attempts (int, optional): Number of attempts to generate
                                              a satisfactory premise. Defaults to 5.

            Returns:
                Optional[List[str]]: A list of generated premises, or None if
                                    no satisfactory premise is found.
            """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating premises...")

        element_provider = StoryElementProvider(genres=genres if genres else self.genres)
        logger.info(f"{caller_name}: Initiated StoryElementProvider...")

        # fetch random element
        logger.info(f"{caller_name}: Fetching random element...")
        element_types = ["world_essence", "character_spark", "conflict_catalyst", "theme_resonance",]
        random_elements = element_provider.get_random_elements(element_types=element_types, genres=genres if genres else self.genres,)
        logger.debug(f"{caller_name}: Random element: {random_elements}")

        # Select a random story element (any type)
        story_element = random_elements['story_element']
        logger.info(f"{caller_name}: Story element retrieved.")
        logger.info(f"{caller_name}:   - Story element: '{story_element}'")

        # Generate a premise prompt using the random element
        prompt = self.generate_premises_prompt(story_element)
        logger.info(f"{caller_name}: Prompt generated...")

        premises = []
        for i in range(1, num_attempts,):
            if(len(premises) == 0):
                # Using the LLM, generate the text containing several premise strings
                raw_premises_string_from_llm = self.llm.generate_inference(prompt).strip()
                logger.info(f"{caller_name}: LLM called for time: {i}")

                for premise_candidate in raw_premises_string_from_llm.split('\n'):
                    premise_candidate = premise_candidate.strip() # remove leading/trailing whitespace
                    # Basic validation
                    if premise_candidate.startswith('"In a world where') and premise_candidate.endswith('"'):
                        new_premise = premise_candidate.strip('"')
                        logger.info(f"{caller_name}: Premise candidate passed validation check: '{new_premise}'")
                        premises.append(new_premise)
            return premises

        logger.info(f"{caller_name}: Failed to generate premise after {num_attempts} attempts")
        return None

    def select_premise(self) -> None:
        """
        The LLM will select the premise from a list generated in the prior step.

        The basis for the premise selection will be the potential for quality storytelling vis-a-vis the opportunity to create a clear goal and compelling conflict.

        The output of this method will be 1. a PREMISE, 2. a GOAL and 3. a set of CONFLICTS or OBSTACLES in the way and necessary to overcome.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Selecting premise based on storytelling potential...")

        # Generate a list of premises using PremiseGenerator
        premise_generator = PremiseGenerator(num_premises=16)
        premises = premise_generator.generate_premises(genres=self.genres,)

        # Construct a prompt for the LLM to select the best premise
        premise_selection_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        You are presented with a list of potential premises for a story.

        Your task is to select the premise that offers the most compelling storytelling potential
        in terms of its ability to generate a clear goal for the characters and present a compelling conflict.

        Please provide the premise, along with one main goal for the characters,
        and a list of potential obstacles or conflicts they must overcome to achieve that goal.

        - Premise: <Chosen Premise>
        - Goal: <Main goal for the story>
        - Conflicts:
            - Conflict 1: <Obstacle or conflict to overcome>
            - Conflict 2: <Obstacle or conflict to overcome>
            - Conflict 3: <Obstacle or conflict to overcome>

        Premise List:
        {chr(10).join(f"- {premise}" for premise in premises)}

        Write in a style inspired by: {self.author_style}
        """

        # Use the LLM to select the premise, goal, and conflicts
        selected_premise_data = self.llm.generate_inference(premise_selection_prompt).strip()
        logger.info(f"{caller_name}: Selected Premise Data: {selected_premise_data}")

        # Parse the LLM's response to extract the selected premise, goal, and conflicts
        selected_premise = None
        goal = None
        conflicts = []
        for line in selected_premise_data.split('\n'):
            if line.startswith("- Premise:"):
                selected_premise = line.replace("- Premise:", "").strip()
            elif line.startswith("- Goal:"):
                goal = line.replace("- Goal:", "").strip()
            elif line.startswith("- Conflict"):
                conflicts.append(line.replace("- Conflict", "").strip())

        # Update the WorldBible with the selected premise, goal, and conflicts
        self.premise = selected_premise
        self.world_bible.add_premise(self.premise)

        #  Add the goal and conflicts to the World Bible
        #  you may want to add these to specific attributes in the WorldBible
        #  or modify existing attributes to accommodate these new elements
        self.world_bible.add_narrative(f"Goal: {goal}", "goals")
        self.world_bible.add_narrative(f"Conflicts:\n{chr(10).join(f'- {conflict}' for conflict in conflicts)}", "conflicts")

        logger.info(f"{caller_name}: Selected premise: '{selected_premise}'")
        logger.info(f"{caller_name}: Goal: '{goal}'")
        logger.info(f"{caller_name}: Conflicts: {conflicts}")
        return

    def get_premise(self, world_bible: WorldBible, author_style: str,
                    genres: List[str] = None, primary_genre: str = None,) -> Optional[str]:
        """
        Generates a list of premise candidates then
        selects a single story premise from that list.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Beginning to select final PREMISE from a list of ...")

        premises = self.generate_premises(genres=genres if genres else self.genres,)

        logger.info(f"{caller_name}: Generated PREMISES: {premises}")

        # @TODO:
        # - implement logic to select premise based on quality assessment vs. some rubric
        # - write premise to unused premises file saved as a separate Gist to be created
        if premises:
            premise = random.choice(premises)
            logger.info(f"{caller_name}: Selected PREMISE: {premise}")
            return premise
        else:
            logger.error(f"Error: Failed to generate premises after multiple attempts. Returning None.")
            return None

        # @TODO see below: testing_get_premise()

    def generate_premise(genre: str, author_style: str, num_premises: int = 16) -> Optional[str]:
        """Generates a premise following a rubric and prints the final selection on the last line."""
        llm = GeminiLLM()
        rubric = """
        Rubric for Evaluating Story Premises:

        Clarity of Goal (1-5): How well-defined and understandable is the protagonist's goal?
        Conflict Potential (1-5): Does the premise suggest a strong, compelling conflict with clear stakes?
        Originality (1-5): Is the premise unique, fresh, and intriguing?
        Genre Alignment (1-5): How well does the premise fit the specified genre(s)?
        Overall Quality (1-5): A subjective assessment of the premise's overall potential.

        Scoring:
        - 1: Very Poor
        - 2: Poor
        - 3: Average
        - 4: Good
        - 5: Excellent
        """

        prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        {rubric}

        Your task is to generate {num_premises} story premises for a {genre} story following this pattern:
        'In a world where [world_essence], a [character_spark] must [conflict_catalyst] to [thematic_resonance] and [aspirational_objective]'

        Then, evaluate each premise based on the rubric provided.
        Choose the premise with the highest Overall Quality score and print it on the LAST line.

        Write in a style inspired by: {author_style}
        """

        response = llm.generate_inference(prompt)
        lines = response.strip().split('\n')
        final_premise = lines[-1].strip() if lines else None

        # Log the entire response for debugging and validation
        logger.debug(f"Full LLM Response for Premise Generation:\n{response}")
        return final_premise

    def _generate_goals(self, world_bible: WorldBible, author_style: str) -> str:
        """
        Generates goals for the characters and plot based on the story premise and world elements.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating GOALS...")

        story_goals_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Given the below story elements, please generate three compelling goals for the story as a whole.
        These goals should be ambitious, achievable, and relevant to the story's themes and conflicts.
        - The plot should have a strong narrative arc, with clear goals and obstacles for the protagonist, and high stakes.
        - The plot should be well-paced, with twists, turns, and surprises that keep the reader engaged, creating a sense of tension and conflict.

        Here are some examples of different types of goals:
        - Thematic goal: "To restore balance to the celestial forces and prevent the world from falling into chaos."
        - Plot-driven goal: "To find the lost artifact that can break the ancient curse."
        - Character-driven goal: "To overcome his past trauma and find redemption."

        Consider these questions:
        - What is the ultimate goal of the story's theme?
        - What crucial events must occur to resolve the central conflict?
        - What are the most pressing desires of the main characters?
        - What are the consequences of failure to achieve these goals?

        Format:
        - Goal 1: <Detailed description of the goal>
        - Goal 2: <Detailed description of the goal>
        - Goal 3: <Detailed description of the goal>

        Do not mention any specific characters by name. Use placeholders like protagonist, antagonist, main characters, etc.

        The characters will be filled in later.

        Primary Genre: {self.primary_genre}

        Premise: {self.premise}

        Write in a style inspired by: {author_style}
        """

        goals_narrative = self.llm.generate_inference(story_goals_prompt).strip()
        logger.info(f"{caller_name}:     - Goals Narrative: {goals_narrative}")
        return goals_narrative

    def _generate_conflicts(self, world_bible: WorldBible, author_style: str) -> str:
        """
        Generates conflicts and adds them to the World Bible.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating CONFLICTS...")

        story_conflicts_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Given the below story elements, please generate three conflicts that challenge the goals of the story.
        These conflicts should increase tension, create obstacles, and drive the plot forward.

        Format:
        - Conflict 1: <Detailed description of the conflict>
        - Conflict 2: <Detailed description of the conflict>
        - Conflict 3: <Detailed description of the conflict>

        Genres: {self.genres}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        <events>
        ### Events:
        {self.world_bible.retrieve_narrative_block("events")}
        </events>

        <cultures>
        ### Cultures:
        {self.world_bible.retrieve_narrative_block("cultures")}
        </cultures>

        Write in a style inspired by: {self.author_style}
        """

        conflicts_narrative = self.llm.generate_inference(story_conflicts_prompt).strip()
        logger.info(f"{caller_name}: Conflicts narrative: {conflicts_narrative}")
        self.world_bible.add_narrative(conflicts_narrative, "conflicts")

        logger.info(f"{caller_name}: Generated CONFLICTS.")
        return conflicts_narrative

    def generate_story_foundation(self, world_bible: WorldBible, author_style: str) -> Tuple[str, str, str]:
        """
        Generates the core foundation of the story, including:
            - Premise
            - Goals
            - Conflicts

        Returns:
            Tuple[str, str, str]: A tuple containing the premise, goals narrative, and conflicts narrative.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating STORY FOUNDATION (premise, goals, conflicts)...")

        # Generate Premise
        premise = self.get_premise(world_bible, author_style, self.genres, self.primary_genre)
        logger.info(f"{caller_name}:   - Premise generated: {premise}")

        # Generate Goals
        goals_narrative = self._generate_goals(world_bible, author_style)
        logger.info(f"{caller_name}:   - Goals Narrative: {goals_narrative}")

        # Generate Conflicts
        conflicts_narrative = self._generate_conflicts(world_bible, author_style)
        logger.info(f"{caller_name}:   - Conflicts Narrative: {conflicts_narrative}")

        logger.info(f"{caller_name}: STORY FOUNDATION generated.")
        return premise, goals_narrative, conflicts_narrative


# We will move this EpubCreator class to a separate module prior to production.
# Shown here for now as a reference to understand how to structure the output
# from StoryWriter. And what metadata and/or tags to save with the Gist.
class EpubCreator:
    """
        A class to facilitate the creation of EPUB books using ebooklib.

        # Example usage:
        if __name__ == "__main__":
            creator = EpubCreator("My Amazing Book", "John Doe")
            creator.add_chapter("Chapter 1", "<h1>Chapter 1</h1><p>This is the first chapter.</p>")
            creator.add_chapter("Chapter 2", "<h1>Chapter 2</h1><p>This is the second chapter.</p>")
            creator.generate_epub("my_amazing_book.epub")

        Notes:

        A class structure using `ebooklib` that encapsulates the EPUB creation process and adds some useful methods and attributes.
        This class will provide a more object-oriented approach to creating an EPUB book programmatically.

        This `EpubCreator` class provides a more structured and extensible way to create EPUB books using `ebooklib`. Here's a breakdown of its features:

        1. **Initialization**: The constructor sets up basic metadata for the book.

        2. **Chapter Management**:
          - `add_chapter()` method for adding chapters with content.
          - `add_chapter_from_file()` method for adding chapters from HTML files.

        3. **Asset Management**:
          - `add_image()` method for including images in the book.
          - `add_css()` method for adding CSS stylesheets.
          - `set_cover()` method for setting the book's cover image.

        4. **Structure**:
          - `set_table_of_contents()` method for customizing the table of contents.

        5. **Generation**:
          - `generate_epub()` method for finalizing and writing the EPUB file.

        6. **Utility**:
          - `_generate_identifier()` private method for creating unique book identifiers.

        This class structure provides several benefits:

        1. **Encapsulation**: All EPUB-related operations are contained within the class.
        2. **Flexibility**: You can easily extend the class with new methods as needed.
        3. **Readability**: The code is organized into logical methods, making it easier to understand and maintain.
        4. **Reusability**: You can create multiple books using the same class structure.

        To use this class, you would typically:

        1. Create an instance of `EpubCreator`.
        2. Add chapters, images, and other content as needed.
        3. Set the cover and table of contents if desired.
        4. Generate the EPUB file.

        This structure allows for programmatic book creation while providing a clean and intuitive interface.
        You can easily extend this class with additional methods for more specific needs, such as adding footnotes,
        creating complex layouts, or integrating with external content sources.
        """

    def __init__(self, title: str, author: str, language: str = "en"):
        """
        Initialize the EpubCreator with basic book metadata.

        :param title: The title of the book
        :param author: The author of the book
        :param language: The language of the book (default is English)
        """
        self.book = epub.EpubBook()
        self.book.set_title(title)
        self.book.add_author(author)
        self.book.set_language(language)
        self.book.set_identifier(self._generate_identifier())
        self.chapters: List[epub.EpubHtml] = []

    def _generate_identifier(self) -> str:
        """
        Generate a unique identifier for the book.

        :return: A unique string identifier
        """
        import uuid
        return str(uuid.uuid4())

    def add_chapter(self, title: str, content: str, file_name: Optional[str] = None) -> None:
        """
        Add a chapter to the book.

        :param title: The title of the chapter
        :param content: The HTML content of the chapter
        :param file_name: Optional file name for the chapter (default is auto-generated)
        """
        if file_name is None:
            file_name = f"chapter_{len(self.chapters) + 1}.xhtml"

        chapter = epub.EpubHtml(title=title, file_name=file_name, lang=self.book.language)
        chapter.content = content
        self.book.add_item(chapter)
        self.chapters.append(chapter)

    def add_chapter_from_file(self, title: str, file_path: str) -> None:
        """
        Add a chapter to the book from an HTML file.

        :param title: The title of the chapter
        :param file_path: The path to the HTML file
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        self.add_chapter(title, content, os.path.basename(file_path))

    def add_image(self, file_path: str, alt_text: str) -> None:
        """
        Add an image to the book.

        :param file_path: The path to the image file
        :param alt_text: Alternative text for the image
        """
        with open(file_path, 'rb') as file:
            image = epub.EpubItem(
                uid=f"image_{len(self.book.get_items()) + 1}",
                file_name=os.path.basename(file_path),
                media_type=f"image/{os.path.splitext(file_path)[1][1:]}",
                content=file.read()
            )
        self.book.add_item(image)

    def add_css(self, file_path: str) -> None:
        """
        Add a CSS file to the book.

        :param file_path: The path to the CSS file
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            css = epub.EpubItem(
                uid=f"style_{len(self.book.get_items()) + 1}",
                file_name=os.path.basename(file_path),
                media_type="text/css",
                content=file.read()
            )
        self.book.add_item(css)

    def set_cover(self, file_path: str) -> None:
        """
        Set the cover image for the book.

        :param file_path: The path to the cover image file
        """
        with open(file_path, 'rb') as file:
            self.book.set_cover("cover.jpg", file.read())

    def set_table_of_contents(self, toc: List[Dict[str, str]]) -> None:
        """
        Set a custom table of contents.

        :param toc: A list of dictionaries with 'title' and 'href' keys
        """
        self.book.toc = [(epub.Section(item['title']), []) for item in toc]

    def generate_epub(self, output_path: str) -> None:
        """
        Generate the EPUB file.

        :param output_path: The path where the EPUB file will be saved
        """
        # Add default NCX and Nav files
        self.book.add_item(epub.EpubNcx())
        self.book.add_item(epub.EpubNav())

        # Define the book's spine
        self.book.spine = ['nav'] + self.chapters

        # Write the EPUB file
        epub.write_epub(output_path, self.book, {})

class StoryWriter():
    AUTHORS = [
        "J.R.R. Tolkein", "J.K. Rowling", "George R.R. Martin", "Stephen King", "Oscar Wilde",
        "Mark Twain", "George Orwell", "Aldous Huxley", "Ernest Hemingway", "Ray Bradbury", "Virginia Woolf",
        "Audrey Niffenegger (The Time Traveler's Wife)", "Margaret Atwood (The Handmaid's Tale)",
        "Graeme Simsion (The Rosie Project)", "Ernest Cline (Ready Player One)", "Leigh Bardugo (Six of Crows)",
        "Gabriel García Márquez (One Hundred Years of Solitude)", "Brandon Sanderson (Mistborn: The Final Empire)",
        "Suzanne Collins (The Hunger Games)", "Laini Taylor (Daughter of Smoke & Bone)", "Dave Eggers (The Circle)",
        ]

    AVAILABLE_GENRES = [
        "generic",
        "romance", "romance", "romance",
        ]

    AUTHOR_PERSONA = """
        <persona>
        You are an award-winning author with a penchant for expansive, intricately
        woven stories. You are engaged in writing your next award-winning novel.
        </persona>
        """

    EDITOR_PERSONA = """
        <persona>
        You are a highly experienced and critically acclaimed book editor working for a prestigious publishing house specializing in fantasy literature.
        You have a keen eye for detail, a deep understanding of narrative structure, and a passion for creating immersive and engaging worlds.

        Your primary goal is to evaluate and refine fantasy world-building documents, ensuring they are consistent, compelling, and have strong potential for captivating stories.

        You excel at:
        * Identifying inconsistencies and plot holes
        * Assessing character depth and arc potential
        * Evaluating world logic and plausibility
        * Suggesting creative improvements and new directions
        * Providing constructive feedback in a clear and concise manner
        * Recognizing and appreciating diverse storytelling approaches and experimental narrative techniques

        You are familiar with a wide range of fantasy subgenres and tropes, and you are always looking for fresh and original ideas.
        You are also adept at working with authors to realize their creative vision while balancing market expectations and genre conventions with innovative approaches.
        </persona>
        """

    def __init__(self, premise: str = None, genres: List[str] = None,
                 author_style: str = None, working_title: str = None,
                 num_main_characters: int = 3, num_supporting_characters: int = 6,):

        # Handle genres
        self.genres = None
        self.primary_genre = None
        self._set_genres(genres)

        # Instantiate variables
        self.premise = premise
        self.working_title = working_title
        self.num_main_characters = num_main_characters
        self.num_supporting_characters = num_supporting_characters
        self.characters = None
        self.series_bible = None
        self.plot_structure = self._get_plot_structure(self.primary_genre)
        self.author_style = random.choice(StoryWriter.AUTHORS) if author_style is None else author_style

        # Instantiate dependency classes
        self.llm = GeminiLLM()
        self.world_bible = WorldBible()
        self.series_bible = SeriesBible()
        self.github_manager = GithubManager()
        self.premise_generator = PremiseGenerator()
        self.prompt_library = WritingPromptLibrary()

        # set author style in world bible
        self.world_bible.add_author_style(self.author_style)

        # other important variables:
        self.world_bible_gist_id = None
        self.world_bible_gist_url = None
        self.series_bible_gist_id = None
        self.series_bible_gist_url = None
        self.one_pager_gist_id = None
        self.one_pager_gist_url = None

    def _set_genres(self, genres: List[str] = None,) -> str:
        """Sets list of genres and the primary genre for this instance"""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Setting GENRES...")

        # return provided list for genres with first element as primary
        if genres is not None:
            self.genres = genres
            self.primary_genre = genres[0] if genres is list else genres
            return

        # otherwise, select random genre

        # Get all available genre names
        # all_available_genres_names = Genre.list_all_genres_names()
        # above yields: "save_the_cat", "seven_basic_plots", etc., so...
        all_available_genres_names = StoryWriter.AVAILABLE_GENRES

        # Choose a random genre
        random_genre = random.choice(all_available_genres_names)
        logger.info(f"{caller_name}: Randomly selected GENRE: {random_genre}")

        # Set the random genre and include generic to match story elements
        self.genres = [random_genre,]
        self.primary_genre = random_genre
        return

    def _get_plot_structure(self, genre: str = None,):
        """Genre-Specific Plot Structure"""
        if self.primary_genre.lower() == Genre.ROMANCE.value["name"].lower():
            plot_structure = Genre.ROMANCE.value['beats']  # Accessing Romance Beats
        elif self.primary_genre.lower() == Genre.SAVE_THE_CAT.value["name"].lower():
            plot_structure = Genre.SAVE_THE_CAT.value['beats'] # Accessing Save the Cat Beats
        else:
            # plot_structure = "No specific plot structure defined for this genre."
            plot_structure = Genre.SAVE_THE_CAT.value['beats'] # Accessing Save the Cat Beats

        return plot_structure

    def _x6_write_chapter(self, chapter_number: int):
        """
        Generates the text for a single chapter.

        @TODO:
        - Access relevant data from self.series_bible (character profiles,
          location dossiers, plot outline, chapter outline).
        - Utilize dynamic prompts based on the chapter outline and
          Karpman Drama Triangle dynamics.
        - Implement a mechanism to control chapter length and pacing.
        - Incorporate feedback loops for refinement.
        """
        chapter_text = f"## Chapter {chapter_number}\n\n"
        # ...  generate chapter text using self.series_bible data ...
        return chapter_text

    def _15_book(self):
        """Writes the actual book, chapter by chapter from the chapter outlines."""

        prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        ### Guidelines:
        - **Focus on Execution:**
          - The writers need to focus on executing the framework and prompts in a way that feels fresh, original, and emotionally resonant.
          - They should strive to subvert clichés, create unique and memorable characters, and build immersive worlds that draw readers in.
        - **Show, Don't Tell:** The writers should focus on showing the characters' emotions and motivations through their actions and dialogue, rather than relying on telling.
        - **Reader Engagement:**
          - The ultimate measure of a good story is its ability to engage readers.
          - The writers should consider whether their stories are emotionally compelling, thought-provoking, and ultimately satisfying for the intended audience.
        """
        logger.info(f"{caller_name}: Generating BOOK...")
        logger.info(f"{caller_name}: Prompt: {prompt}")

        self.llm.generate_inference(prompt).strip()
        return

        # --- Chapter Writing ---
        for chapter_number in range(1, len(self.series_bible['plot_outline']) + 1):  # Assuming plot_outline is a list in series_bible
            chapter_text = self._08_write_chapter(chapter_number)
            logger.info(f"{caller_name}: Generated Chapter {chapter_number}:\n{chapter_text}")

    def _14_1pager(self):
        """Generates a one-pager summarizing the story idea and saves it to a new Gist."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating ONE-PAGER in ONE-SHOT...")

        # --- Extract Data from World Bible ---
        series_bible_content = '\n'.join(self.series_bible.chapter_outlines)

        # --- Construct the One-Pager Prompt ---
        one_pager_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        <task>
        Your task is to create a comprehensive and compelling ONE-PAGER for a novel, aimed at attracting potential editors or publishers.

        Use the following TEMPLATE and fill in all the sections with creative and engaging content.
        Refer to the provided CONTEXT, especially the chapter outlines in the Series Bible, to accurately represent the story.
        </task>

        ---

        </template>

        #### *ONE-PAGER*

        # {self.world_bible.working_title} ⭐

        **Genres:** [Genres]

        ## Logline 🎬
        [25 words max]
        [Write a concise, impactful summary of the story's core concept and conflict.]

        ## Scorecard 📊
        [Provide a quantitative assessment of the book's potential for quality storytelling and audience enjoyment.]
        [Score each category from 0-100, with brief justifications for each score.]

        *Scores are based on estimated percentiles, where 50 is the median across all genres, books, and authors.*

        - [Score] : **Plot Originality and Execution** 💡
          - [Brief justification]
        - [Score] : **Character Development and Depth** 🧠
          - [Brief justification]
        - [Score] : **World-building and Setting** 🌎
          - [Brief justification]
        - [Score] : **Plot Structure and Narrative** 🛠️
          - [Brief justification]
        - [Score] : **Thematic Resonance** 🌳
          - [Brief justification]
        - [Score] : **Pacing and Structure** ⏱️
          - [Brief justification]
        - [Score] : **Prose Quality and Style** ✍️
          - [Brief justification]
        - [Score] : **Market Potential and Appeal** 🫰
          - [Brief justification]
        - [Score] : **Emotional Impact** 💘
          - [Brief justification]
        - [Score] : **Uniqueness within Genre** 🦄
          - [Brief justification]
        - [Score] : **Overall Narrative Cohesion**: 🪡
          - [Brief justification]

        Overall Score: **[Score average]** 🏆

        **Final Assessment:** 🤔
        [2-3 sentences summarizing the book's strengths, areas for improvement, and overall potential based on the scores.]

        ## Series Potential 🌱
        - Score: [Score] 💫
          - [50 words max]
          - [If applicable, briefly outline the potential for sequels or a series.]
          - [Mention any overarching storylines or unresolved conflicts that could be explored in future books.]

        ## Target Audience 🎭
        [Describe the ideal reader for this book. Be specific about age, interests, and reading preferences.]
        [Include specific affinity titles and authors as thoroughly as possible to guide the marketing team.]

        **Affinity Titles and Authors:** 🤝
        - *[Title 1]* by [Author 1]
        - *[Title 2]* by [Author 2]
        - *[Title 3]* by [Author 3]
        - [...]

        ## Market Comparison 🔍
        [Think of two existing books that share similarities with your story.]
        <b>*\[Different Book Title\]* meets *\[Different Book Title\]*</b> 💥
        - [A brief explanation of how this story combines elements of both.]

        ## Unique Selling Points 💎
        - [What makes this book stand out from others in the genre?]
        - [Highlight a unique feature or element that would make readers choose this story.]

        ## Blurb 🎁
        [100-200 words]
        [Write a short, enticing summary that appears on the back cover or in online listings.]
        [Purpose: To entice readers to buy the book by giving a taste of the story without revealing too much.]

        ## Main Characters 👥
        1. **[Character Name]:** [One-sentence description]
        2. **[Character Name]:** [One-sentence description]
        3. **[Character Name]:** [One-sentence description]

        ## MacGuffin 🎯
        [50-75 words]
        [Describe the central object, device, or goal that drives the plot forward.]
        [Explain its significance to the characters and the overall story.]
        [Highlight how it creates conflict and motivates character actions.]

        ## Conflicts & Obstacles 🏹
        [100-150 words]
        - **External Conflicts:** ⚔️ [List 2-3 major external challenges faced by the protagonist(s)]
        - **Internal Conflicts:** 🌀 [Describe 1-2 internal struggles or moral dilemmas faced by key characters]
        - **Antagonistic Forces:** 👿 [Outline the main opposition or villains in the story]
        [Briefly explain how these conflicts escalate throughout the narrative]

        ## Key Themes 🧩
        - **[Theme 1]:** [One-sentence description]
        - **[Theme 2]:** [One-sentence description]
        - **[Theme 3]:** [One-sentence description]

        ## Synopsis ✏️
        [1,000 words max]
        [Provide a brief, captivating overview of the plot, highlighting key events and character arcs.]
        [Include a detailed breakdown of the entire plot, major character arcs, and the resolution of the story.]

        ## Ending 🏁
        [100 words max]
        [Reveal all major plot twists and the ending.]

        ## Cover Art 🎨
        [Generate 3 distinct AI image prompts for potential cover art designs.]
        [Each prompt should capture key elements of the story, appeal to the target audience, and stand out in the genre.]
        [For each prompt, add a brief explanation (1-2 sentences) of how it relates to the story and why it would appeal to the target audience.]
        - **Plot**: 🗡️
          - [Describe a striking visual concept that encapsulates the main theme or conflict of the book.]
          - [Include details on style, mood, colors, and key elements to include.]
          - [Briefly explain how it relates.]
        - **Character**: 👤
          - [Focus on a character-centric design, featuring the protagonist or a pivotal scene.]
          - [Specify artistic style, lighting, and any symbolic elements to incorporate.]
          - [Briefly explain how it relates.]
        - **Background**: 🏰
          - [Create a more abstract or symbolic representation of the story's core ideas.]
          - [Detail the composition, color palette, and any unique visual metaphors to use.]
          - [Briefly explain how it relates.]

        </template>

        ---

        <context>
        CONTEXT:

        **Premise:** {self.premise}

        **Series Bible (Chapter Outlines):**
        ```yaml
        {series_bible_content}
        ```
        </context>
        """

        # --- Generate the One-Pager ---
        one_pager = self.llm.generate_inference(one_pager_prompt).strip()
        one_pager += "<hr/>"
        one_pager += f"\n\n**Starting Premise:** {self.premise}"
        one_pager += f"\n\n**Genre:** {self.primary_genre}"
        one_pager += f"\n\n**Genres:** {', '.join(self.genres)}"
        one_pager += f"\n\n**Author Style:** {self.world_bible.author_style}"
        one_pager += f"\n\n**References:** [Chapter Outlines]({self.series_bible_gist_url}) | [World Bible]({self.world_bible_gist_url})"

        # --- Save to a new Gist ---
        unsanitized_filename = f"onepager_{self.world_bible.working_title}.md"
        sanitized_filename = self.github_manager.sanitize_filename(unsanitized_filename)
        logger.info(f"{caller_name}: Creating and saving ONE-PAGER to Gist...")
        working_title = self.world_bible.working_title if self.world_bible.working_title else "Untitled"
        description = f"One-Pager for '{working_title}'"
        self.one_pager_gist_id, self.one_pager_gist_url = self.github_manager.save_to_gist(
            filename=sanitized_filename, content=one_pager, description=description,)
        logger.info(f"{caller_name}: ONE-PAGER saved to Gist: {self.one_pager_gist_url}")

    def _13_outline(self):
        """
        Generates a series of chapter outlines incorporating plot and character
        arc development, genre-specific plot structure, and character relationships.

        Saves to Series Bible, a new Gist, for the first time. We begin to leave the
        World Bible (world creation) and begin to create the Series Bible (storywriting) phase.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating CHAPTER OUTLINES...")

        for chapter_number, beat in enumerate(self.plot_structure, 1,):  # Iterate over the beats

            # --- Get Beat Questions ---
            beat_questions = beat.get('prompts', [])  # Get questions from the beat (if available)

            # --- Get Random Prompts ---
            prompts_menu = self.prompt_library.get_stringified_prompt_menu()

            # --- Construct Chapter Outline Prompt ---
            chapter_outline_prompt = f"""
            {StoryWriter.AUTHOR_PERSONA}

            Task: Create Chapter Outline for Chapter {chapter_number}

            Instructions:
            1. Chapter Number: {chapter_number}

            2. Beat Questions and Answers:
              - For each question, first cite each of the questions from the beat verbatim.
                Then rephrase the question in the context of the story.
                Then, provide the answers to the rephrased questions.
                - Beat Questions Verbatim:
                  {chr(10).join(f'- "{question}"' for question in beat_questions)}
                - Questions Rephrased in Story Context:
                  - "[Question 1 (rephrased)]"
                  - "[Question 2 (rephrased)]"
                  - "[Question 3 (rephrased)]"
                  - ...
                - Answers:
                  - [Answer to rephrased question 1]
                  - [Answer to rephrased question 2]
                  - [Answer to rephrased question 3]
                  - ...

            3. Plot Points:
              Outline 3-5 key plot points for this chapter, aligning with the chapter's beat.
              Genre Structure: Refer to the following plot structure:
              <plot_structure>
              {yaml.dump(self.plot_structure)}
              </plot_structure>

            4. Character Development:
              For each relevant character:
              - Detail their arc progression in this chapter.
              - Assign numerical values (-100 to 100) for their arc position at the chapter's beginning and end.
              - Provide 1-2 specific development points.

            5. Character Relationships:
              Include important notes about character interactions and relationship developments.

            6. Open Loops:
              Specify 1-2 new story loops to be opened in this chapter, with brief descriptions.

            7. Close Loops:
              If applicable, identify loops from previous chapters to be closed, referencing their origin.

            8. Dynamic Prompts:
              Choose 3-5 relevant prompts from the list below and include their full text:
              <prompt_menu>
              {yaml.dump(prompts_menu)}
              </prompt_menu>

            9. Dialogue Cues:
              For each important interaction in the plot points:
              - Suggest specific lines of dialogue or key phrases that the characters could say.
              - Focus on dialogue that reveals character, creates conflict, or advances the plot.

            10. Chapter Title: [Create a concise, engaging title reflecting the chapter's content and corresponding beat.]



            World Bible Reference
            Refer to the below World Bible for detailed information on characters, relationships, locations, items, events, and cultures.

            Output Format:
            Chapter [chapter_number]:  Corresponds to Beat [beat_number] - [beat_name]
                Beat Questions Verbatim:
                - "[Rephrased Beat question 1]"
                - "[Rephrased Beat question 2]"
                - "[Rephrased Beat question 3]"
                - ...
                Questions Rephrased in Story Context:
                - "[Answer to rephrased question 1]"
                - "[Answer to rephrased question 2]"
                - "[Answer to rephrased question 3]"
                - ...
                Answers:
                - [Answer to rephrased question 1]
                - [Answer to rephrased question 2]
                - [Answer to rephrased question 3]
                - ...
                Plot Points:
                - [plot_point_1]
                - [plot_point_2]
                - [plot_point_3]
                - [plot_point_4]
                - [plot_point_5]
                Character Development:
                    [character_name]:
                        Arc: [arc_type]
                        Beginning: [numerical_value]
                        End: [numerical_value]
                        Development Points:
                        - [development_point_1]
                        - [development_point_2]
                Character Relationships:
                - [relationship_note]
                Open Loops:
                - Loop 1: [brief_description]
                - Loop 2: [brief_description]
                Close Loops:
                - Chapter [previous_chapter_number] loop: [brief_description]
                Dynamic Prompts:
                - "[full_text_of_chosen_prompt_1]"
                - "[full_text_of_chosen_prompt_2]"
                - "[full_text_of_chosen_prompt_3]"
                Dialogue Cues:
                - [Character A] to [Character B]: [Dialogue suggestion or key phrase]
                - [Character B] to [Character A]: [Dialogue suggestion or key phrase]
                - [Character C] to [Character D]: [Dialogue suggestion or key phrase]
                - ... [Other characters as necessary...]
                Title: [chapter_title]


            <example_output>
            Example Output:

            Chapter 1: Corresponds to Beat 1 - Opening Image
                Title: The Weaver's Loom
                Beat Questions Verbatim:
                - "How can you visually represent the protagonist's world before the story begins?"
                - "What symbolic elements can you include to foreshadow the journey ahead?"
                - "How does the opening reflect the story's genre and tone?"
                Questions Rephrased in Story Context:
                - "How can you visually represent Sapphire's world before she receives the amulet?"
                - "What symbolic elements can you incorporate to foreshadow Sapphire's journey into the world of dream weaving?"
                - "How does the bustling marketplace of Azure reflect the fantastical elements of the story?"
                Answers:
                - The bustling marketplace of Azure, with its vibrant colors, exotic goods, and diverse people, represents Sapphire's ordinary world before the arrival of the mysterious stranger. However, the presence of dream weavers and their intricate tapestries hints at the magical undercurrents that will soon disrupt her life.
                - The mythical landscape depicted in Sapphire's tapestry foreshadows her future adventures in the realm of dreams. The amulet, with its intricate design and mysterious aura, also serves as a symbol of the unknown power that she will encounter.
                - The marketplace, despite its mundane appearance, is a place where dreams are bought and sold, hinting at the fantastical nature of the story. The vibrant colors and energetic atmosphere also reflect the sense of wonder and adventure that will unfold."
                Plot Points:
                - The story opens in a bustling marketplace in the city of Azure, a vibrant hub of trade and culture.
                - We meet Sapphire, a skilled weaver known for her intricate tapestries that capture the essence of dreams.
                - Sapphire works on an ambitious tapestry depicting a mythical landscape, foreshadowing the journey to come.
                - A mysterious stranger in dark robes approaches Sapphire, sparking her curiosity.
                - The stranger gifts Sapphire a small, intricately woven amulet, imbued with a sense of foreboding.
                Character Development:
                    Sapphire:
                        Arc: Rise
                        Beginning: -10
                        End: 5
                        Development Points:
                        - Sapphire is initially depicted as a confident and skilled weaver, content in her routine.
                        - She shows growing curiosity and openness to change when intrigued by the stranger and the amulet.
                    Mysterious Stranger:
                        Arc: Unknown
                        Beginning: 0
                        End: 10
                        Development Points:
                        - The stranger radiates an aura of power and intrigue, shrouded in mystery.
                        - Their interaction with Sapphire suggests a hidden purpose and influence on her future.
                Character Relationships:
                - The encounter between Sapphire and the mysterious stranger is marked by tension and intrigue.
                Open Loops:
                - Loop 1: The nature and purpose of the stranger's gift to Sapphire remains a mystery.
                - Loop 2: The significance of Sapphire's mythical landscape tapestry and its connection to future events is unclear.
                Close Loops:
                - None (This is the opening chapter)
                Dynamic Prompts:
                - "Descriptive Prompts: Describe the bustling marketplace of Azure, capturing its sights, sounds, and energy."
                - "Worldbuilding Prompts: Detail the city of Azure, its architecture, culture, and people."
                - "Dialogue Prompts: Craft the initial conversation between Sapphire and the mysterious stranger, focusing on building tension and intrigue."
                Dialogue Cues:
                - Sapphire to Mysterious Stranger: "What is this amulet for?"
                - Mysterious Stranger to Sapphire: "It is a guide. But beware its power."
            </example_output>

            <context>
            World Bible

            Genre: {self.primary_genre}

            Premise: {self.premise}

            Title: {self.world_bible.working_title}

            <characters>
            ### Characters:
            {self.world_bible.characters}
            </characters>

            <goals>
            ### Goals:
            {self.world_bible.retrieve_narrative_block("goals")}
            </goals>

            <conflicts>
            ### Conflicts:
            {self.world_bible.retrieve_narrative_block("conflicts")}
            </conflicts>

            <relationships>
            ### Relationships:
            {self.world_bible.retrieve_narrative_block("relationships")}
            </relationships>

            <rules>
            ### Rules:
            {self.world_bible.retrieve_narrative_block("rules")}
            </rules>

            <locations>
            ### Locations:
            {self.world_bible.retrieve_narrative_block("locations")}
            </locations>

            <items>
            ### Items:
            {self.world_bible.retrieve_narrative_block("items")}
            </items>

            <events>
            ### Events:
            {self.world_bible.retrieve_narrative_block("events")}
            </events>

            <cultures>
            ### Cultures:
            {self.world_bible.retrieve_narrative_block("cultures")}
            </cultures>

            <critique>
            ### Critique:
            {self.world_bible.retrieve_narrative_block("critique")}
            </critique>

            <revision>
            ### Revision:
            {self.world_bible.retrieve_narrative_block("revision")}
            </revision>

            </context>

            Format your response in plain text only. Do not include any markdown, HTML or other formatting.
            """

            # Generate the chapter outline using the LLM
            chapter_outline_yaml = self.llm.generate_inference(chapter_outline_prompt).strip()
            logger.info(f"{caller_name}: Chapter Outline {chapter_number}: {chapter_outline_yaml}")

            # Add the generated chapter outline (already YAML) to the SeriesBible
            self.series_bible.add_chapter_outline(chapter_outline=chapter_outline_yaml, chapter_number=chapter_number)

        # === Save the Series Bible to a Gist ===
        logger.info(f"{caller_name}: Saving Series Bible to Gist...")
        series_bible_yaml = "\n".join(self.series_bible.chapter_outlines)  # Combine the outlines
        self.series_bible_gist_id, self.series_bible_gist_url = self.github_manager.save_to_gist(
            filename=f"seriesbible_{self.world_bible.working_title.lower().replace(' ', '_')}.yaml",
            content=series_bible_yaml,
            description=f"Series Bible for '{self.world_bible.working_title}'",
        )
        logger.info(f"{caller_name}: SERIES BIBLE saved to Gist: {self.series_bible_gist_url}")

    def _12_save(self):
        """Creates and saves the World Bible to a Gist."""
        MAX_DESCRIPTION_LENGTH = 255

        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Creating and saving WORLD BIBLE to Gist...")

        world_bible_serialized = vars(self.world_bible)
        world_bible_stringified = str(world_bible_serialized) # Directly convert to string
        logger.info(f"{caller_name}: WORLD BIBLE: {self.world_bible}")

        # Convert working title to snake case for the gist description
        # working_title_snake_case = re.sub(r'[^a-z0-9]+', '_', self.working_title.lower()).strip('_')
        gist_description = f'World Bible for "{self.working_title}": {self.premise}'

        # Ensure the description does not exceed 255 characters
        if len(gist_description) > MAX_DESCRIPTION_LENGTH:
            gist_description = gist_description[:MAX_DESCRIPTION_LENGTH]

        self.world_bible_gist_id, self.world_bible_gist_url = self.github_manager.save_to_gist(
            # filename=f"worldbible_{random.randint(1, 999999,)}.txt",
            # filename=f"worldbible_{working_title_snake_case}.txt",
            filename=self.github_manager.sanitize_filename(f"worldbible_{self.working_title}.json"),
            content=world_bible_stringified,
            description=gist_description,
        )
        logger.info(f"{caller_name}: WORLD BIBLE saved to Gist: {self.world_bible_gist_url}")

    def _11_revision(self):
        """Revises the WorldBible based on editor's critique and suggestions"""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating REVISION...")

        revision_prompt = f"""
        You are an author provided with the following "World Bible" document which includes (at the end) a critique from your book editor.

        Your task is to revise the "WorldBible" based on the editor's feedback.

        1.  Carefully analyze the editor's critique and identify the key areas for improvement.
        2.  Address each critique point by revising existing elements or adding new ones.
        3.  Ensure that the revised "WorldBible" is consistent, focused, and compelling.

        ## World Bible

        Genre: {self.primary_genre}

        Premise: {self.premise}

        Title: {self.world_bible.working_title}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        <events>
        ### Events:
        {self.world_bible.retrieve_narrative_block("events")}
        </events>

        <cultures>
        ### Cultures:
        {self.world_bible.retrieve_narrative_block("cultures")}
        </cultures>

        <critique>
        ### Critique:
        {self.world_bible.retrieve_narrative_block("critique")}
        </critique>

        Continue being creative and imaginative!

        Write in a style inspired by: {self.author_style}

        Remember, your goal is to create a rich and immersive world that has the potential to support compelling stories.
        """

        revision = self.llm.generate_inference(revision_prompt).strip()
        logger.info(f"{caller_name}: Revision: {revision}")
        self.world_bible.add_narrative(revision, "revision")

        logger.info(f"{caller_name}: Generated REVISION.")
        return revision

    def _10_critique(self):
        """Generates critique from editor for feedback"""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating cultures...")

        critique_prompt = f"""
        {StoryWriter.EDITOR_PERSONA}

        <task>
        You are tasked with evaluating the following story with the working title: {self.world_bible.working_title}.

        Your goal is to provide a comprehensive CRITIQUE, assessing the following aspects:

        1. **Consistency:** Are the elements and narratives internally coherent? Are there any contradictions or inconsistencies? Consider both explicit and implicit elements of the world.
        2. **Focus:** Is the core conflict of the world clear and compelling? Are all the elements relevant to the main themes and conflicts? How well does the world support the central narrative?
        3. **Character Development:** Are the characters well-developed with distinct personalities, motivations, and potential for growth? Do their arcs align with the overall narrative direction? Consider both major and minor characters.
        4. **World Logic:** Is the world internally consistent in terms of its magic systems, cultures, and historical events? Are the rules of the world clear and believable? How well are these elements integrated into the narrative?
        5. **Originality:** Does the world offer something fresh and unique? Is it intriguing and captivating enough to draw in readers? How does it stand out from other works in the genre?
        6. **Cultural Depth:** How well-developed and distinct are the cultures within the world? Do they have unique customs, beliefs, and social structures that feel authentic and contribute to the narrative?
        7. **Narrative Potential:** Does the world-building create opportunities for compelling storylines and character arcs? Are there unexplored aspects of the world that could lead to interesting plot developments?
        8. **Sensory Immersion:** How well does the world-building engage the reader's senses? Are the descriptions vivid and evocative?
        9. **Thematic Resonance:** What themes does the world-building explore? How effectively are these themes woven into the fabric of the world?
        10. **Reader Engagement:** How well might this world captivate and maintain reader interest? Are there elements that could be particularly appealing or potentially off-putting to the target audience?

        Please provide your feedback in a structured and organized manner, highlighting both strengths and weaknesses.
        Be specific in your suggestions for improvement, and feel free to recommend new directions or plot threads that could enhance the world.

        In your critique:
        * Acknowledge intentional ambiguities or complexities that may serve the author's creative vision
        * Consider both traditional and non-traditional storytelling approaches
        * Provide examples from the text to support your points
        * Offer constructive criticism that respects the author's creative choices while suggesting potential enhancements
        * Balance pointing out areas for improvement with recognizing and expanding on the world's strengths

        Remember, the goal is to help refine and enhance the world-building while respecting the author's unique vision and creative intentions.
        </task>

        Genre: {self.primary_genre}

        Premise: {self.premise}

        Title: {self.world_bible.working_title}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        <events>
        ### Events:
        {self.world_bible.retrieve_narrative_block("events")}
        </events>

        <cultures>
        ### Cultures:
        {self.world_bible.retrieve_narrative_block("cultures")}
        </cultures>
        """

        critique_feedback = self.llm.generate_inference(critique_prompt).strip()
        logger.info(f"{caller_name}: Critique: {critique_feedback}")
        self.world_bible.add_narrative(critique_feedback, "critique")

        logger.info(f"{caller_name}: Generated CRITIQUE.")
        return critique_feedback

    def _09_title(self):
        """Generates a working title and adds it to the World Bible."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating WORKING TITLE...")

        working_title_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Suggest a captivating WORKING TITLE for a story with the following world bible and story elements:

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        <events>
        ### Events:
        {self.world_bible.retrieve_narrative_block("events")}
        </events>

        <cultures>
        ### Cultures:
        {self.world_bible.retrieve_narrative_block("cultures")}
        </cultures>

        The WORKING TITLE should be concise, intriguing, and relevant to the story's themes and genre.

        Because we will need to parse this to feed into a program, make in a single line and do not include any
        additional words or characters before or after your WORKING TITLE. Respond with only your WORKING TITLE.
        Do not include the words 'WORKING TITLE:' nor any quotation marks, nor any colons, etc. Only the title!
        """
        self.working_title = self.llm.generate_inference(working_title_prompt).strip()
        self.world_bible.add_working_title(self.working_title)
        logger.info(f"{caller_name}: WORKING TITLE generated: {self.working_title}")
        return self.working_title

    def _08_cultures(self):
        """Generates cultures and adds them to the World Bible."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating CULTURES...")

        cultures_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Your task is to develop a captivating narrative that introduces at least two
        unique and well-defined cultures within a world shaped by the following elements:

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        <events>
        ### Events:
        {self.world_bible.retrieve_narrative_block("events")}
        </events>

        Consider these aspects when crafting the cultures:

        *   **Values and Beliefs:** What are the core principles that guide these cultures? How do they view the world, their place in it, and the concept of interconnected souls?
        *   **Customs and Traditions:** What are their unique practices, rituals, or ceremonies?  How do they celebrate, mourn, or mark significant life events?
        *   **Social Structure:** How is their society organized? Are there hierarchies, classes, or governing bodies?
        *   **Art and Expression:**  What forms of art, music, literature, or storytelling are prevalent in each culture?
        *   **Relationship to World Elements:** How have locations, items, and past events influenced the development of these cultures?
        *   **Conflict Potential:** How might the values or beliefs of these cultures lead to conflict or tension within the story?

        Format your response as a descriptive narrative, perhaps through the perspective of a character experiencing these cultures for the first time.
        Make sure to showcase the unique characteristics of each culture in a vivid and compelling way.

        Write in a style inspired by: {self.author_style}
        """

        cultures_narrative = self.llm.generate_inference(cultures_prompt).strip()
        logger.info(f"{caller_name}: Cultures narrative: {cultures_narrative}")
        self.world_bible.add_narrative(cultures_narrative, "cultures")

        logger.info(f"{caller_name}: Generated CULTURES.")
        return cultures_narrative

    def _07_events(self):
        """Generates events and adds them to the World Bible."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating EVENTS...")

        events_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Your task is to craft a narrative that introduces at least two significant EVENTS
        that have occurred or will occur in a story world with the following elements:

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        <items>
        ### Items:
        {self.world_bible.retrieve_narrative_block("items")}
        </items>

        Consider the following when designing the EVENTS:

        *   **Impact on Characters:** How do these events affect the characters or push them along their individual arcs?
            For instance, a character with a fall-rise arc might experience a significant setback.
        *   **Connection to Locations/Items:** Do the events involve any of the previously generated locations or items?
        *   **Contribution to Conflict:** How do these events contribute to the central conflict or create new challenges?
        *   **Emotional Resonance:** Describe the events in a way that evokes strong emotions in the reader.
        *   **Show, Don't Tell:** Instead of describing the EVENT, create a scene that shows a character experiencing it. Use vivid sensory details and dialogue to bring it to life.

        Format your response as a compelling narrative that seamlessly integrates the EVENTS into the story world.
        Be detailed and vivid in your descriptions!

        Write in a style inspired by: {self.author_style}
        """

        events_narrative = self.llm.generate_inference(events_prompt).strip()
        logger.info(f"{caller_name}: Events narrative: {events_narrative}")
        self.world_bible.add_narrative(events_narrative, "events")

        logger.info(f"{caller_name}: Generated EVENTS.")
        return events_narrative

    def _06_items(self):
        """Generates items and adds them to the World Bible."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating ITEMS...")

        items_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Your task is to imagine and describe at least two unique and
        compelling ITEMS that would exist in a story with the following elements:

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        <locations>
        ### Locations:
        {self.world_bible.retrieve_narrative_block("locations")}
        </locations>

        Consider these aspects when creating the ITEMS:

        *   **Connection to Characters:** How might these ITEMS be connected to specific characters or their archetypes?
            For example, a warrior might have a special sword, or a mage a powerful staff.
        *   **Function and Purpose:** What are the ITEMS used for? Do they have special abilities or significance?
        *   **Visual Description:** Describe the ITEMS with vivid details, making them visually interesting and unique.
        *   **Impact on Story:** How might these ITEMS influence events, character arcs, or the overall plot?
        *   **Tie Narrative to Conflict:** Write a narrative that showcases how a character's interaction with ITEM contributes to the central conflict of the story.

        Format your response as a narrative that introduces the ITEMS naturally, perhaps through a character
        discovering or interacting with them. Be creative and detailed in your descriptions!

        Write in a style inspired by: {self.author_style}
        """

        items_narrative = self.llm.generate_inference(items_prompt).strip()
        logger.info(f"{caller_name}: Items narrative: {items_narrative}")
        self.world_bible.add_narrative(items_narrative, "items")

        logger.info(f"{caller_name}: Generated ITEMS.")
        return items_narrative

    def _05_locations(self):
        """Generates locations and adds them to the World Bible."""
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating LOCATIONS...")

        locations_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Your task is to weave a narrative describing at least two distinct
        LOCATIONS for a story with the following genres, premise, and characters:

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        <rules>
        ### Rules:
        {self.world_bible.retrieve_narrative_block("rules")}
        </rules>

        Consider the following when creating the LOCATIONS:

        *   **Relevance to Premise:** How do the LOCATIONS reflect the unique aspects of the world (e.g., interconnected souls)?
        *   **Character Arcs:** How can each location play a role in a character's journey or their specific arc (rise, fall, etc.)?
        *   **Potential for Conflict:** Can these LOCATIONS serve as settings for future conflicts or challenges?
        *   **Visual Distinctiveness:** Describe each location with vivid imagery, making them unique and memorable.
        *   **Show, Don't Tell:** Instead of describing the LOCATION, create a scene that shows a character experiencing it. Use vivid sensory details and dialogue to bring it to life.
        *   **Tie Narrative to Conflict:** Write a narrative that showcases how a character's interaction with LOCATION contributes to the central conflict of the story.

        Format your response as a compelling story that introduces the LOCATIONS organically.
        Be creative and imaginative!

        Write in a style inspired by: {self.author_style}
        """

        locations_narrative = self.llm.generate_inference(locations_prompt).strip()
        logger.info(f"{caller_name}: Locations narrative: {locations_narrative}")
        self.world_bible.add_narrative(locations_narrative, "locations")

        logger.info(f"{caller_name}: Generated LOCATIONS.")
        return locations_narrative

    def _04_rules(self):
        """
        "...Ensure that the world is internally consistent and believable, with a clear system of rules and dynamics..."
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating WORLD RULES...")

        world_rules_prompt = f"""
        {StoryWriter.AUTHOR_PERSONA}

        Your task is to develop a set of rules that govern the world in which the story takes place.
        These rules should be clear, consistent, and believable.
        The rules should be relevant to the genres, premise, and the World Bible elements.

        Consider these aspects:
        - Magic Systems (if applicable): Define the rules of magic in the world, including its limits and potential consequences.
        - Technology Levels: Describe the technological advancements or limitations of the world.
        - Social Norms: Explain any unique social customs, laws, or expectations that shape the society.
        - Political Systems: Describe the governing structures, ideologies, and power dynamics in the world.
        - Cultural Differences: Explain any significant cultural variations or conflicts within the world.

        Format your response as a list of rules, with each rule clearly described:
        - Rule 1: <Detailed Description of the rule>
        - Rule 2: <Detailed Description of the rule>
        - Rule 3: <Detailed Description of the rule>

        Genre: {self.primary_genre}

        Premise: {self.premise}

        <characters>
        ### Characters:
        {self.world_bible.characters}
        </characters>

        <goals>
        ### Goals:
        {self.world_bible.retrieve_narrative_block("goals")}
        </goals>

        <conflicts>
        ### Conflicts:
        {self.world_bible.retrieve_narrative_block("conflicts")}
        </conflicts>

        <relationships>
        ### Relationships:
        {self.world_bible.retrieve_narrative_block("relationships")}
        </relationships>

        Write in a style inspired by: {self.author_style}
        """

        rules_narrative = self.llm.generate_inference(world_rules_prompt).strip()
        logger.info(f"{caller_name}: World Rules: {rules_narrative}")

        # Add narrative to WorldBible
        self.world_bible.add_narrative(rules_narrative, "rules")

        logger.info(f"{caller_name}: Generated WORLD RULES.")
        return rules_narrative

    def _03_relationships(self):
        """
        Generates relationships between elements and adds them to the World Bible.

        Purpose:
            Connecting elements within the World Bible is crucial for laying a solid foundation for a compelling narrative.
            This involves weaving together the characters, their arcs, the story premise, and the world elements
            (locations, events, items, cultures) in a meaningful way.
            """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating RELATIONSHIPS between characters and world elements...")

        relationships_prompt = f"""
            {StoryWriter.AUTHOR_PERSONA}

            Given the following genres, premise and characters, your task is to weave a narrative describing the relationships between all the characters.
            Try to follow as closely to the material we give you but with the following caveats.

            Caveat 1: The characters are complex, multi-faceted and nuanced. Some of their attributes might conflict. Feel free to emphasize, de-emphasize,
            eliminate or reconfigure their attributes accordingly to best fit with the narrative you are trying to express.

            Caveat 2: Similarly, the premise might also be overly convoluted. Feel free to simplify the premise in the context of your narrative response as well.
            In fact, if it helps you build a more compelling narrative, feel free to rephrase the premise into a modified or simplified version using the same format:
            'In a world where [world_essence], [character_spark] must [conflict_catalyst] to [theme_resonance].' Where you can modify any or all of those variables.

            Genre: {self.primary_genre}

            Premise: {self.premise}

            <characters>
            ### Characters:
            {self.world_bible.characters}
            </characters>

            <goals>
            ### Goals:
            {self.world_bible.retrieve_narrative_block("goals")}
            </goals>

            <conflicts>
            ### Conflicts:
            {self.world_bible.retrieve_narrative_block("conflicts")}
            </conflicts>

            Consider the following when weaving the relationships:

            Format your response as a compelling story with a few paragraphs. Include as much detail as possible.
            To define the relationships, use descriptive language like 'friend', 'enemy', 'lover', 'mentor',
            'family', 'owns', 'located in', 'involved in', and other similar relative phrases appropriate to the genres.

            Make every relationship advance the story. Be creative and imaginative!

            Write in a style inspired by: {self.author_style}
            """

        relationships = self.llm.generate_inference(relationships_prompt).strip()
        logger.info(f"{caller_name}: Relationships: {relationships}")

        # Add narrative to WorldBible
        self.world_bible.add_narrative(relationships, "relationships")

        logger.info(f"{caller_name}: Generated RELATIONSHIPS between characters and world elements.")
        return relationships

    def _02_characters(self) -> None:
        """
        Generates an initial starting main cast of 3 random characters.

        Returns:
            List[Character]: List of generated characters.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating {self.num_main_characters} random CAST...")

        # Generate characters
        # @TODO: use self.num_supporting_characters to generate supporting cast
        # @TODO: instruct LLM prompt to only use characters provided either main or supporting and never generate their own
        # @TODO: instruct LLM prompt to stick to the character descriptions provided in detail and don't
        #        invent their own details like occupation unless they have to to make the story work
        # @TODO: add similar entropy loss guardrails for other world elements: i.e.,
        #        items, events, locations, etc. add corporations? using "fake" library
        characters = []
        for i in range(1, self.num_main_characters + 1,):
            logger.info(f"{caller_name}: Generating CHARACTER {i}...")
            gender = "male" if i == 1 else "female" if i == 2 else None
            logger.info(f"{caller_name}:   - GENDER: {gender}")
            character = Character(gender=gender, genres=self.genres,)
            logger.info(f"{caller_name}: Character {i} generated.")
            characters.append(character)
            logger.info(f"{caller_name}: Adding CHARACTER {i} to CAST...")
            # populate world bible
            serializable_character = character.to_dict()
            # serializable_character = vars(character)
            self.world_bible.add_character(serializable_character)

            logger.info(f"{caller_name}: Character {i}: {json.dumps(serializable_character['full_name'])} added to WORLD BIBLE.")
            logger.info(f"{caller_name}:   - Details: {json.dumps(serializable_character)}")

        logger.info(f"{caller_name}: {self.num_main_characters} CHARACTERS successfully generated.")
        logger.info(f"{caller_name}: Generated CAST of {self.num_main_characters} CHARACTERS: {yaml.dump(self.world_bible.characters, default_flow_style=False)}")
        self.characters = characters
        return characters

    def _01_premise(self) -> None:
        """
        Generates a random story premise using PremiseGenerator
        class and adds premise to WorldBible as description.
        """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: Generating random premise...")

        plot_premise = self.premise_generator.get_premise(
            genres=self.genres, primary_genre=self.primary_genre,
            world_bible=self.world_bible, author_style=self.author_style,)

        logger.info(f"{caller_name}: Story premise generated.")
        logger.info(f"{caller_name}: Premise: '{plot_premise}'")

        self.premise = plot_premise
        self.world_bible.add_premise(self.premise)
        return plot_premise

    def run(self):
        """
            Runs the story generation process.

            Returns:
                The generated story text.

            @TODO:
            - Implement a mechanism to control chapter length and pacing
            - Incorporate feedback loops for refinement
            - Utilize dynamic prompts based on the chapter outline
            - Experiment with randomly assigning roles

            @TODO:
            important gist ids:
            81ce2662c3044ad9a9dc4a61f1355a39 - series bible index
            2fc92b5c01cf23a0ed573ff26b864ae7 - story_elements.yaml
            16 MBTI personality definitions
            """
        caller_name = get_caller_name()
        logger.info(f"{caller_name}: SPINNING UP a new story by StoryWriter...")

        # generate story elements
        #           --- generate premise ---
        self._01_premise()       # premise
        #           --- build world ---
        self._02_characters()    # characters
        self._03_relationships() # relationships
        self._04_rules()         # rules
        self._05_locations()     # locations
        self._06_items()         # items
        self._07_events()        # events
        self._08_cultures()      # cultures
        #           --- plan plot ---
        self._09_title()         # title
        self._10_critique()      # feedback
        self._11_revision()      # revision
        self._12_save()          # -save-
        self._13_outline()       # outline
        self._14_1pager()        # 1-pager
        #           --- write story ---
        # self._15_book()        # book      # @TODO: maybe separate module

        logger.info(f"{caller_name}: FINISHED")
        return

writer = StoryWriter()
writer.run()


INFO:ai_bookwriter.utils:_set_genres: Setting GENRES...
INFO:ai_bookwriter.utils:_set_genres: Randomly selected GENRE: romance
INFO:ai_bookwriter.utils:__init__: Setting up Gemini model...
INFO:ai_bookwriter.utils:__init__: Gemini model configured: gemini-1.5-flash-latest
INFO:ai_bookwriter.utils:__init__: Initializing GithubManager...
INFO:ai_bookwriter.utils:__init__: GithubManager initialized.
INFO:ai_bookwriter.utils:__init__: Initializing GithubManager...
INFO:ai_bookwriter.utils:__init__: GithubManager initialized.
INFO:ai_bookwriter.utils:__init__: Setting up Gemini model...
INFO:ai_bookwriter.utils:__init__: Gemini model configured: gemini-1.5-flash-latest
INFO:ai_bookwriter.utils:run: SPINNING UP a new story by StoryWriter...
INFO:ai_bookwriter.utils:_01_premise: Generating random premise...
INFO:ai_bookwriter.utils:get_premise: Beginning to select final PREMISE from a list of ...
INFO:ai_bookwriter.utils:generate_premises: Generating premises...
INFO:ai_bookwriter.utils:__ini

KeyboardInterrupt: 

In [5]:
import yaml
from enum import Enum

def extract_genre_data(genres: Enum) -> Dict:
    """
    Extracts genre data from an Enum and creates a dictionary.

    Args:
        genres (Enum): The Genre enum containing the genre data.

    Returns:
        Dict: A dictionary mapping genre names to their attributes.
    """
    genre_data = {}
    for genre in genres:
        # Check if the genre value is a dictionary
        if isinstance(genre.value, dict):
            genre_data[genre.name.lower()] = {
                "name": genre.value.get("name", None),
                "description": genre.value.get("description", None),
                "total_word_count": genre.value.get("total_word_count", None),
                "beats": genre.value.get("beats", None),
                "saving_the_cat_examples": genre.value.get("saving_the_cat_examples", None),
                "flexibility_options": genre.value.get("flexibility_options", None),
                "theme_development": genre.value.get("theme_development", None),
                "character_development_tools": genre.value.get("character_development_tools", None)
            }
        else:
            # If it's not a dictionary, assume it's the name
            genre_data[genre.name.lower()] = {
                "name": genre.value
            }
    return genre_data

# Example usage
genre_data_dict = extract_genre_data(Genre)
print(genre_data_dict)
print(yaml.dump(genre_data_dict, indent=2))

{'romance': {'name': 'Romance', 'description': None, 'total_word_count': 53125, 'beats': [{'name': 'Intro H1', 'chapter': 1, 'act': 1, 'word_count': 1500, 'example': 'Meet Corrina', 'prompts': ['How can you concisely introduce the first main character with just enough detail about their personality,\n                     background, and current situation to immediately hook the reader into feeling a sense of personal empathy\n                     and a strong emotional connection with this character?', "How can you describe the first main character's personal traits in a way that will become relevant to the upcoming conflict?", "What are the first main character's most significant flaws and desires?", 'What immediate goal or desire of the first character could drive them and lead to a central conflict?', "How can you hint at the first character's flaws that will play into the central conflict?", "What action or scenario would best reveal the first main character's personality and motiv

In [6]:
plot_structure = Genre.SAVE_THE_CAT.value['beats']
print(yaml.dump(plot_structure))

- act: 1
  chapter: 1
  character_focus: Protagonist's initial state
  example: Introduce the protagonist in their 'before' state
  name: Opening Image
  prompts:
  - How can you visually represent the protagonist's world before the story begins?
  - What symbolic elements can you include to foreshadow the journey ahead?
  - How does the opening reflect the story's genre and tone?
  screenplay_page: 1
  theme_connection: Subtle introduction of thematic elements
  word_count: 1500
- act: 1
  chapter: 2
  character_focus: Protagonist's worldview
  example: A character states the theme in a way the protagonist doesn't yet understand
  name: Theme Stated
  prompts:
  - How can you subtly introduce the story's theme through dialogue or action?
  - What misunderstanding of the theme does the protagonist currently have?
  - How can the theme be presented in a way that's natural to the story's world?
  screenplay_page: 5
  theme_connection: Explicit statement of theme, often misunderstood
  wo

In [7]:
print(Genre.list_all_genres_names())

['romance', 'save_the_cat', 'hero_journey', 'three_act_structure', 'seven_basic_plots', 'mystery_detective', 'crime', 'thriller', 'mystery', 'fantasy', 'sci_fi', 'young_adult', 'horror', 'historical_fiction', 'childrens', 'inspirational', 'religious', 'erotica', 'action', 'adventure', 'comedy', 'drama', 'poetry', 'western', 'dystopian', 'paranormal', 'genre_to_authors', 'genre_to_books']


## Revised Development Plan

### Phase 1: Premise Generation

1.  **Random Premise Generation:**
    -   Implement a function `generate_random_premise` within `StoryWriter` to generate premises based on the provided template and random lists (World Essence, Character Spark, Conflict Catalyst, Thematic Resonance).
    -   Thoroughly test this function to ensure it produces diverse and engaging premises.

### Phase 2: Character Generation

1.  **Random Character Generation:**
    -   Develop a function `generate_random_characters` to create characters with random attributes (name, gender, personality, aspirations, fears, past events, archetype/s).
    -   Integrate this with `StoryWriter` to generate characters after the premise.

### Phase 3: Dynamic Questions & Answers

1.  **Question Prompt Generation:**
    -   Create a function `generate_question_prompts` that dynamically generates prompts for questions related to the premise and characters.
    -   This function should leverage the generated premise, characters, and the 'Save the Cat' concept from the `Genre` class.

2.  **Prompt for Answers:**
    -   Develop a mechanism to present the generated question prompts to the user (perhaps through Colab forms) and collect their answers.

### Phase 4: One-Pager & Refinement

1.  **One-Pager Generation:**
    -   Implement a function `generate_one_pager` that summarizes the story idea using the generated premise, characters, answers to dynamic questions, and insights from the selected genre.

2.  **Refinement Loop:**
    -   Provide a mechanism for the user to review and refine the one-pager, incorporating their feedback.

### Phase 5: World Bible & Series Bible

1.  **World Bible Generation:**
    -   Develop a function `generate_world_bible` that creates a detailed World Bible based on the refined one-pager, including elements like characters, locations, items, events, and cultures.

2.  **Series Bible Generation:**
    -   Implement a function `generate_series_bible` to create a modular Series Bible, including character profiles, location dossiers, a plot outline, and chapter outlines incorporating the Karpman Drama Triangle dynamics.

### Future Phases: Chapter Writing, Feedback Loops, & Refinement

-   After establishing the core elements (premise, characters, questions/answers, one-pager, World Bible, Series Bible), we'll move on to:
    -   Implementing `ChapterWriter` for generating individual chapters.
    -   Integrating feedback loops at each stage.
    -   Refining the entire process based on testing and user feedback.

### Code Warehousing

-   The existing `StoryPremiseGenerator` and elements within the `Character` class that are not immediately required will be warehoused for potential future use.

### My Perspective

This revised plan aligns perfectly with your vision for a chronological workflow. It allows for iterative development, testing, and refinement at each stage.

Let's proceed with Phase 1.  I'll analyze the `StoryWriter` class and propose code changes to implement `generate_random_premise`.

## Process for Programmatic Story Generation

**I. Seed Generation and Initial Development**

  * **A. Define World Essence and Thematic Resonance:**
    - Utilize the "In a world where..." trope to establish a unique premise (World Essence).
    - Premise in code:
```python
premise = f"In a world where {world_essence}, {character_spark} must {conflict_catalyst} to {theme_resonance}.
```
    - Example:
    > "In a world where magic is a dying art, a young apprentice must master his craft to save the last remaining mages from extinction."
    - Randomly generate a thematic resonance to guide the story's underlying message.
    - Examples of World Essence:
       - "In a world where magic is fading from existence..."
       - "In a world where technology has become sentient..."
       - "In a world where memories can be traded like currency..."
    - Provide a comprehensive list of diverse world essences.
    - [These are some examples of "world essence." give me many, many more like these.](https://gist.github.com/grayox/2fc92b5c01cf23a0ed573ff26b864ae7)
    - Examples of Thematic Resonance:
      - "Explore the nature of consciousness."
      - "Challenge the concept of free will."
      - "Redefine the boundaries of humanity."

  * **B. Generate Core Characters and Relationships:**
    - Define character archetypes with specific attribute ranges (e.g., "reluctant hero," "cynical mentor").
    - Randomly select archetypes for the main character (MC) and two supporting characters (SCs), forming a "drama triangle."
    - Within each archetype, randomly generate character attributes:
      - Name and gender
      - Personality type (MBTI)
      - Aspirations, motivations, and fears
      - Past events that have shaped their worldview.
    - Determine the author whose storytelling style will inspire the narrative.

  * **C. Story Premise Generation:**
    - Craft a captivating story premise based on the chosen World Essence, Thematic Resonance, and core characters.

  * **D. Dynamic Question Generation and Narrative Deepening:**
    - Utilize dynamic prompts to ask questions about the premise and initial characters, providing the World Essence, Thematic Resonance, character descriptions, and the "Save the Cat" moment (see Genre class) as context.
      - Example prompts:
          ```
          "I will provide you with a captivating story premise, the intended thematic resonance, a set of core characters, and their 'Save the Cat' moments. Your task is to contribute to the art of crafting a compelling narrative by generating a list of insightful and thought-provoking questions. Your questions should guide the writer towards crafting a more detailed and emotionally resonant narrative. Your questions should help the writer deepen the audience's emotional connection with the main character/s. These questions should guide the writer in exploring the main characters' motivations, desires, fears, and vulnerabilities, ultimately inspiring a more nuanced and engaging story from the audience's perspective."
          ```
          ```
          "I have an intriguing story premise, a defined thematic resonance, and a set of well-developed characters to share. Please provide a list of thought-provoking questions designed to guide our writing team in crafting a compelling narrative and plot arc. Focus on elements that will enhance character development, conflict, and thematic depth."
          ```

  * **E. Standard Questions - Round 2:**
    - Answer the following questions to further develop the story:
      - **What is the McGuffin?** (What does the MC seek? ...their Goal?)
      - **Generate additional supporting characters.**
      - **Define the antagonist and their motivations.**
      - **What are the obstacles preventing the MC from achieving their Goal?**
      - **Why should we care about the plight of the MC?** (What is the "Save the Cat" story that makes us care about the MC and root for them?)
      - **Bullet point the major plot thrust points in the rising action portion of the storytelling.** (Insert genre-specific frameworks here, like the 20 Beat Framework for Romance Novels)
      - **Are there any unexpected plot twists or turns in the narrative arc?**

**II. Worldbuilding**

   * **A. World Bible Generation:**
      - Develop a World Bible detailing characters, locations, items, events, and cultures relevant to the story.
      - The World Bible is generated **after** the initial character generation, dynamic question and answer session, and Standard Questions - Round 2, leveraging the insights gained from these steps to inform worldbuilding.
      - Utilize dynamic prompts to gather information about the world, providing the World Essence, Thematic Resonance, character details, premise, and all previous question/answer sessions as context.
         - Example prompt (building upon the previous example):
            ```
            "Given the story premise, thematic resonance, core characters, their 'Save the Cat' moments, the insightful questions generated in the previous step, and answers to Standard Questions - Round 2, elaborate on the world in which this story takes place. Create a comprehensive World Bible that includes details about the following:
            - Characters: Expand on the existing characters and introduce new ones as needed.
            - Locations: Describe important locations, their significance, and their history.
            - Items: Detail unique objects, their functions, and their histories.
            - Events: Outline impactful events that have shaped the world and its inhabitants.
            - Cultures: Describe the customs, beliefs, values, and social structures of relevant cultures.
            Ensure the World Bible is consistent with the information provided and enriches the narrative possibilities."
            ```
      - Incorporate a feedback loop for refining the World Bible.
      - Structure of the World Bible:
        - Characters: Name, personality, backstory, relationships, etc.
        - Locations: Descriptions, significance, history, etc.
        - Items: Purpose, powers, history, etc.
        - Events: Impact on the world, significance to the plot, etc.
        - Cultures: Customs, beliefs, values, social structure, etc.

**III. Narrative Expansion**

   * **A. One-Pager:**
     - Generate a concise summary of the story, incorporating the World Essence, Thematic Resonance, key characters, major plot points, insights from the World Bible, and answers to Standard Questions - Round 2.
     - Feedback loop for refining the one-pager.

   * **B. Series Bible:**
     - Create a modular Series Bible:
        - **Character Profiles:** Dedicated modules for each character detailing background, relationships, motivations, arcs, etc.
        - **Location Dossiers:** Detailed modules for each location with information on history, culture, significance, etc.
        - **Plot Outline:** A separate document outlining the overall plot, referencing the character and location modules.
        - **Chapter Outlines:** Individual outlines for each chapter, building upon the plot outline, focusing on:
           - **Plot details:** What is happening.
           - **Character motivations:** Why it is happening.
           - **Karpman Drama Triangle dynamics** (Victim, Persecutor, Rescuer) within the chapter.
             - Experiment with randomly assigning these roles or having the AI model determine them.
             - Ensure drama triangle shifts contribute to conflict and character development.
             - Consider dynamic prompts to guide the exploration of these dynamics.
     - Feedback loops are incorporated at the chapter outline stage and after each chapter is written.
     - Utilize dynamic prompts to guide the writing process for each chapter, providing all previous elements as context.
        - Example prompt:
           ```
           "Given the current state of the story, the World Essence, Thematic Resonance, relevant character profiles and location dossiers, the upcoming beat, and the drama triangle dynamics for this chapter, write a compelling chapter that [specific instructions, such as character development, conflict escalation, or worldbuilding]."
           ```

**IV. Code Implementation**

  * **A. Modularity:**
    - Break down the code into logical modules (classes, functions) based on the process steps.
    - Ensure each module has a single, well-defined responsibility.

  * **B. Redundancy Elimination:**
    - Identify and eliminate duplicate code sections.
    - Create reusable functions or classes for common tasks.

  * **C. Reusability:**
    - Design the code to be adaptable to different world essences, genres, and story elements.
    - Utilize parameterization and configuration options.

  * **D. Google Colab Best Practices:**
    - Leverage Colab features for efficient code development and execution.
    - Utilize appropriate libraries for data manipulation, text generation, and user interaction.